In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# print(tf.__version__)
import cv2
import glob
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd  
from scipy.special import softmax

In [ ]:
one_hot_test_dataset = pd.read_pickle("./one_hot_test_dataset.pkl")  

In [ ]:
test_dataset = pd.read_pickle("./test_dataset.pkl")  

In [ ]:
one_hot_train_dataset = pd.read_pickle("./one_hot_train_dataset.pkl")  

In [ ]:
train_dataset = pd.read_pickle("./train_dataset.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.7.pkl")  

In [ ]:
# final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.pkl")  

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

3161
43


In [ ]:
min_train_dataset = pd.read_pickle("./new_min_train_dataset_0.7_v2.pkl") 

## Load database

Αρχικά κάνουμε extract το database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Παρατηρούμε ότι αποτελείται από ένα train και ένα validation set. Κάνουμε extract:

In [ ]:
!tar -xvf "/content/drive/MyDrive/Διπλωματικη/train_set.tar"

Streaming output truncated to the last 5000 lines.
train_set/annotations/392066_lnd.npy
train_set/annotations/83536_lnd.npy
train_set/annotations/26361_lnd.npy
train_set/annotations/313969_aro.npy
train_set/annotations/286825_exp.npy
train_set/annotations/392215_aro.npy
train_set/annotations/330087_lnd.npy
train_set/annotations/110001_val.npy
train_set/annotations/371482_aro.npy
train_set/annotations/27940_aro.npy
train_set/annotations/35729_exp.npy
train_set/annotations/156731_aro.npy
train_set/annotations/145081_aro.npy
train_set/annotations/93043_exp.npy
train_set/annotations/279607_lnd.npy
train_set/annotations/48472_lnd.npy
train_set/annotations/206512_exp.npy
train_set/annotations/84264_aro.npy
train_set/annotations/187769_lnd.npy
train_set/annotations/300707_exp.npy
train_set/annotations/123766_exp.npy
train_set/annotations/173216_aro.npy
train_set/annotations/262096_lnd.npy
train_set/annotations/194946_aro.npy
train_set/annotations/411301_aro.npy
train_set/annotations/43103_aro

In [ ]:
!tar -xvf "/content/drive/MyDrive/Διπλωματικη/val_set.tar"

Streaming output truncated to the last 5000 lines.
val_set/annotations/678_lnd.npy
val_set/annotations/1526_lnd.npy
val_set/annotations/5420_aro.npy
val_set/annotations/4628_exp.npy
val_set/annotations/5254_lnd.npy
val_set/annotations/4769_aro.npy
val_set/annotations/1994_lnd.npy
val_set/annotations/4840_lnd.npy
val_set/annotations/3546_val.npy
val_set/annotations/2606_lnd.npy
val_set/annotations/1282_val.npy
val_set/annotations/948_lnd.npy
val_set/annotations/2672_lnd.npy
val_set/annotations/1599_aro.npy
val_set/annotations/2177_exp.npy
val_set/annotations/1830_val.npy
val_set/annotations/4676_aro.npy
val_set/annotations/3228_aro.npy
val_set/annotations/5196_aro.npy
val_set/annotations/714_val.npy
val_set/annotations/2296_exp.npy
val_set/annotations/934_lnd.npy
val_set/annotations/1511_val.npy
val_set/annotations/2923_exp.npy
val_set/annotations/4198_aro.npy
val_set/annotations/886_lnd.npy
val_set/annotations/1924_exp.npy
val_set/annotations/1462_lnd.npy
val_set/annotations/812_exp.np

Αναφορικά με τα annotations δεδομένου του μεγάλου όγκου των δεδομένων δεν μπορούμε να δούμε μέσω του colab την δομή των αρχείων. Τυπώνοντας τα περιεχόμενα του directory βλέπουμε ότι αποτελείται από npy αρχεία που αφορούν τόσο τις τιμές valence,arousal όσο και τα expressions και τα landmarks.

In [ ]:
from glob import glob
glob("/content/train_set/annotations/*", recursive = True)

['/content/train_set/annotations/283130_lnd.npy',
 '/content/train_set/annotations/34377_aro.npy',
 '/content/train_set/annotations/357735_exp.npy',
 '/content/train_set/annotations/28988_exp.npy',
 '/content/train_set/annotations/275589_aro.npy',
 '/content/train_set/annotations/62980_aro.npy',
 '/content/train_set/annotations/220531_aro.npy',
 '/content/train_set/annotations/252988_val.npy',
 '/content/train_set/annotations/42554_aro.npy',
 '/content/train_set/annotations/380268_aro.npy',
 '/content/train_set/annotations/335304_exp.npy',
 '/content/train_set/annotations/224525_aro.npy',
 '/content/train_set/annotations/184895_exp.npy',
 '/content/train_set/annotations/208311_aro.npy',
 '/content/train_set/annotations/223132_lnd.npy',
 '/content/train_set/annotations/385947_val.npy',
 '/content/train_set/annotations/238841_lnd.npy',
 '/content/train_set/annotations/360686_lnd.npy',
 '/content/train_set/annotations/110130_exp.npy',
 '/content/train_set/annotations/74899_aro.npy',
 '/co

Θέλουμε 4 λίστες που περιέχουν τα paths για κάθε στοιχείο, μία για το image path, μία για το valence, μία για το arousal και μία για τα expressions.

## Για το train set

In [ ]:
import glob

In [ ]:
expressions = sorted(glob.glob("/content/train_set/annotations/*_exp.npy"))
print(len(expressions))
print(type(expressions))

287651
<class 'list'>


In [ ]:
expressions_list = []
valence_list = []
arousal_list = []

In [ ]:
for fname in expressions:
  expressions_list.append(int(np.load(fname)))

In [ ]:
valence = sorted(glob.glob("/content/train_set/annotations/*_val.npy"))
print(len(valence))
print(type(valence))

287651
<class 'list'>


In [ ]:
for fname in valence:
  valence_list.append(float(np.load(fname)))

In [ ]:
arousal = sorted(glob.glob("/content/train_set/annotations/*_aro.npy"))
print(len(arousal))
print(type(arousal))

287651
<class 'list'>


In [ ]:
for fname in arousal:
   arousal_list.append(float(np.load(fname)))

In [ ]:
print(len(expressions_list))
print(len(valence_list))
print(len(arousal_list))

287651
287651
287651


Επομένως έχουμε περίπου 280.000 εικόνες.

Τώρα θέλουμε να φτιάξουμε μία λίστα που περιέχει τα paths των εικόνων με την ίδια σειρά με την οποία είναι οι παραπάνω λίστες. Αυτό το πετυχαίνουμε με μία regex expression:

In [ ]:
import re
image_locations = []
for i in expressions:
    result=re.search('/content/train_set/annotations/(.*)_exp.npy', i)
    # print(result.group(1))
    number = result.group(1)
    # print(number)
    image_locations.append('/content/train_set/images/'+str(number)+'.jpg')

In [ ]:
print(len(image_locations))

287651


## Για το test set

In [ ]:
test_expressions = sorted(glob.glob("/content/val_set/annotations/*_exp.npy"))
print(len(test_expressions))
print(type(test_expressions))

3999
<class 'list'>


In [ ]:
test_expressions_list = []
test_arousal_list = []

In [ ]:
for fname in test_expressions:
  test_expressions_list.append(int(np.load(fname)))

In [ ]:
test_arousal = sorted(glob.glob("/content/val_set/annotations/*_aro.npy"))
print(len(test_arousal))
print(type(test_arousal))

3999
<class 'list'>


In [ ]:
test_image_locations = []
for i in test_arousal:
    result=re.search('/content/val_set/annotations/(.*)_aro.npy', i)
    number = result.group(1)
    test_image_locations.append('/content/val_set/images/'+str(number)+'.jpg')

In [ ]:
test_dataset = pd.DataFrame(list(zip(test_image_locations, test_expressions_list)),
              columns=['file_loc','expr'])
test_dataset

,file_loc,expr
0,/content/val_set/images/0.jpg,0
1,/content/val_set/images/1001.jpg,0
2,/content/val_set/images/1002.jpg,4
3,/content/val_set/images/1003.jpg,5
4,/content/val_set/images/1007.jpg,2
...,...,...
3994,/content/val_set/images/994.jpg,1
3995,/content/val_set/images/995.jpg,1
3996,/content/val_set/images/996.jpg,6
3997,/content/val_set/images/999.jpg,3


In [ ]:
one_hot_test_dataset = pd.get_dummies(test_dataset.expr, prefix='expr')
one_hot_test_dataset["file_loc"]=test_image_locations
one_hot_test_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
0,1,0,0,0,0,0,0,0,/content/val_set/images/0.jpg
1,1,0,0,0,0,0,0,0,/content/val_set/images/1001.jpg
2,0,0,0,0,1,0,0,0,/content/val_set/images/1002.jpg
3,0,0,0,0,0,1,0,0,/content/val_set/images/1003.jpg
4,0,0,1,0,0,0,0,0,/content/val_set/images/1007.jpg
...,...,...,...,...,...,...,...,...,...
3994,0,1,0,0,0,0,0,0,/content/val_set/images/994.jpg
3995,0,1,0,0,0,0,0,0,/content/val_set/images/995.jpg
3996,0,0,0,0,0,0,1,0,/content/val_set/images/996.jpg
3997,0,0,0,1,0,0,0,0,/content/val_set/images/999.jpg


In [ ]:
one_hot_test_dataset.to_pickle("./one_hot_test_dataset.pkl")  

In [ ]:
one_hot_test_dataset = pd.read_pickle("./one_hot_test_dataset.pkl")  

In [ ]:
test_dataset.to_pickle("./test_dataset.pkl")  

In [ ]:
test_dataset = pd.read_pickle("./test_dataset.pkl")  

Μαζί με valence και arousal:

In [ ]:
valence_list = []
arousal_list = []

In [ ]:
valence = sorted(glob.glob("/content/val_set/annotations/*_val.npy"))
print(len(valence))
print(type(valence))

3999
<class 'list'>


In [ ]:
for fname in valence:
  valence_list.append(float(np.load(fname)))

In [ ]:
arousal = sorted(glob.glob("/content/val_set/annotations/*_aro.npy"))
print(len(arousal))
print(type(arousal))

3999
<class 'list'>


In [ ]:
for fname in arousal:
   arousal_list.append(float(np.load(fname)))

In [ ]:
test_dataset_va = pd.DataFrame(list(zip(test_image_locations, test_expressions_list, valence_list, arousal_list)),
              columns=['file_loc','expr','valence','arousal'])
test_dataset_va

,file_loc,expr,valence,arousal
0,/content/val_set/images/0.jpg,0,-0.176846,-0.077640
1,/content/val_set/images/1001.jpg,0,-0.135501,0.004839
2,/content/val_set/images/1002.jpg,4,-0.114510,0.857748
3,/content/val_set/images/1003.jpg,5,-0.642857,0.484127
4,/content/val_set/images/1007.jpg,2,-0.796971,-0.229129
...,...,...,...,...
3994,/content/val_set/images/994.jpg,1,0.468254,0.119048
3995,/content/val_set/images/995.jpg,1,0.531746,0.015873
3996,/content/val_set/images/996.jpg,6,-0.416183,0.701703
3997,/content/val_set/images/999.jpg,3,0.176846,0.599551


In [ ]:
test_dataset_va.to_pickle("./test_dataset_va.pkl")  

## Simple label smoothing

Δημιουργούμε το train dataframe μας και μετατρέπουμε τα basic expressions σε one hot encoded μορφή.

In [ ]:
train_dataset = pd.DataFrame(list(zip(image_locations, expressions_list, valence_list, arousal_list)),
              columns=['file_loc','expr','valence','arousal'])
train_dataset

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
3,/content/train_set/images/100002.jpg,0,0.089659,0.114565
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
...,...,...,...,...
287646,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,3,0.406504,0.609756


In [ ]:
len(image_locations)

287651

In [ ]:
one_hot_train_dataset = pd.get_dummies(train_dataset.expr, prefix='expr')
one_hot_train_dataset["file_loc"]=image_locations
one_hot_train_dataset["valence"]=valence_list
one_hot_train_dataset["arousal"]=arousal_list
one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg,0.785714,-0.055556
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg,0.784476,-0.137627
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg,0.774293,0.091947
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg,0.089659,0.114565
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
one_hot_train_dataset.to_pickle("./one_hot_train_dataset.pkl")  

In [ ]:
one_hot_train_dataset = pd.read_pickle("./one_hot_train_dataset.pkl")  

In [ ]:
train_dataset.to_pickle("./train_dataset.pkl")  

In [ ]:
train_dataset = pd.read_pickle("./train_dataset.pkl")  

Τώρα θέλουμε να δημιουργήσουμε 8 clusters όπου για κάθε συναίσθημα έχουμε την μέση τιμή valence και arousal.

0: Neutral, 1: Happiness, 2: Sadness, 3: Surprise, 4: Fear, 5: Disgust, 6: Anger, 7: Contempt

In [ ]:
train_dataset = pd.read_pickle("./train_dataset.pkl")  

In [ ]:
mean_va = []

In [ ]:
neutral_mean_valence = train_dataset[train_dataset["expr"] == 0]["valence"].mean()
neutral_mean_arousal = train_dataset[train_dataset["expr"] == 0]["arousal"].mean()
mean_va.append((neutral_mean_valence,neutral_mean_arousal))

happy_mean_valence = train_dataset[train_dataset["expr"] == 1]["valence"].mean()
happy_mean_arousal = train_dataset[train_dataset["expr"] == 1]["arousal"].mean()
mean_va.append((happy_mean_valence,happy_mean_arousal))

sad_mean_valence = train_dataset[train_dataset["expr"] == 2]["valence"].mean()
sad_mean_arousal = train_dataset[train_dataset["expr"] == 2]["arousal"].mean()
mean_va.append((sad_mean_valence,sad_mean_arousal))

surprise_mean_valence = train_dataset[train_dataset["expr"] == 3]["valence"].mean()
surprise_mean_arousal = train_dataset[train_dataset["expr"] == 3]["arousal"].mean()
mean_va.append((surprise_mean_valence,surprise_mean_arousal))

fear_mean_valence = train_dataset[train_dataset["expr"] == 4]["valence"].mean()
fear_mean_arousal = train_dataset[train_dataset["expr"] == 4]["arousal"].mean()
mean_va.append((fear_mean_valence,fear_mean_arousal))

disgust_mean_valence = train_dataset[train_dataset["expr"] == 5]["valence"].mean()
disgust_mean_arousal = train_dataset[train_dataset["expr"] == 5]["arousal"].mean()
mean_va.append((disgust_mean_valence,disgust_mean_arousal))

anger_mean_valence = train_dataset[train_dataset["expr"] == 6]["valence"].mean()
anger_mean_arousal = train_dataset[train_dataset["expr"] == 6]["arousal"].mean()
mean_va.append((anger_mean_valence,anger_mean_arousal))

contempt_mean_valence = train_dataset[train_dataset["expr"] == 7]["valence"].mean()
contempt_mean_arousal = train_dataset[train_dataset["expr"] == 7]["arousal"].mean()
mean_va.append((contempt_mean_valence,contempt_mean_arousal))

In [ ]:
print("Mean valence,arousal:",mean_va)

Mean valence,arousal: [(-0.06271889776157277, -0.019574175715602216), (0.6643282289671539, 0.07028038624907934), (-0.6370063573392514, -0.2570043582489493), (0.1804439995542938, 0.6893279744258338), (-0.12574468561147695, 0.7661802158670429), (-0.69371899698396, 0.45734382349986846), (-0.45273569245960943, 0.566777641852745), (-0.5145737456826667, 0.5828403601946668)]


Με βάση αυτά τα 8 clusters θέλουμε για κάθε εικόνα να βρούμε την ευκλείδια απόσταση των τιμών valence και arousal της από τα 8 clusters. Για την ευκλείδια απόσταση χρησιμοποιούμε την συνάρτηση `numpy.linalg.norm(a-b)`.

In [ ]:
one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg,0.785714,-0.055556
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg,0.784476,-0.137627
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg,0.774293,0.091947
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg,0.089659,0.114565
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
list_of_distances = []
for j in range(one_hot_train_dataset.shape[0]): #for every image
  # print(j)
  distances = []
  for i in mean_va: #for every cluster`
    # print(i)
    a = np.array((float(one_hot_train_dataset.loc[j]["valence"]),float(one_hot_train_dataset.loc[j]["arousal"])))
    # print(a)
    i = np.array(i)
    # print(i)
    distance = np.linalg.norm(a-i)
    # print(distance)
    distances.append(distance)
  list_of_distances.append(distances)

In [ ]:
len(list_of_distances)

287651

In [ ]:
print(list_of_distances[:100])

[[0.8491955280720885, 0.1748405011561023, 1.4369115551722629, 0.9597933698921618, 1.2271946409006502, 1.565818575439517, 1.3860217547600933, 1.4485501798575644], [0.8553804206987465, 0.24012698336731142, 1.4264862585700755, 1.0240650307915375, 1.282719447099935, 1.593439904710724, 1.4236848918993505, 1.4854640550562213], [0.8444086430989531, 0.11207901685773346, 1.4537995514584794, 0.842330282910349, 1.124570078011788, 1.5128033060099466, 1.3156987746680977, 1.3791858049856491], [0.20300845700296868, 0.5763727135589644, 0.8161536408947903, 0.5818885962484529, 0.6862953202279338, 0.8550899824143909, 0.7061788734426278, 0.7644471116175379], [0.49807908334138257, 0.23465022194608212, 1.1105254299566085, 0.6880594619314757, 0.9082124518115505, 1.1964785247919099, 1.0242962049437059, 1.0858996293286742], [0.8972409748475645, 1.2623300239850068, 1.1297325116352044, 0.5412235626074996, 0.2257446733791946, 0.5153524583430708, 0.2889227570692755, 0.3045993284041953], [0.5941797874801882, 0.6923

Τώρα θέλουμε να δημιουργήσουμε μία νέα λίστα όπου κάθε στοιχείο κάθε υπολίστας θα υπολογίζεται σύμφωνα με τον τύπο 
`max(υπολίστας)/στοιχείο υπολίστας`

In [ ]:
list_of_distances[2]

[0.8444086430989531,
 0.11207901685773346,
 1.4537995514584794,
 0.842330282910349,
 1.124570078011788,
 1.5128033060099466,
 1.3156987746680977,
 1.3791858049856491]

In [ ]:
new_list_of_distances = []
for lists in list_of_distances: #for every list
  new_distances = []
  # print(lists)
  maxim = max(lists)
  # print(maxim)
  for element in lists:
    new_distances.append(maxim/element)
  new_list_of_distances.append(new_distances)

In [ ]:
print(new_list_of_distances[:100])

[[1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338, 1.080955701233273], [1.8628435560976115, 6.635821940399388, 1.1170383837472115, 1.5559948409517466, 1.2422357112564935, 1.0, 1.1192363659804676, 1.0726882951404813], [1.7915535545183503, 13.497649679870145, 1.0405858940403947, 1.795974021951384, 1.3452281325896074, 1.0, 1.149809770394877, 1.096881435801674], [4.212090447058994, 1.4835712418348428, 1.0477071222483463, 1.4695080603526511, 1.245950478185392, 1.0, 1.2108688245597328, 1.1185731091390438], [2.402185847205805, 5.09898739864323, 1.077398583154156, 1.7389173334427133, 1.317399384257917, 1.0, 1.1680981722056336, 1.1018315988666445], [1.4069018907651523, 1.0, 1.1173707147348335, 2.3323633913929585, 5.591848547693562, 2.44944989307623, 4.369091714303196, 4.144231146530724], [1.8800891539031201, 1.6134740933087202, 1.0, 7.10343872309813, 3.097337078659351, 1.3169850349106715, 1.8478533114576954, 1.6741574021262

Στην συνέχεια τις περνάμε από την συνάρτηση softmax.

In [ ]:
list_of_softmax = []
for i in new_list_of_distances:
  list_of_softmax.append(list(softmax(i)))

In [ ]:
print(list_of_softmax[:100])

[[0.0008126117694640895, 0.996561348641954, 0.00038225197722363975, 0.0006570637237013212, 0.00046049509308150605, 0.0003494529196855682, 0.0003978560851138351, 0.0003789197897761381], [0.008171948465130446, 0.9665043070381162, 0.0038763818214052077, 0.006012608346730891, 0.004393382914199992, 0.0034482395124759164, 0.0038849114102864233, 0.003708220491654876], [8.243100556642764e-06, 0.9999621223962459, 3.889998973752158e-06, 8.279619570289157e-06, 5.275382203423245e-06, 3.73528081229377e-06, 4.3389516783519444e-06, 4.11526995880356e-06], [0.7359421396069785, 0.048068750861717524, 0.031086330107182245, 0.047397482442725104, 0.037902289987640995, 0.029638110613584217, 0.036595668209637604, 0.033369228170534046], [0.05630374496742934, 0.835109289940594, 0.014968886101615255, 0.029005705071503244, 0.01902919898229305, 0.01385401668297976, 0.016390032505484577, 0.015339125748101269], [0.00923789715095388, 0.006149755887384009, 0.006915623329184503, 0.023307513567919443, 0.6068374335188694

In [ ]:
print(list_of_softmax[0])
print(type(list_of_softmax[0]))

[0.0008126117694640895, 0.996561348641954, 0.00038225197722363975, 0.0006570637237013212, 0.00046049509308150605, 0.0003494529196855682, 0.0003978560851138351, 0.0003789197897761381]
<class 'list'>


Τώρα θέλουμε να δημιουργήσουμε έναν γραμμικό συνδυασμό των πραγματικών labels και αυτών που προέκυψαν από label smoothing με βάση τον τύπο

`new_labels = λ*real_labels + (1-λ)*smoothed_labels`



In [ ]:
one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg,0.785714,-0.055556
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg,0.784476,-0.137627
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg,0.774293,0.091947
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg,0.089659,0.114565
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
labels = list(zip(one_hot_train_dataset["expr_0"],one_hot_train_dataset["expr_1"],one_hot_train_dataset["expr_2"],one_hot_train_dataset["expr_3"],one_hot_train_dataset["expr_4"],one_hot_train_dataset["expr_5"],one_hot_train_dataset["expr_6"],one_hot_train_dataset["expr_7"]))
labels = [list(x) for x in labels]

In [ ]:
print(labels[:100])

[[0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0

### λ = 0.7

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.7 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.3 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.00024378353083922684, 0.9989684045925862, 0.00011467559316709192, 0.00019711911711039638, 0.00013814852792445181, 0.00010483587590567046, 0.00011935682553415053, 0.00011367593693284144], [0.002451584539539134, 0.9899512921114348, 0.0011629145464215622, 0.0018037825040192674, 0.0013180148742599977, 0.001034471853742775, 0.001165473423085927, 0.0011124661474964628], [2.472930166992829e-06, 0.9999886367188737, 1.1669996921256473e-06, 2.483885871086747e-06, 1.5826146610269734e-06, 1.1205842436881309e-06, 1.3016855035055834e-06, 1.234580987641068e-06], [0.9207826418820935, 0.014420625258515257, 0.009325899032154673, 0.014219244732817531, 0.011370686996292298, 0.008891433184075264, 0.01097870046289128, 0.010010768451160213], [0.016891123490228803, 0.9505327869821781, 0.0044906658304845765, 0.008701711521450974, 0.005708759694687915, 0.004156205004893928, 0.004917009751645373, 0.00460173772443038], [0.002771369145286164, 0.0018449267662152026, 0.0020746869987553506, 0.006992254070375833, 

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,2.437835e-04,9.989684e-01,1.146756e-04,1.971191e-04,1.381485e-04,0.000105,0.000119,1.136759e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,2.451585e-03,9.899513e-01,1.162915e-03,1.803783e-03,1.318015e-03,0.001034,0.001165,1.112466e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,2.472930e-06,9.999886e-01,1.167000e-06,2.483886e-06,1.582615e-06,0.000001,0.000001,1.234581e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,9.207826e-01,1.442063e-02,9.325899e-03,1.421924e-02,1.137069e-02,0.008891,0.010979,1.001077e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,1.689112e-02,9.505328e-01,4.490666e-03,8.701712e-03,5.708760e-03,0.004156,0.004917,4.601738e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,0.300000,0.700000,1.726580e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.996000e-01,6.594273e-05,5.930794e-05,6.052851e-05,5.414606e-05,0.000049,0.000058,5.318166e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,4.193482e-11,1.805477e-11,5.534368e-11,3.028212e-11,5.246988e-11,0.300000,0.700000,1.541449e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,5.127475e-03,8.804876e-03,2.475755e-03,9.616210e-01,1.050463e-02,0.003089,0.004408,3.969684e-03,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./final_train_dataset_l_0.7.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.7.pkl")  

Στην συνέχεια εκπαιδεύουμε κατά τα γνωστά το νευρωνικό με βάση το βέλτιστο μοντέλο που είχαμε βρει για classification.

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 287651 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

3161
43


###### resnet lr = 10^-4 (best result)

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
3161/3161 [==============================] - ETA: 0s - loss: 0.9247 - accuracy: 0.7260
Epoch 1: val_accuracy improved from -inf to 0.43986, saving model to /content/weights0.4399.hdf5
3161/3161 [==============================] - 688s 215ms/step - loss: 0.9247 - accuracy: 0.7260 - val_loss: 1.6083 - val_accuracy: 0.4399
Epoch 2/5
3161/3161 [==============================] - ETA: 0s - loss: 0.8126 - accuracy: 0.7680
Epoch 2: val_accuracy improved from 0.43986 to 0.44261, saving model to /content/weights0.4426.hdf5
3161/3161 [==============================] - 679s 215ms/step - loss: 0.8126 - accuracy: 0.7680 - val_loss: 1.6153 - val_accuracy: 0.4426
Epoch 3/5
3161/3161 [==============================] - ETA: 0s - loss: 0.7458 - accuracy: 0.7956
Epoch 3: val_accuracy improved from 0.44261 to 0.44786, saving model to /content/weights0.4479.hdf5
3161/3161 [==============================] - 679s 215ms/step - loss: 0.7458 - accuracy: 0.7956 - val_loss: 1.6343 - val_accuracy: 0.4479
E

In [ ]:
y_pred = model_resnet.predict(test_generator)

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)
y_pred_max

array([0, 0, 3, ..., 6, 3, 2])

In [ ]:
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.48662165541385344
f1 score 0.4516158669280129
average accuracy 0.48651052104208414


### λ = 0

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 1 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.0008126117694640895, 0.996561348641954, 0.00038225197722363975, 0.0006570637237013212, 0.00046049509308150605, 0.0003494529196855682, 0.0003978560851138351, 0.0003789197897761381], [0.008171948465130446, 0.9665043070381162, 0.0038763818214052077, 0.006012608346730891, 0.004393382914199992, 0.0034482395124759164, 0.0038849114102864233, 0.003708220491654876], [8.243100556642764e-06, 0.9999621223962459, 3.889998973752158e-06, 8.279619570289157e-06, 5.275382203423245e-06, 3.73528081229377e-06, 4.3389516783519444e-06, 4.11526995880356e-06], [0.7359421396069785, 0.048068750861717524, 0.031086330107182245, 0.047397482442725104, 0.037902289987640995, 0.029638110613584217, 0.036595668209637604, 0.033369228170534046], [0.05630374496742934, 0.835109289940594, 0.014968886101615255, 0.029005705071503244, 0.01902919898229305, 0.01385401668297976, 0.016390032505484577, 0.015339125748101269], [0.00923789715095388, 0.006149755887384009, 0.006915623329184503, 0.023307513567919443, 0.6068374335188694

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,8.126118e-04,9.965613e-01,3.822520e-04,6.570637e-04,4.604951e-04,0.000349,3.978561e-04,3.789198e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,8.171948e-03,9.665043e-01,3.876382e-03,6.012608e-03,4.393383e-03,0.003448,3.884911e-03,3.708220e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,8.243101e-06,9.999621e-01,3.889999e-06,8.279620e-06,5.275382e-06,0.000004,4.338952e-06,4.115270e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,7.359421e-01,4.806875e-02,3.108633e-02,4.739748e-02,3.790229e-02,0.029638,3.659567e-02,3.336923e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,5.630374e-02,8.351093e-01,1.496889e-02,2.900571e-02,1.902920e-02,0.013854,1.639003e-02,1.533913e-02,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-10,7.438074e-11,2.133667e-10,1.252420e-10,2.200589e-10,1.000000,2.559365e-09,5.755267e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.986668e-01,2.198091e-04,1.976931e-04,2.017617e-04,1.804869e-04,0.000164,1.917808e-04,1.772722e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-10,6.018255e-11,1.844789e-10,1.009404e-10,1.748996e-10,1.000000,2.573964e-09,5.138162e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-02,2.934959e-02,8.252517e-03,8.720699e-01,3.501543e-02,0.010297,1.469209e-02,1.323228e-02,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./final_train_dataset_l_0.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.pkl")  

Στην συνέχεια εκπαιδεύουμε κατά τα γνωστά το νευρωνικό με βάση το βέλτιστο μοντέλο που είχαμε βρει για classification.

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 287651 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

3161
43


###### resnet lr = 10^-4

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


3161/3161 [==============================] - ETA: 0s - loss: 1.1510 - accuracy: 0.6783
Epoch 1: val_accuracy improved from -inf to 0.42886, saving model to /content/weights0.4289.hdf5
3161/3161 [==============================] - 690s 216ms/step - loss: 1.1510 - accuracy: 0.6783 - val_loss: 1.6133 - val_accuracy: 0.4289
Epoch 2/5
3161/3161 [==============================] - ETA: 0s - loss: 1.0594 - accuracy: 0.7144
Epoch 2: val_accuracy improved from 0.42886 to 0.42911, saving model to /content/weights0.4291.hdf5
3161/3161 [==============================] - 679s 215ms/step - loss: 1.0594 - accuracy: 0.7144 - val_loss: 1.6508 - val_accuracy: 0.4291
Epoch 3/5
3161/3161 [==============================] - ETA: 0s - loss: 1.0066 - accuracy: 0.7364
Epoch 3: val_accuracy improved from 0.42911 to 0.45586, saving model to /content/weights0.4559.hdf5
3161/3161 [==============================] - 681s 215ms/step - loss: 1.0066 - accuracy: 0.7364 - val_loss: 1.5475 - val_accuracy: 0.4559
Epoch 4/5
3

In [ ]:
y_pred = model_resnet.predict(test_generator)

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)

In [ ]:
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.43760940235058765
f1 score 0.3979493045961889
average accuracy 0.4375065130260521


###### resnet lr = 10^-3

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
3161/3161 [==============================] - ETA: 0s - loss: 1.1715 - accuracy: 0.6700
Epoch 1: val_accuracy improved from -inf to 0.37134, saving model to /content/weights0.3713.hdf5
3161/3161 [==============================] - 706s 222ms/step - loss: 1.1715 - accuracy: 0.6700 - val_loss: 1.7654 - val_accuracy: 0.3713
Epoch 2/5
3161/3161 [==============================] - ETA: 0s - loss: 1.1100 - accuracy: 0.6930
Epoch 2: val_accuracy improved from 0.37134 to 0.40135, saving model to /content/weights0.4014.hdf5
3161/3161 [==============================] - 679s 215ms/step - loss: 1.1100 - accuracy: 0.6930 - val_loss: 1.7244 - val_accuracy: 0.4014
Epoch 3/5
3161/3161 [==============================] - ETA: 0s - loss: 1.0826 - accuracy: 0.7045
Epoch 3: val_accuracy did not improve from 0.40135
3161/3161 [==============================] - 676s 214ms/step - loss: 1.0826 - accuracy: 0.7045 - val_loss: 1.6918 - val_accuracy: 0.3848
Epoch 4/5
3161/3161 [=============================

In [ ]:
y_pred = model_resnet.predict(test_generator)

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
y_true

array([0, 0, 4, ..., 6, 3, 0])

In [ ]:
y_pred_max

array([0, 0, 3, ..., 7, 3, 0])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.42285571392848215
f1 score 0.38483405236969814
average accuracy 0.4227545090180361


### λ = 0.9

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.9 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.1 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[8.126117694640896e-05, 0.9996561348641955, 3.8225197722363975e-05, 6.570637237013212e-05, 4.6049509308150605e-05, 3.494529196855682e-05, 3.9785608511383514e-05, 3.789197897761381e-05], [0.0008171948465130447, 0.9966504307038117, 0.0003876381821405208, 0.0006012608346730892, 0.00043933829141999924, 0.00034482395124759167, 0.00038849114102864235, 0.00037082204916548763], [8.243100556642764e-07, 0.9999962122396246, 3.8899989737521585e-07, 8.279619570289157e-07, 5.275382203423245e-07, 3.73528081229377e-07, 4.338951678351945e-07, 4.11526995880356e-07], [0.9735942139606979, 0.004806875086171752, 0.0031086330107182245, 0.00473974824427251, 0.0037902289987640997, 0.002963811061358422, 0.0036595668209637605, 0.003336922817053405], [0.005630374496742935, 0.9835109289940595, 0.0014968886101615255, 0.0029005705071503247, 0.001902919898229305, 0.001385401668297976, 0.0016390032505484578, 0.001533912574810127], [0.000923789715095388, 0.0006149755887384009, 0.0006915623329184504, 0.0023307513567919

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,8.126118e-05,9.996561e-01,3.822520e-05,6.570637e-05,4.604951e-05,3.494529e-05,3.978561e-05,3.789198e-05,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,8.171948e-04,9.966504e-01,3.876382e-04,6.012608e-04,4.393383e-04,3.448240e-04,3.884911e-04,3.708220e-04,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,8.243101e-07,9.999962e-01,3.889999e-07,8.279620e-07,5.275382e-07,3.735281e-07,4.338952e-07,4.115270e-07,0.774293,0.091947
3,/content/train_set/images/100002.jpg,9.735942e-01,4.806875e-03,3.108633e-03,4.739748e-03,3.790229e-03,2.963811e-03,3.659567e-03,3.336923e-03,0.089659,0.114565
4,/content/train_set/images/100004.jpg,5.630374e-03,9.835109e-01,1.496889e-03,2.900571e-03,1.902920e-03,1.385402e-03,1.639003e-03,1.533913e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-11,7.438074e-12,2.133667e-11,1.252420e-11,2.200589e-11,1.000000e-01,9.000000e-01,5.755267e-10,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.998667e-01,2.198091e-05,1.976931e-05,2.017617e-05,1.804869e-05,1.643640e-05,1.917808e-05,1.772722e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-11,6.018255e-12,1.844789e-11,1.009404e-11,1.748996e-11,1.000000e-01,9.000000e-01,5.138162e-10,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-03,2.934959e-03,8.252517e-04,9.872070e-01,3.501543e-03,1.029666e-03,1.469209e-03,1.323228e-03,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./final_train_dataset_l_0.9.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.9.pkl")  

### λ = 0.5

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.5 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.5 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.00040630588473204477, 0.998280674320977, 0.00019112598861181987, 0.0003285318618506606, 0.00023024754654075302, 0.0001747264598427841, 0.00019892804255691756, 0.00018945989488806906], [0.004085974232565223, 0.9832521535190581, 0.0019381909107026038, 0.0030063041733654457, 0.002196691457099996, 0.0017241197562379582, 0.0019424557051432117, 0.001854110245827438], [4.121550278321382e-06, 0.999981061198123, 1.944999486876079e-06, 4.139809785144578e-06, 2.6376911017116224e-06, 1.867640406146885e-06, 2.1694758391759722e-06, 2.05763497940178e-06], [0.8679710698034893, 0.024034375430858762, 0.015543165053591122, 0.023698741221362552, 0.018951144993820498, 0.014819055306792108, 0.018297834104818802, 0.016684614085267023], [0.02815187248371467, 0.9175546449702969, 0.0074844430508076275, 0.014502852535751622, 0.009514599491146525, 0.00692700834148988, 0.008195016252742288, 0.007669562874050634], [0.00461894857547694, 0.0030748779436920045, 0.0034578116645922513, 0.011653756783959721, 0.303418

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,4.063059e-04,9.982807e-01,1.911260e-04,3.285319e-04,2.302475e-04,0.000175,0.000199,1.894599e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,4.085974e-03,9.832522e-01,1.938191e-03,3.006304e-03,2.196691e-03,0.001724,0.001942,1.854110e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,4.121550e-06,9.999811e-01,1.944999e-06,4.139810e-06,2.637691e-06,0.000002,0.000002,2.057635e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,8.679711e-01,2.403438e-02,1.554317e-02,2.369874e-02,1.895114e-02,0.014819,0.018298,1.668461e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,2.815187e-02,9.175546e-01,7.484443e-03,1.450285e-02,9.514599e-03,0.006927,0.008195,7.669563e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,7.947356e-11,3.719037e-11,1.066833e-10,6.262100e-11,1.100294e-10,0.500000,0.500000,2.877633e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.993334e-01,1.099045e-04,9.884657e-05,1.008809e-04,9.024343e-05,0.000082,0.000096,8.863610e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6.989136e-11,3.009128e-11,9.223947e-11,5.047021e-11,8.744980e-11,0.500000,0.500000,2.569081e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,8.545792e-03,1.467479e-02,4.126259e-03,9.360349e-01,1.750772e-02,0.005148,0.007346,6.616139e-03,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./final_train_dataset_l_0.5.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.5.pkl")  

### λ = 0.3

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.3 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.7 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.0005688282386248627, 0.9975929440493678, 0.0002675763840565478, 0.00045994460659092484, 0.00032234656515705423, 0.0002446170437798977, 0.00027849925957968455, 0.00026524385284329666], [0.005720363925591312, 0.9765530149266812, 0.002713467274983645, 0.004208825842711624, 0.0030753680399399942, 0.002413767658733141, 0.002719437987200496, 0.002595754344158413], [5.770170389649935e-06, 0.9999734856773721, 2.7229992816265103e-06, 5.795733699202409e-06, 3.692767542396271e-06, 2.6146965686056387e-06, 3.037266174846361e-06, 2.8806889711624915e-06], [0.8151594977248848, 0.03364812560320227, 0.02176043107502757, 0.03317823770990757, 0.026531602991348695, 0.02074667742950895, 0.025616967746746322, 0.02335845971937383], [0.03941262147720054, 0.8845765029584158, 0.010478220271130679, 0.02030399355005227, 0.013320439287605134, 0.009697811678085831, 0.011473022753839202, 0.010737388023670887], [0.0064665280056677155, 0.004304829121168806, 0.004840936330429152, 0.01631525949754361, 0.4247862034632

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,5.688282e-04,9.975929e-01,2.675764e-04,4.599446e-04,3.223466e-04,0.000245,0.000278,2.652439e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,5.720364e-03,9.765530e-01,2.713467e-03,4.208826e-03,3.075368e-03,0.002414,0.002719,2.595754e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,5.770170e-06,9.999735e-01,2.722999e-06,5.795734e-06,3.692768e-06,0.000003,0.000003,2.880689e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,8.151595e-01,3.364813e-02,2.176043e-02,3.317824e-02,2.653160e-02,0.020747,0.025617,2.335846e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,3.941262e-02,8.845765e-01,1.047822e-02,2.030399e-02,1.332044e-02,0.009698,0.011473,1.073739e-02,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.112630e-10,5.206652e-11,1.493567e-10,8.766940e-11,1.540412e-10,0.700000,0.300000,4.028687e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.990668e-01,1.538664e-04,1.383852e-04,1.412332e-04,1.263408e-04,0.000115,0.000134,1.240905e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,9.784791e-11,4.212779e-11,1.291353e-10,7.065829e-11,1.224297e-10,0.700000,0.300000,3.596713e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.196411e-02,2.054471e-02,5.776762e-03,9.104489e-01,2.451080e-02,0.007208,0.010284,9.262595e-03,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./final_train_dataset_l_0.3.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./final_train_dataset_l_0.3.pkl")  

### Label smoothing χωρίς το contempt

Σε αυτή την περίπτωση θα δοκιμάσουμε να δημιουργήσουμε 7 μόνο clusters, αφαιρώντας δηλαδή το contempt για το οποίο έχουμε λίγες εικόνες.

In [ ]:
new_mean_va = []

In [ ]:
neutral_mean_valence = train_dataset[train_dataset["expr"] == 0]["valence"].mean()
neutral_mean_arousal = train_dataset[train_dataset["expr"] == 0]["arousal"].mean()
new_mean_va.append((neutral_mean_valence,neutral_mean_arousal))

happy_mean_valence = train_dataset[train_dataset["expr"] == 1]["valence"].mean()
happy_mean_arousal = train_dataset[train_dataset["expr"] == 1]["arousal"].mean()
new_mean_va.append((happy_mean_valence,happy_mean_arousal))

sad_mean_valence = train_dataset[train_dataset["expr"] == 2]["valence"].mean()
sad_mean_arousal = train_dataset[train_dataset["expr"] == 2]["arousal"].mean()
new_mean_va.append((sad_mean_valence,sad_mean_arousal))

surprise_mean_valence = train_dataset[train_dataset["expr"] == 3]["valence"].mean()
surprise_mean_arousal = train_dataset[train_dataset["expr"] == 3]["arousal"].mean()
new_mean_va.append((surprise_mean_valence,surprise_mean_arousal))

fear_mean_valence = train_dataset[train_dataset["expr"] == 4]["valence"].mean()
fear_mean_arousal = train_dataset[train_dataset["expr"] == 4]["arousal"].mean()
new_mean_va.append((fear_mean_valence,fear_mean_arousal))

disgust_mean_valence = train_dataset[train_dataset["expr"] == 5]["valence"].mean()
disgust_mean_arousal = train_dataset[train_dataset["expr"] == 5]["arousal"].mean()
new_mean_va.append((disgust_mean_valence,disgust_mean_arousal))

anger_mean_valence = train_dataset[train_dataset["expr"] == 6]["valence"].mean()
anger_mean_arousal = train_dataset[train_dataset["expr"] == 6]["arousal"].mean()
new_mean_va.append((anger_mean_valence,anger_mean_arousal))

In [ ]:
print("Mean valence,arousal:",new_mean_va)

Mean valence,arousal: [(-0.06271889776157277, -0.019574175715602216), (0.6643282289671539, 0.07028038624907934), (-0.6370063573392514, -0.2570043582489493), (0.1804439995542938, 0.6893279744258338), (-0.12574468561147695, 0.7661802158670429), (-0.69371899698396, 0.45734382349986846), (-0.45273569245960943, 0.566777641852745)]


Με βάση αυτά τα 7 clusters θέλουμε για κάθε εικόνα να βρούμε την ευκλείδια απόσταση των τιμών valence και arousal της από τα 7 clusters. Για την ευκλείδια απόσταση χρησιμοποιούμε την συνάρτηση `numpy.linalg.norm(a-b)`.

In [ ]:
one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg,0.785714,-0.055556
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg,0.784476,-0.137627
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg,0.774293,0.091947
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg,0.089659,0.114565
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
new_list_of_distances = []
for j in range(one_hot_train_dataset.shape[0]): #for every image
  distances = []
  for i in new_mean_va: #for every cluster`
    a = np.array((float(one_hot_train_dataset.loc[j]["valence"]),float(one_hot_train_dataset.loc[j]["arousal"])))
    i = np.array(i)
    distance = np.linalg.norm(a-i)
    distances.append(distance)
  new_list_of_distances.append(distances)

In [ ]:
print(new_list_of_distances[:100])

[[0.8491955280720885, 0.1748405011561023, 1.4369115551722629, 0.9597933698921618, 1.2271946409006502, 1.565818575439517, 1.3860217547600933], [0.8553804206987465, 0.24012698336731142, 1.4264862585700755, 1.0240650307915375, 1.282719447099935, 1.593439904710724, 1.4236848918993505], [0.8444086430989531, 0.11207901685773346, 1.4537995514584794, 0.842330282910349, 1.124570078011788, 1.5128033060099466, 1.3156987746680977], [0.20300845700296868, 0.5763727135589644, 0.8161536408947903, 0.5818885962484529, 0.6862953202279338, 0.8550899824143909, 0.7061788734426278], [0.49807908334138257, 0.23465022194608212, 1.1105254299566085, 0.6880594619314757, 0.9082124518115505, 1.1964785247919099, 1.0242962049437059], [0.8972409748475645, 1.2623300239850068, 1.1297325116352044, 0.5412235626074996, 0.2257446733791946, 0.5153524583430708, 0.2889227570692755], [0.5941797874801882, 0.6923637500860796, 1.1171109739099627, 0.15726340684512025, 0.3606681951431299, 0.8482336126057303, 0.6045452671937037], [0.7

Τώρα θέλουμε να δημιουργήσουμε μία νέα λίστα όπου κάθε στοιχείο κάθε υπολίστας θα υπολογίζεται σύμφωνα με τον τύπο 
`max(υπολίστας)/στοιχείο υπολίστας`

In [ ]:
list_of_distances[2]

[0.8444086430989531,
 0.11207901685773346,
 1.4537995514584794,
 0.842330282910349,
 1.124570078011788,
 1.5128033060099466,
 1.3156987746680977,
 1.3791858049856491]

In [ ]:
new_new_list_of_distances = []
for lists in  new_list_of_distances: #for every list
  new_distances = []
  # print(lists)
  maxim = max(lists)
  # print(maxim)
  for element in lists:
    new_distances.append(maxim/element)
    new_new_list_of_distances.append(new_distances)

In [ ]:
print( new_new_list_of_distances[:100])

[[1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8438846221839666, 8.95569713587993, 1.0897111724122786, 1.6314121607398118, 1.275933355030256, 1.0, 1.129721499725338], [1.8628435560976115, 6.635821940399388, 1.1170383837472115, 1.5559948409517466, 1.2422357112564935, 1.0, 1.1192363659804676], [1.862843556

Στην συνέχεια τις περνάμε από την συνάρτηση softmax.

In [ ]:
new_list_of_softmax = []
for i in new_new_list_of_distances:
   new_list_of_softmax.append(list(softmax(i)))

In [ ]:
print(new_list_of_softmax[:100])

[[0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391085994034, 0.0003823968749671131, 0.0006573127925264821, 0.0004606696499284128, 0.0003495853845059738, 0.00039800689780388013], [0.0008129198008641372, 0.9969391

Τώρα θέλουμε να δημιουργήσουμε έναν γραμμικό συνδυασμό των πραγματικών labels και αυτών που προέκυψαν από label smoothing με βάση τον τύπο

`new_labels = λ*real_labels + (1-λ)*smoothed_labels`



In [ ]:
labels = list(zip(one_hot_train_dataset["expr_0"],one_hot_train_dataset["expr_1"],one_hot_train_dataset["expr_2"],one_hot_train_dataset["expr_3"],one_hot_train_dataset["expr_4"],one_hot_train_dataset["expr_5"],one_hot_train_dataset["expr_6"],one_hot_train_dataset["expr_7"]))
labels = [list(x) for x in labels]

Θα χρησιμοποιήσουμε το λ που είχε τα καλύτερα αποτελέσματα, δηλαδή το 0.7

#### λ = 0.7

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.7 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.3 for j in new_list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.00024387594025924116, 0.9990817325798209, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.00011940206934116403], [0.00024387594025924116, 0.9990817325798209, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.00011940206934116403], [0.00024387594025924116, 0.9990817325798209, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.00011940206934116403], [0.7002438759402592, 0.299081732579821, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.00011940206934116403], [0.00024387594025924116, 0.9990817325798209, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.00011940206934116403], [0.00024387594025924116, 0.299081732579821, 0.00011471906249013393, 0.00019719383775794463, 0.00013820089497852385, 0.00010487561535179213, 0.7001194020693411], [0.0002438759

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]

In [ ]:
final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','valence','arousal'])
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,valence,arousal
0,/content/train_set/images/0.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.774293,0.091947
3,/content/train_set/images/100002.jpg,0.700244,0.299082,0.000115,0.000197,0.000138,0.000105,0.000119,0.089659,0.114565
4,/content/train_set/images/100004.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0.700065,0.299717,0.000029,0.000080,0.000044,0.000029,0.000035,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,0.000065,0.299717,0.000029,0.700080,0.000044,0.000029,0.000035,0.406504,0.609756


In [ ]:
final_train_dataset.to_pickle("./new_final_train_dataset_l_0.7.pkl")  

In [ ]:
final_train_dataset = pd.read_pickle("./new_final_train_dataset_l_0.7.pkl")  

## Data cleaning

In [ ]:
train_dataset

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
3,/content/train_set/images/100002.jpg,0,0.089659,0.114565
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
...,...,...,...,...
287646,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,3,0.406504,0.609756


Θέλουμε να εφαρμόσουμε μία μορφή data cleaning. Αρχικά παίρνουμε τα true labels και κρατάμε όσα είναι happy.

### happy

#### va > 0

In [ ]:
happy = train_dataset.loc[train_dataset['expr'] == 1]
happy

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
8,/content/train_set/images/100011.jpg,1,0.547619,0.000000
...,...,...,...,...
287639,/content/train_set/images/9998.jpg,1,0.394197,0.015768
287640,/content/train_set/images/99990.jpg,1,0.754936,0.116144
287644,/content/train_set/images/99995.jpg,1,0.431943,0.163295
287645,/content/train_set/images/99996.jpg,1,0.566529,0.026795


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι και οι δύο θετικές. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
happy_wrong_1 = happy.loc[(happy['valence'] <= 0) | (happy['arousal'] <= 0)]
happy_wrong_1

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
8,/content/train_set/images/100011.jpg,1,0.547619,0.000000
16,/content/train_set/images/100022.jpg,1,0.755632,-0.049900
22,/content/train_set/images/10002.jpg,1,0.781910,-0.183150
...,...,...,...,...
287594,/content/train_set/images/99914.jpg,1,0.491718,-0.125086
287600,/content/train_set/images/99920.jpg,1,0.753968,-0.134921
287601,/content/train_set/images/99921.jpg,1,0.531746,0.000000
287607,/content/train_set/images/99929.jpg,1,0.816477,-0.057944


Άρα έχουμε 37317 εικόνες που είναι annotated ως happy ενώ δεν θα έπρεπε.

#### va > 0.1

In [ ]:
happy = train_dataset.loc[train_dataset['expr'] == 1]
happy

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
8,/content/train_set/images/100011.jpg,1,0.547619,0.000000
...,...,...,...,...
287639,/content/train_set/images/9998.jpg,1,0.394197,0.015768
287640,/content/train_set/images/99990.jpg,1,0.754936,0.116144
287644,/content/train_set/images/99995.jpg,1,0.431943,0.163295
287645,/content/train_set/images/99996.jpg,1,0.566529,0.026795


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι και οι δύο θετικές. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
happy_wrong_2 = happy.loc[(happy['valence'] <= 0.1) | (happy['arousal'] <= 0.1)]
happy_wrong_2

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
8,/content/train_set/images/100011.jpg,1,0.547619,0.000000
...,...,...,...,...
287613,/content/train_set/images/99938.jpg,1,0.563632,0.079014
287618,/content/train_set/images/99951.jpg,1,0.717405,0.093575
287624,/content/train_set/images/99968.jpg,1,0.650743,0.042107
287639,/content/train_set/images/9998.jpg,1,0.394197,0.015768


Άρα έχουμε 75111 εικόνες που είναι annotated ως happy ενώ δεν θα έπρεπε.

### neutral

In [ ]:
neutral = train_dataset.loc[train_dataset['expr'] == 0]
neutral

,file_loc,expr,valence,arousal
3,/content/train_set/images/100002.jpg,0,0.089659,0.114565
10,/content/train_set/images/100013.jpg,0,-0.067751,-0.043554
13,/content/train_set/images/100018.jpg,0,0.300039,-0.101626
19,/content/train_set/images/100025.jpg,0,-0.095238,-0.079365
23,/content/train_set/images/100031.jpg,0,-0.047619,-0.071429
...,...,...,...,...
287636,/content/train_set/images/99982.jpg,0,-0.058072,-0.091947
287637,/content/train_set/images/99984.jpg,0,-0.079365,-0.055556
287642,/content/train_set/images/99993.jpg,0,-0.188734,-0.183895
287643,/content/train_set/images/99994.jpg,0,0.000000,0.000000


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι ανάμεσα στο -0.1 και στο 0.1. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
neutral_wrong = neutral.loc[((neutral['valence'] > 0.1) | (neutral['valence'] < -0.1)) | ((neutral['arousal'] > 0.1) | (neutral['arousal'] < -0.1))]
neutral_wrong

,file_loc,expr,valence,arousal
3,/content/train_set/images/100002.jpg,0,0.089659,0.114565
13,/content/train_set/images/100018.jpg,0,0.300039,-0.101626
45,/content/train_set/images/100056.jpg,0,-0.247577,-0.342394
52,/content/train_set/images/100064.jpg,0,-0.396825,0.144300
63,/content/train_set/images/100080.jpg,0,-0.416140,0.152760
...,...,...,...,...
287602,/content/train_set/images/99922.jpg,0,-0.034507,0.103520
287612,/content/train_set/images/99937.jpg,0,-0.228405,-0.332226
287625,/content/train_set/images/99969.jpg,0,-0.316543,-0.151390
287630,/content/train_set/images/99974.jpg,0,-0.378475,0.178915


Άρα έχουμε 31745 εικόνες που είναι annotated ως neutral ενώ δεν θα έπρεπε.

### sad

In [ ]:
sad = train_dataset.loc[train_dataset['expr'] == 2]
sad

,file_loc,expr,valence,arousal
17,/content/train_set/images/100023.jpg,2,-0.880952,-0.428571
29,/content/train_set/images/100039.jpg,2,-0.492063,-0.277778
36,/content/train_set/images/100045.jpg,2,-0.928571,-0.126984
43,/content/train_set/images/100052.jpg,2,-0.444444,-0.158730
47,/content/train_set/images/100058.jpg,2,-0.508130,-0.208091
...,...,...,...,...
287580,/content/train_set/images/99899.jpg,2,-0.759775,-0.348432
287629,/content/train_set/images/99973.jpg,2,-0.747508,-0.256091
287631,/content/train_set/images/99975.jpg,2,-0.849206,-0.357143
287632,/content/train_set/images/99977.jpg,2,-0.580720,-0.208091


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
sad_wrong = sad.loc[sad['valence'] >= 0]
sad_wrong

,file_loc,expr,valence,arousal
75609,/content/train_set/images/198507.jpg,2,0.000000,-0.150794
151507,/content/train_set/images/29709.jpg,2,0.000000,-0.174603
210069,/content/train_set/images/372823.jpg,2,0.880952,0.285714
215380,/content/train_set/images/37968.jpg,2,0.000000,0.349206
277288,/content/train_set/images/86432.jpg,2,0.000000,0.000000


Άρα έχουμε 5 εικόνες που είναι annotated ως sad ενώ δεν θα έπρεπε.

### surprise -> not deleted

In [ ]:
surprise = train_dataset.loc[train_dataset['expr'] == 3]
surprise

,file_loc,expr,valence,arousal
6,/content/train_set/images/100008.jpg,3,0.150966,0.534852
31,/content/train_set/images/100040.jpg,3,0.280681,0.692025
33,/content/train_set/images/100042.jpg,3,-0.048393,0.871080
62,/content/train_set/images/100077.jpg,3,0.211353,0.659938
86,/content/train_set/images/100113.jpg,3,0.329075,0.687185
...,...,...,...,...
287523,/content/train_set/images/99817.jpg,3,0.158562,0.649351
287533,/content/train_set/images/9982.jpg,3,0.145180,0.551684
287573,/content/train_set/images/9988.jpg,3,0.353656,0.647539
287604,/content/train_set/images/99925.jpg,3,-0.190476,0.920635


Θέλουμε να δούμε αν οι τιμές arousal είναι > 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
surprise_wrong = surprise.loc[surprise['arousal'] <= 0]
surprise_wrong

,file_loc,expr,valence,arousal
21583,/content/train_set/images/128202.jpg,3,-0.452381,-0.285714
33398,/content/train_set/images/143623.jpg,3,-0.444444,-0.246032
48877,/content/train_set/images/163672.jpg,3,-0.174603,-0.420635
61320,/content/train_set/images/179855.jpg,3,-0.759775,-0.033875
86022,/content/train_set/images/21205.jpg,3,-0.555556,-0.285714
96191,/content/train_set/images/225289.jpg,3,-0.857143,-0.309524
108426,/content/train_set/images/241081.jpg,3,-0.311462,-0.173034
158463,/content/train_set/images/306168.jpg,3,0.895277,-0.058072
164504,/content/train_set/images/314021.jpg,3,-0.493612,-0.014518
169518,/content/train_set/images/320443.jpg,3,-0.444444,-0.253968


Άρα έχουμε 20 εικόνες που είναι annotated ως surprise ενώ δεν θα έπρεπε.

### fear

In [ ]:
fear = train_dataset.loc[train_dataset['expr'] == 4]
fear

,file_loc,expr,valence,arousal
7,/content/train_set/images/100009.jpg,4,-0.126422,0.758533
85,/content/train_set/images/100112.jpg,4,-0.116144,0.754936
178,/content/train_set/images/10022.jpg,4,-0.183895,0.740418
251,/content/train_set/images/100315.jpg,4,-0.095238,0.642857
333,/content/train_set/images/100424.jpg,4,-0.069138,0.717310
...,...,...,...,...
287492,/content/train_set/images/99783.jpg,4,-0.139470,0.876669
287544,/content/train_set/images/99852.jpg,4,-0.091947,0.958188
287574,/content/train_set/images/99891.jpg,4,-0.028088,0.661136
287627,/content/train_set/images/99970.jpg,4,-0.095238,0.777778


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
fear_wrong = fear.loc[fear['valence'] >= 0]
fear_wrong

,file_loc,expr,valence,arousal
7825,/content/train_set/images/110226.jpg,4,0.000000,0.873016
14079,/content/train_set/images/118469.jpg,4,0.183895,0.783972
15060,/content/train_set/images/119726.jpg,4,0.000000,0.793651
24065,/content/train_set/images/131398.jpg,4,0.008627,0.543478
27482,/content/train_set/images/135832.jpg,4,0.271003,0.740418
34596,/content/train_set/images/145158.jpg,4,0.000000,0.785714
39339,/content/train_set/images/151263.jpg,4,0.000000,0.452899
44540,/content/train_set/images/158050.jpg,4,0.237330,0.788547
53565,/content/train_set/images/169861.jpg,4,0.009679,0.880759
82724,/content/train_set/images/207755.jpg,4,0.000000,0.785714


Άρα έχουμε 29 εικόνες που είναι annotated ως fear ενώ δεν θα έπρεπε.

### disgust

In [ ]:
disgust = train_dataset.loc[train_dataset['expr'] == 5]
disgust

,file_loc,expr,valence,arousal
35,/content/train_set/images/100044.jpg,5,-0.725900,0.391986
91,/content/train_set/images/100119.jpg,5,-0.841270,0.468254
99,/content/train_set/images/100128.jpg,5,-0.571429,0.492063
145,/content/train_set/images/100186.jpg,5,-0.761905,0.460317
174,/content/train_set/images/100223.jpg,5,-0.743187,0.502340
...,...,...,...,...
287201,/content/train_set/images/99421.jpg,5,-0.740418,0.474255
287359,/content/train_set/images/99612.jpg,5,-0.849206,0.436508
287480,/content/train_set/images/99770.jpg,5,-0.769841,0.452381
287493,/content/train_set/images/99784.jpg,5,-0.706349,0.428571


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
disgust_wrong = disgust.loc[disgust['valence'] >= 0]
disgust_wrong

,file_loc,expr,valence,arousal
44069,/content/train_set/images/157446.jpg,5,0.319396,0.750097
66005,/content/train_set/images/186009.jpg,5,0.000000,0.532327
87585,/content/train_set/images/21405.jpg,5,0.203252,0.788811
111600,/content/train_set/images/245265.jpg,5,0.179055,0.803329
139445,/content/train_set/images/281484.jpg,5,0.009679,0.483933
155590,/content/train_set/images/302443.jpg,5,0.193573,0.827526
232216,/content/train_set/images/401503.jpg,5,0.232288,0.788811
261684,/content/train_set/images/66269.jpg,5,0.300039,0.735579


Άρα έχουμε 8 εικόνες που είναι annotated ως disgust ενώ δεν θα έπρεπε.

### anger

In [ ]:
anger = train_dataset.loc[train_dataset['expr'] == 6]
anger

,file_loc,expr,valence,arousal
5,/content/train_set/images/100006.jpg,6,-0.341270,0.833333
14,/content/train_set/images/100019.jpg,6,-0.578013,0.807686
20,/content/train_set/images/100026.jpg,6,-0.333732,0.577804
21,/content/train_set/images/100028.jpg,6,-0.205255,0.589837
24,/content/train_set/images/100032.jpg,6,-0.168219,0.431332
...,...,...,...,...
287605,/content/train_set/images/99926.jpg,6,-0.460317,0.793651
287633,/content/train_set/images/99978.jpg,6,-0.750097,0.517809
287638,/content/train_set/images/99988.jpg,6,-0.742731,0.410872
287646,/content/train_set/images/99997.jpg,6,-0.753266,0.447745


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
anger_wrong = anger.loc[anger['valence'] >= 0]
anger_wrong

,file_loc,expr,valence,arousal
14485,/content/train_set/images/118982.jpg,6,0.090580,0.500345
68914,/content/train_set/images/189784.jpg,6,0.174216,0.614595
72335,/content/train_set/images/194247.jpg,6,0.103520,0.448585
148249,/content/train_set/images/292863.jpg,6,0.181159,0.638371
161722,/content/train_set/images/310379.jpg,6,0.117446,0.395046
252925,/content/train_set/images/54912.jpg,6,0.087108,0.367789
264219,/content/train_set/images/69634.jpg,6,0.000000,0.619435


Άρα έχουμε 7 εικόνες που είναι annotated ως anger ενώ δεν θα έπρεπε.

### contempt

In [ ]:
contempt = train_dataset.loc[train_dataset['expr'] == 7]
contempt

,file_loc,expr,valence,arousal
9,/content/train_set/images/100012.jpg,7,-0.493612,0.561363
38,/content/train_set/images/100047.jpg,7,-0.590399,0.643631
123,/content/train_set/images/10015.jpg,7,-0.637379,0.674252
196,/content/train_set/images/100251.jpg,7,-0.421022,0.454897
294,/content/train_set/images/100368.jpg,7,-0.611111,0.634921
...,...,...,...,...
286493,/content/train_set/images/98490.jpg,7,-0.417799,0.495822
286534,/content/train_set/images/98540.jpg,7,-0.295199,0.609756
286847,/content/train_set/images/98971.jpg,7,-0.626984,0.650794
286978,/content/train_set/images/99129.jpg,7,-0.411343,0.488773


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
contempt_wrong = contempt.loc[contempt['valence'] >= 0]
contempt_wrong

,file_loc,expr,valence,arousal
3431,/content/train_set/images/104499.jpg,7,0.222222,0.420635
3832,/content/train_set/images/105045.jpg,7,0.043554,0.575881
9831,/content/train_set/images/112930.jpg,7,0.119048,0.555556
10829,/content/train_set/images/11419.jpg,7,0.246032,0.507936
12229,/content/train_set/images/116034.jpg,7,0.280681,0.672667
...,...,...,...,...
270619,/content/train_set/images/77895.jpg,7,0.237127,0.614595
273690,/content/train_set/images/81824.jpg,7,0.154859,0.619435
277265,/content/train_set/images/86404.jpg,7,0.230159,0.500000
279340,/content/train_set/images/8913.jpg,7,0.230159,0.388889


Άρα έχουμε 109 εικόνες που είναι annotated ως contempt ενώ δεν θα έπρεπε.

### Διαγραφή λάθος δεδομένων

#### λ = 0.7

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,2.437835e-04,9.989684e-01,1.146756e-04,1.971191e-04,1.381485e-04,0.000105,0.000119,1.136759e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,2.451585e-03,9.899513e-01,1.162915e-03,1.803783e-03,1.318015e-03,0.001034,0.001165,1.112466e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,2.472930e-06,9.999886e-01,1.167000e-06,2.483886e-06,1.582615e-06,0.000001,0.000001,1.234581e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,9.207826e-01,1.442063e-02,9.325899e-03,1.421924e-02,1.137069e-02,0.008891,0.010979,1.001077e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,1.689112e-02,9.505328e-01,4.490666e-03,8.701712e-03,5.708760e-03,0.004156,0.004917,4.601738e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,0.300000,0.700000,1.726580e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.996000e-01,6.594273e-05,5.930794e-05,6.052851e-05,5.414606e-05,0.000049,0.000058,5.318166e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,4.193482e-11,1.805477e-11,5.534368e-11,3.028212e-11,5.246988e-11,0.300000,0.700000,1.541449e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,5.127475e-03,8.804876e-03,2.475755e-03,9.616210e-01,1.050463e-02,0.003089,0.004408,3.969684e-03,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
2,/content/train_set/images/100001.jpg,2.472930e-06,9.999886e-01,1.167000e-06,2.483886e-06,1.582615e-06,1.120584e-06,1.301686e-06,1.234581e-06,0.774293,0.091947
4,/content/train_set/images/100004.jpg,1.689112e-02,9.505328e-01,4.490666e-03,8.701712e-03,5.708760e-03,4.156205e-03,4.917010e-03,4.601738e-03,0.430701,0.048393
5,/content/train_set/images/100006.jpg,2.771369e-03,1.844927e-03,2.074687e-03,6.992254e-03,1.820512e-01,7.860809e-03,7.535990e-01,4.280568e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,1.550669e-03,1.187763e-03,6.431337e-04,9.877339e-01,5.237972e-03,8.830110e-04,1.501479e-03,1.262072e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,6.433659e-65,4.436553e-65,4.077989e-65,5.564383e-64,1.000000e+00,8.804869e-65,3.022920e-64,2.161619e-64,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,3.000000e-01,7.000000e-01,1.726580e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.996000e-01,6.594273e-05,5.930794e-05,6.052851e-05,5.414606e-05,4.930919e-05,5.753425e-05,5.318166e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,4.193482e-11,1.805477e-11,5.534368e-11,3.028212e-11,5.246988e-11,3.000000e-01,7.000000e-01,1.541449e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,5.127475e-03,8.804876e-03,2.475755e-03,9.616210e-01,1.050463e-02,3.088997e-03,4.407626e-03,3.969684e-03,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
5,/content/train_set/images/100006.jpg,2.771369e-03,1.844927e-03,2.074687e-03,6.992254e-03,1.820512e-01,7.860809e-03,7.535990e-01,4.280568e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,1.550669e-03,1.187763e-03,6.431337e-04,9.877339e-01,5.237972e-03,8.830110e-04,1.501479e-03,1.262072e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,6.433659e-65,4.436553e-65,4.077989e-65,5.564383e-64,1.000000e+00,8.804869e-65,3.022920e-64,2.161619e-64,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,1.074046e-18,5.129836e-19,8.575738e-19,1.180242e-18,3.742558e-18,4.987394e-17,3.337363e-06,9.999967e-01,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,1.000000e+00,3.663053e-15,4.654616e-15,3.496348e-15,3.328615e-15,3.370818e-15,3.771811e-15,3.517095e-15,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,3.000000e-01,7.000000e-01,1.726580e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.996000e-01,6.594273e-05,5.930794e-05,6.052851e-05,5.414606e-05,4.930919e-05,5.753425e-05,5.318166e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,4.193482e-11,1.805477e-11,5.534368e-11,3.028212e-11,5.246988e-11,3.000000e-01,7.000000e-01,1.541449e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,5.127475e-03,8.804876e-03,2.475755e-03,9.616210e-01,1.050463e-02,3.088997e-03,4.407626e-03,3.969684e-03,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./min_train_dataset_0.7.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./min_train_dataset_0.7_v2.pkl") 

In [ ]:
min_train_dataset = pd.read_pickle("./min_train_dataset_0.7.pkl")  

#### λ = 0.7 (χωρίς το contempt)

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,valence,arousal
0,/content/train_set/images/0.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.774293,0.091947
3,/content/train_set/images/100002.jpg,0.700244,0.299082,0.000115,0.000197,0.000138,0.000105,0.000119,0.089659,0.114565
4,/content/train_set/images/100004.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0.700065,0.299717,0.000029,0.000080,0.000044,0.000029,0.000035,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,0.000065,0.299717,0.000029,0.700080,0.000044,0.000029,0.000035,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) ]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,valence,arousal
2,/content/train_set/images/100001.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.774293,0.091947
4,/content/train_set/images/100004.jpg,0.000244,0.999082,0.000115,0.000197,0.000138,0.000105,0.000119,0.430701,0.048393
5,/content/train_set/images/100006.jpg,0.000244,0.299082,0.000115,0.000197,0.000138,0.000105,0.700119,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,0.000244,0.299082,0.000115,0.700197,0.000138,0.000105,0.000119,0.150966,0.534852
7,/content/train_set/images/100009.jpg,0.002461,0.291030,0.001167,0.001810,0.701323,0.001038,0.001170,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0.700065,0.299717,0.000029,0.000080,0.000044,0.000029,0.000035,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,0.000065,0.299717,0.000029,0.700080,0.000044,0.000029,0.000035,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) ]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,valence,arousal
5,/content/train_set/images/100006.jpg,0.000244,0.299082,0.000115,0.000197,0.000138,0.000105,0.700119,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,0.000244,0.299082,0.000115,0.700197,0.000138,0.000105,0.000119,0.150966,0.534852
7,/content/train_set/images/100009.jpg,0.002461,0.291030,0.001167,0.001810,0.701323,0.001038,0.001170,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,0.002461,0.291030,0.001167,0.001810,0.001323,0.001038,0.001170,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,0.702461,0.291030,0.001167,0.001810,0.001323,0.001038,0.001170,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0.700065,0.299717,0.000029,0.000080,0.000044,0.000029,0.000035,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,0.000065,0.299717,0.000029,0.000080,0.000044,0.000029,0.700035,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,0.000065,0.299717,0.000029,0.700080,0.000044,0.000029,0.000035,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./new_min_train_dataset_0.7.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./new_min_train_dataset_0.7_v2.pkl") 

In [ ]:
min_train_dataset_2 = pd.read_pickle("./new_min_train_dataset_0.7_v2.pkl")  

#### λ = 0

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,8.126118e-04,9.965613e-01,3.822520e-04,6.570637e-04,4.604951e-04,0.000349,3.978561e-04,3.789198e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,8.171948e-03,9.665043e-01,3.876382e-03,6.012608e-03,4.393383e-03,0.003448,3.884911e-03,3.708220e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,8.243101e-06,9.999621e-01,3.889999e-06,8.279620e-06,5.275382e-06,0.000004,4.338952e-06,4.115270e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,7.359421e-01,4.806875e-02,3.108633e-02,4.739748e-02,3.790229e-02,0.029638,3.659567e-02,3.336923e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,5.630374e-02,8.351093e-01,1.496889e-02,2.900571e-02,1.902920e-02,0.013854,1.639003e-02,1.533913e-02,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-10,7.438074e-11,2.133667e-10,1.252420e-10,2.200589e-10,1.000000,2.559365e-09,5.755267e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.986668e-01,2.198091e-04,1.976931e-04,2.017617e-04,1.804869e-04,0.000164,1.917808e-04,1.772722e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-10,6.018255e-11,1.844789e-10,1.009404e-10,1.748996e-10,1.000000,2.573964e-09,5.138162e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-02,2.934959e-02,8.252517e-03,8.720699e-01,3.501543e-02,0.010297,1.469209e-02,1.323228e-02,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
2,/content/train_set/images/100001.jpg,8.243101e-06,9.999621e-01,3.889999e-06,8.279620e-06,5.275382e-06,3.735281e-06,4.338952e-06,4.115270e-06,0.774293,0.091947
4,/content/train_set/images/100004.jpg,5.630374e-02,8.351093e-01,1.496889e-02,2.900571e-02,1.902920e-02,1.385402e-02,1.639003e-02,1.533913e-02,0.430701,0.048393
5,/content/train_set/images/100006.jpg,9.237897e-03,6.149756e-03,6.915623e-03,2.330751e-02,6.068374e-01,2.620270e-02,1.786635e-01,1.426856e-01,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,5.168897e-03,3.959209e-03,2.143779e-03,9.591130e-01,1.745991e-02,2.943370e-03,5.004930e-03,4.206906e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,2.144553e-64,1.478851e-64,1.359330e-64,1.854794e-63,1.000000e+00,2.934956e-64,1.007640e-63,7.205395e-64,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-10,7.438074e-11,2.133667e-10,1.252420e-10,2.200589e-10,1.000000e+00,2.559365e-09,5.755267e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.986668e-01,2.198091e-04,1.976931e-04,2.017617e-04,1.804869e-04,1.643640e-04,1.917808e-04,1.772722e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-10,6.018255e-11,1.844789e-10,1.009404e-10,1.748996e-10,1.000000e+00,2.573964e-09,5.138162e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-02,2.934959e-02,8.252517e-03,8.720699e-01,3.501543e-02,1.029666e-02,1.469209e-02,1.323228e-02,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
5,/content/train_set/images/100006.jpg,9.237897e-03,6.149756e-03,6.915623e-03,2.330751e-02,6.068374e-01,2.620270e-02,1.786635e-01,1.426856e-01,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,5.168897e-03,3.959209e-03,2.143779e-03,9.591130e-01,1.745991e-02,2.943370e-03,5.004930e-03,4.206906e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,2.144553e-64,1.478851e-64,1.359330e-64,1.854794e-63,1.000000e+00,2.934956e-64,1.007640e-63,7.205395e-64,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,3.580152e-18,1.709945e-18,2.858579e-18,3.934141e-18,1.247519e-17,1.662465e-16,1.112454e-05,9.999889e-01,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,1.000000e+00,1.221018e-14,1.551539e-14,1.165449e-14,1.109538e-14,1.123606e-14,1.257270e-14,1.172365e-14,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-10,7.438074e-11,2.133667e-10,1.252420e-10,2.200589e-10,1.000000e+00,2.559365e-09,5.755267e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.986668e-01,2.198091e-04,1.976931e-04,2.017617e-04,1.804869e-04,1.643640e-04,1.917808e-04,1.772722e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-10,6.018255e-11,1.844789e-10,1.009404e-10,1.748996e-10,1.000000e+00,2.573964e-09,5.138162e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-02,2.934959e-02,8.252517e-03,8.720699e-01,3.501543e-02,1.029666e-02,1.469209e-02,1.323228e-02,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./min_train_dataset_0.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./min_train_dataset_0_v2.pkl") 

In [ ]:
min_train_dataset = pd.read_pickle("./min_train_dataset_0_v2.pkl")  

#### λ = 0.9

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,8.126118e-05,9.996561e-01,3.822520e-05,6.570637e-05,4.604951e-05,3.494529e-05,3.978561e-05,3.789198e-05,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,8.171948e-04,9.966504e-01,3.876382e-04,6.012608e-04,4.393383e-04,3.448240e-04,3.884911e-04,3.708220e-04,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,8.243101e-07,9.999962e-01,3.889999e-07,8.279620e-07,5.275382e-07,3.735281e-07,4.338952e-07,4.115270e-07,0.774293,0.091947
3,/content/train_set/images/100002.jpg,9.735942e-01,4.806875e-03,3.108633e-03,4.739748e-03,3.790229e-03,2.963811e-03,3.659567e-03,3.336923e-03,0.089659,0.114565
4,/content/train_set/images/100004.jpg,5.630374e-03,9.835109e-01,1.496889e-03,2.900571e-03,1.902920e-03,1.385402e-03,1.639003e-03,1.533913e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-11,7.438074e-12,2.133667e-11,1.252420e-11,2.200589e-11,1.000000e-01,9.000000e-01,5.755267e-10,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.998667e-01,2.198091e-05,1.976931e-05,2.017617e-05,1.804869e-05,1.643640e-05,1.917808e-05,1.772722e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-11,6.018255e-12,1.844789e-11,1.009404e-11,1.748996e-11,1.000000e-01,9.000000e-01,5.138162e-10,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-03,2.934959e-03,8.252517e-04,9.872070e-01,3.501543e-03,1.029666e-03,1.469209e-03,1.323228e-03,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
2,/content/train_set/images/100001.jpg,8.243101e-07,9.999962e-01,3.889999e-07,8.279620e-07,5.275382e-07,3.735281e-07,4.338952e-07,4.115270e-07,0.774293,0.091947
4,/content/train_set/images/100004.jpg,5.630374e-03,9.835109e-01,1.496889e-03,2.900571e-03,1.902920e-03,1.385402e-03,1.639003e-03,1.533913e-03,0.430701,0.048393
5,/content/train_set/images/100006.jpg,9.237897e-04,6.149756e-04,6.915623e-04,2.330751e-03,6.068374e-02,2.620270e-03,9.178663e-01,1.426856e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,5.168897e-04,3.959209e-04,2.143779e-04,9.959113e-01,1.745991e-03,2.943370e-04,5.004930e-04,4.206906e-04,0.150966,0.534852
7,/content/train_set/images/100009.jpg,2.144553e-65,1.478851e-65,1.359330e-65,1.854794e-64,1.000000e+00,2.934956e-65,1.007640e-64,7.205395e-65,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-11,7.438074e-12,2.133667e-11,1.252420e-11,2.200589e-11,1.000000e-01,9.000000e-01,5.755267e-10,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.998667e-01,2.198091e-05,1.976931e-05,2.017617e-05,1.804869e-05,1.643640e-05,1.917808e-05,1.772722e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-11,6.018255e-12,1.844789e-11,1.009404e-11,1.748996e-11,1.000000e-01,9.000000e-01,5.138162e-10,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-03,2.934959e-03,8.252517e-04,9.872070e-01,3.501543e-03,1.029666e-03,1.469209e-03,1.323228e-03,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
5,/content/train_set/images/100006.jpg,9.237897e-04,6.149756e-04,6.915623e-04,2.330751e-03,6.068374e-02,2.620270e-03,9.178663e-01,1.426856e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,5.168897e-04,3.959209e-04,2.143779e-04,9.959113e-01,1.745991e-03,2.943370e-04,5.004930e-04,4.206906e-04,0.150966,0.534852
7,/content/train_set/images/100009.jpg,2.144553e-65,1.478851e-65,1.359330e-65,1.854794e-64,1.000000e+00,2.934956e-65,1.007640e-64,7.205395e-65,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,3.580152e-19,1.709945e-19,2.858579e-19,3.934141e-19,1.247519e-18,1.662465e-17,1.112454e-06,9.999989e-01,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,1.000000e+00,1.221018e-15,1.551539e-15,1.165449e-15,1.109538e-15,1.123606e-15,1.257270e-15,1.172365e-15,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.589471e-11,7.438074e-12,2.133667e-11,1.252420e-11,2.200589e-11,1.000000e-01,9.000000e-01,5.755267e-10,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.998667e-01,2.198091e-05,1.976931e-05,2.017617e-05,1.804869e-05,1.643640e-05,1.917808e-05,1.772722e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,1.397827e-11,6.018255e-12,1.844789e-11,1.009404e-11,1.748996e-11,1.000000e-01,9.000000e-01,5.138162e-10,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.709158e-03,2.934959e-03,8.252517e-04,9.872070e-01,3.501543e-03,1.029666e-03,1.469209e-03,1.323228e-03,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./min_train_dataset_0.9.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./min_train_dataset_0.9_v2.pkl") 

In [ ]:
min_train_dataset_2 = pd.read_pickle("./min_train_dataset_0.9_v2.pkl")  

#### λ = 0.5

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,4.063059e-04,9.982807e-01,1.911260e-04,3.285319e-04,2.302475e-04,0.000175,0.000199,1.894599e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,4.085974e-03,9.832522e-01,1.938191e-03,3.006304e-03,2.196691e-03,0.001724,0.001942,1.854110e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,4.121550e-06,9.999811e-01,1.944999e-06,4.139810e-06,2.637691e-06,0.000002,0.000002,2.057635e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,8.679711e-01,2.403438e-02,1.554317e-02,2.369874e-02,1.895114e-02,0.014819,0.018298,1.668461e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,2.815187e-02,9.175546e-01,7.484443e-03,1.450285e-02,9.514599e-03,0.006927,0.008195,7.669563e-03,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,7.947356e-11,3.719037e-11,1.066833e-10,6.262100e-11,1.100294e-10,0.500000,0.500000,2.877633e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.993334e-01,1.099045e-04,9.884657e-05,1.008809e-04,9.024343e-05,0.000082,0.000096,8.863610e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6.989136e-11,3.009128e-11,9.223947e-11,5.047021e-11,8.744980e-11,0.500000,0.500000,2.569081e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,8.545792e-03,1.467479e-02,4.126259e-03,9.360349e-01,1.750772e-02,0.005148,0.007346,6.616139e-03,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
2,/content/train_set/images/100001.jpg,4.121550e-06,9.999811e-01,1.944999e-06,4.139810e-06,2.637691e-06,1.867640e-06,2.169476e-06,2.057635e-06,0.774293,0.091947
4,/content/train_set/images/100004.jpg,2.815187e-02,9.175546e-01,7.484443e-03,1.450285e-02,9.514599e-03,6.927008e-03,8.195016e-03,7.669563e-03,0.430701,0.048393
5,/content/train_set/images/100006.jpg,4.618949e-03,3.074878e-03,3.457812e-03,1.165376e-02,3.034187e-01,1.310135e-02,5.893317e-01,7.134279e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,2.584449e-03,1.979605e-03,1.071890e-03,9.795565e-01,8.729954e-03,1.471685e-03,2.502465e-03,2.103453e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,1.072277e-64,7.394254e-65,6.796648e-65,9.273972e-64,1.000000e+00,1.467478e-64,5.038201e-64,3.602698e-64,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,7.947356e-11,3.719037e-11,1.066833e-10,6.262100e-11,1.100294e-10,5.000000e-01,5.000000e-01,2.877633e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.993334e-01,1.099045e-04,9.884657e-05,1.008809e-04,9.024343e-05,8.218198e-05,9.589041e-05,8.863610e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6.989136e-11,3.009128e-11,9.223947e-11,5.047021e-11,8.744980e-11,5.000000e-01,5.000000e-01,2.569081e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,8.545792e-03,1.467479e-02,4.126259e-03,9.360349e-01,1.750772e-02,5.148328e-03,7.346044e-03,6.616139e-03,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
5,/content/train_set/images/100006.jpg,4.618949e-03,3.074878e-03,3.457812e-03,1.165376e-02,3.034187e-01,1.310135e-02,5.893317e-01,7.134279e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,2.584449e-03,1.979605e-03,1.071890e-03,9.795565e-01,8.729954e-03,1.471685e-03,2.502465e-03,2.103453e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,1.072277e-64,7.394254e-65,6.796648e-65,9.273972e-64,1.000000e+00,1.467478e-64,5.038201e-64,3.602698e-64,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,1.790076e-18,8.549727e-19,1.429290e-18,1.967070e-18,6.237596e-18,8.312323e-17,5.562272e-06,9.999944e-01,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,1.000000e+00,6.105088e-15,7.757693e-15,5.827247e-15,5.547692e-15,5.618030e-15,6.286352e-15,5.861826e-15,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,7.947356e-11,3.719037e-11,1.066833e-10,6.262100e-11,1.100294e-10,5.000000e-01,5.000000e-01,2.877633e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.993334e-01,1.099045e-04,9.884657e-05,1.008809e-04,9.024343e-05,8.218198e-05,9.589041e-05,8.863610e-05,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6.989136e-11,3.009128e-11,9.223947e-11,5.047021e-11,8.744980e-11,5.000000e-01,5.000000e-01,2.569081e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,8.545792e-03,1.467479e-02,4.126259e-03,9.360349e-01,1.750772e-02,5.148328e-03,7.346044e-03,6.616139e-03,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./min_train_dataset_0.5.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./min_train_dataset_0.5_v2.pkl") 

In [ ]:
min_train_dataset_2 = pd.read_pickle("./min_train_dataset_0.5_v2.pkl")  

#### λ = 0.3

In [ ]:
final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/0.jpg,5.688282e-04,9.975929e-01,2.675764e-04,4.599446e-04,3.223466e-04,0.000245,0.000278,2.652439e-04,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,5.720364e-03,9.765530e-01,2.713467e-03,4.208826e-03,3.075368e-03,0.002414,0.002719,2.595754e-03,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,5.770170e-06,9.999735e-01,2.722999e-06,5.795734e-06,3.692768e-06,0.000003,0.000003,2.880689e-06,0.774293,0.091947
3,/content/train_set/images/100002.jpg,8.151595e-01,3.364813e-02,2.176043e-02,3.317824e-02,2.653160e-02,0.020747,0.025617,2.335846e-02,0.089659,0.114565
4,/content/train_set/images/100004.jpg,3.941262e-02,8.845765e-01,1.047822e-02,2.030399e-02,1.332044e-02,0.009698,0.011473,1.073739e-02,0.430701,0.048393
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.112630e-10,5.206652e-11,1.493567e-10,8.766940e-11,1.540412e-10,0.700000,0.300000,4.028687e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.990668e-01,1.538664e-04,1.383852e-04,1.412332e-04,1.263408e-04,0.000115,0.000134,1.240905e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,9.784791e-11,4.212779e-11,1.291353e-10,7.065829e-11,1.224297e-10,0.700000,0.300000,3.596713e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.196411e-02,2.054471e-02,5.776762e-03,9.104489e-01,2.451080e-02,0.007208,0.010284,9.262595e-03,0.406504,0.609756


Επομένως τώρα θέλουμε να διαγράψουμε από το final_train_dataset τις εικόνες που έχουν λάθος annotation.

Για happy va > 0

In [ ]:
min_train_dataset = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_1['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
2,/content/train_set/images/100001.jpg,5.770170e-06,9.999735e-01,2.722999e-06,5.795734e-06,3.692768e-06,2.614697e-06,3.037266e-06,2.880689e-06,0.774293,0.091947
4,/content/train_set/images/100004.jpg,3.941262e-02,8.845765e-01,1.047822e-02,2.030399e-02,1.332044e-02,9.697812e-03,1.147302e-02,1.073739e-02,0.430701,0.048393
5,/content/train_set/images/100006.jpg,6.466528e-03,4.304829e-03,4.840936e-03,1.631526e-02,4.247862e-01,1.834189e-02,4.250644e-01,9.987991e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,3.618228e-03,2.771446e-03,1.500645e-03,9.713791e-01,1.222194e-02,2.060359e-03,3.503451e-03,2.944835e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,1.501187e-64,1.035196e-64,9.515307e-65,1.298356e-63,1.000000e+00,2.054470e-64,7.053481e-64,5.043777e-64,-0.126422,0.758533
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.112630e-10,5.206652e-11,1.493567e-10,8.766940e-11,1.540412e-10,7.000000e-01,3.000000e-01,4.028687e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.990668e-01,1.538664e-04,1.383852e-04,1.412332e-04,1.263408e-04,1.150548e-04,1.342466e-04,1.240905e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,9.784791e-11,4.212779e-11,1.291353e-10,7.065829e-11,1.224297e-10,7.000000e-01,3.000000e-01,3.596713e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.196411e-02,2.054471e-02,5.776762e-03,9.104489e-01,2.451080e-02,7.207660e-03,1.028446e-02,9.262595e-03,0.406504,0.609756


Για happy va > 0.1

In [ ]:
min_train_dataset_2 = final_train_dataset.loc[~final_train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~final_train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~final_train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
min_train_dataset_2

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
5,/content/train_set/images/100006.jpg,6.466528e-03,4.304829e-03,4.840936e-03,1.631526e-02,4.247862e-01,1.834189e-02,4.250644e-01,9.987991e-02,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,3.618228e-03,2.771446e-03,1.500645e-03,9.713791e-01,1.222194e-02,2.060359e-03,3.503451e-03,2.944835e-03,0.150966,0.534852
7,/content/train_set/images/100009.jpg,1.501187e-64,1.035196e-64,9.515307e-65,1.298356e-63,1.000000e+00,2.054470e-64,7.053481e-64,5.043777e-64,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,2.506107e-18,1.196962e-18,2.001006e-18,2.753899e-18,8.732635e-18,1.163725e-16,7.787180e-06,9.999922e-01,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,1.000000e+00,8.547123e-15,1.086077e-14,8.158146e-15,7.766768e-15,7.865242e-15,8.800893e-15,8.206556e-15,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,/content/train_set/images/99997.jpg,1.112630e-10,5.206652e-11,1.493567e-10,8.766940e-11,1.540412e-10,7.000000e-01,3.000000e-01,4.028687e-09,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,9.990668e-01,1.538664e-04,1.383852e-04,1.412332e-04,1.263408e-04,1.150548e-04,1.342466e-04,1.240905e-04,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,9.784791e-11,4.212779e-11,1.291353e-10,7.065829e-11,1.224297e-10,7.000000e-01,3.000000e-01,3.596713e-09,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,1.196411e-02,2.054471e-02,5.776762e-03,9.104489e-01,2.451080e-02,7.207660e-03,1.028446e-02,9.262595e-03,0.406504,0.609756


In [ ]:
min_train_dataset.to_pickle("./min_train_dataset_0.3.pkl")  

In [ ]:
min_train_dataset_2.to_pickle("./min_train_dataset_0.3_v2.pkl") 

In [ ]:
min_train_dataset_2 = pd.read_pickle("./min_train_dataset_0.3_v2.pkl")  

## Relabelling

### λ = 0.7

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 218431 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

2400
43


###### 1st version of happy, lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
2400/2400 [==============================] - ETA: 0s - loss: 0.9316 - accuracy: 0.7112
Epoch 1: val_accuracy improved from -inf to 0.47037, saving model to /content/weights0.4704.hdf5
2400/2400 [==============================] - 585s 236ms/step - loss: 0.9316 - accuracy: 0.7112 - val_loss: 1.5411 - val_accuracy: 0.4704
Epoch 2/5
2400/2400 [==============================] - ETA: 0s - loss: 0.7897 - accuracy: 0.7643
Epoch 2: val_accuracy improved from 0.47037 to 0.48437, saving model to /content/weights0.4844.hdf5
2400/2400 [==============================] - 516s 215ms/step - loss: 0.7897 - accuracy: 0.7643 - val_loss: 1.4520 - val_accuracy: 0.4844
Epoch 3/5
2400/2400 [==============================] - ETA: 0s - loss: 0.7024 - accuracy: 0.7983
Epoch 3: val_accuracy improved from 0.48437 to 0.49387, saving model to /content/weights0.4939.hdf5
2400/2400 [==============================] - 516s 215ms/step - loss: 0.7024 - accuracy: 0.7983 - val_loss: 1.5433 - val_accuracy: 0.4939
E

In [ ]:
y_pred = model_resnet.predict(test_generator)

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)
y_pred_max

array([0, 0, 3, ..., 6, 3, 2])

In [ ]:
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5101275318829708
f1 score 0.48921016145711654
average accuracy 0.5100425851703407


##### 2nd version of happy, lr = 10^-4 

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
1985/1985 [==============================] - ETA: 0s - loss: 1.0518 - accuracy: 0.6716
Epoch 1: val_accuracy improved from -inf to 0.46787, saving model to /content/weights0.4679.hdf5
1985/1985 [==============================] - 498s 240ms/step - loss: 1.0518 - accuracy: 0.6716 - val_loss: 1.4591 - val_accuracy: 0.4679
Epoch 2/5
1985/1985 [==============================] - ETA: 0s - loss: 0.8800 - accuracy: 0.7359
Epoch 2: val_accuracy improved from 0.46787 to 0.48962, saving model to /content/weights0.4896.hdf5
1985/1985 [==============================] - 442s 223ms/step - loss: 0.8800 - accuracy: 0.7359 - val_loss: 1.4472 - val_accuracy: 0.4896
Epoch 3/5
1985/1985 [==============================] - ETA: 0s - loss: 0.7839 - accuracy: 0.7730
Epoch 3: val_accuracy did not improve from 0.48962
1985/1985 [==============================] - 440s 222ms/step - loss: 0.7839 - accuracy: 0.7730 - val_loss: 1.5276 - val_accuracy: 0.4789
Epoch 4/5
1985/1985 [=============================

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 0.4542 - accuracy: 0.9105
Epoch 1: val_accuracy improved from -inf to 0.51113, saving model to /content/weights0.5111.hdf5
1985/1985 [==============================] - 431s 217ms/step - loss: 0.4542 - accuracy: 0.9105 - val_loss: 1.8443 - val_accuracy: 0.5111
Epoch 2/5
1985/1985 [==============================] - ETA: 0s - loss: 0.3992 - accuracy: 0.9349
Epoch 2: val_accuracy improved from 0.51113 to 0.51663, saving model to /content/weights0.5166.hdf5
1985/1985 [==============================] - 437s 220ms/step - loss: 0.3992 - accuracy: 0.9349 - val_loss: 1.8213 - val_accuracy: 0.5166
Epoch 3/5
1985/1985 [==============================] - ETA: 0s - loss: 0.3734 - accuracy: 0.9453
Epoch 3: val_accuracy did not improve from 0.51663
1985/1985 [==============================] - 438s 221ms/step - loss: 0.3734 - accuracy: 0.9453 - val_loss: 2.2707 - val_accuracy: 0.4789
Epoch 4/5
1985/1985 [==============================] - ETA: 

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5166.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5166291572893223
f1 score 0.5008811244897844
average accuracy 0.5165546092184369


#### kl divergence with 2nd version

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="kl_divergence" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 0.8521 - accuracy: 0.6711
Epoch 1: val_accuracy improved from -inf to 0.48612, saving model to /content/weights0.4861.hdf5
1985/1985 [==============================] - 448s 222ms/step - loss: 0.8521 - accuracy: 0.6711 - val_loss: 1.4045 - val_accuracy: 0.4861
Epoch 2/5
1985/1985 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.7362
Epoch 2: val_accuracy improved from 0.48612 to 0.49762, saving model to /content/weights0.4976.hdf5
1985/1985 [==============================] - 442s 222ms/step - loss: 0.6833 - accuracy: 0.7362 - val_loss: 1.4253 - val_accuracy: 0.4976
Epoch 3/5
1985/1985 [==============================] - ETA: 0s - loss: 0.5825 - accuracy: 0.7752
Epoch 3: val_accuracy improved from 0.49762 to 0.50413, saving model to /content/weights0.5041.hdf5
1985/1985 [==============================] - 441s 222ms/step - loss: 0.5825 - accuracy: 0.7752 - val_loss: 1.4675 - val_accuracy: 0.5041
Epoch 4/5
1

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
1985/1985 [==============================] - ETA: 0s - loss: 0.2511 - accuracy: 0.9132
Epoch 1: val_accuracy improved from -inf to 0.50838, saving model to /content/weights0.5084.hdf5
1985/1985 [==============================] - 434s 219ms/step - loss: 0.2511 - accuracy: 0.9132 - val_loss: 1.7112 - val_accuracy: 0.5084
Epoch 2/5
1985/1985 [==============================] - ETA: 0s - loss: 0.2044 - accuracy: 0.9338
Epoch 2: val_accuracy did not improve from 0.50838
1985/1985 [==============================] - 440s 221ms/step - loss: 0.2044 - accuracy: 0.9338 - val_loss: 1.9291 - val_accuracy: 0.4934
Epoch 3/5
1985/1985 [==============================] - ETA: 0s - loss: 0.1716 - accuracy: 0.9465
Epoch 3: val_accuracy improved from 0.50838 to 0.51463, saving model to /content/weights0.5146.hdf5
1985/1985 [==============================] - 441s 222ms/step - loss: 0.1716 - accuracy: 0.9465 - val_loss: 1.8606 - val_accuracy: 0.5146
Epoch 4/5
1985/1985 [=============================

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5146.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5146286571642911
f1 score 0.4992800117131153
average accuracy 0.5145591182364729


#### kl divergence with 2nd version lr = 10^-5

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="kl_divergence" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 1.1693 - accuracy: 0.5546
Epoch 1: val_accuracy improved from -inf to 0.37734, saving model to /content/weights0.3773.hdf5
1985/1985 [==============================] - 452s 224ms/step - loss: 1.1693 - accuracy: 0.5546 - val_loss: 1.8040 - val_accuracy: 0.3773
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8274 - accuracy: 0.6825
Epoch 2: val_accuracy improved from 0.37734 to 0.42711, saving model to /content/weights0.4271.hdf5
1985/1985 [==============================] - 443s 223ms/step - loss: 0.8274 - accuracy: 0.6825 - val_loss: 1.6621 - val_accuracy: 0.4271
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.6797 - accuracy: 0.7394
Epoch 3: val_accuracy improved from 0.42711 to 0.43661, saving model to /content/weights0.4366.hdf5
1985/1985 [==============================] - 443s 223ms/step - loss: 0.6797 - accuracy: 0.7394 - val_loss: 1.6524 - val_accuracy: 0.4366
Epoch 4/1

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

### λ = 0.7 (χωρίς το contempt)

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset_2,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180746 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1986
43


###### 2nd version of happy, lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(7, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3899 - accuracy: 0.6059
Epoch 1: val_accuracy improved from -inf to 0.33833, saving model to /content/weights0.3383.hdf5
1986/1986 [==============================] - 468s 226ms/step - loss: 1.3899 - accuracy: 0.6059 - val_loss: 1.5709 - val_accuracy: 0.3383
Epoch 2/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3444 - accuracy: 0.6364
Epoch 2: val_accuracy improved from 0.33833 to 0.37209, saving model to /content/weights0.3721.hdf5
1986/1986 [==============================] - 428s 216ms/step - loss: 1.3444 - accuracy: 0.6364 - val_loss: 1.5348 - val_accuracy: 0.3721
Epoch 3/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3439 - accuracy: 0.6399
Epoch 3: val_accuracy did not improve from 0.37209
1986/1986 [==============================] - 427s 215ms/step - loss: 1.3439 - accuracy: 0.6399 - val_loss: 1.5865 - val_accuracy: 0.3703
Epoch 4/10
1986/1986 [=========================

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.32983245811452866
f1 score 0.24920455150857276
average accuracy 0.32975


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3854 - accuracy: 0.6107
Epoch 1: val_accuracy improved from -inf to 0.43861, saving model to /content/weights0.4386.hdf5
1986/1986 [==============================] - 486s 236ms/step - loss: 1.3854 - accuracy: 0.6107 - val_loss: 1.6583 - val_accuracy: 0.4386
Epoch 2/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3430 - accuracy: 0.6384
Epoch 2: val_accuracy did not improve from 0.43861
1986/1986 [==============================] - 446s 224ms/step - loss: 1.3430 - accuracy: 0.6384 - val_loss: 1.6854 - val_accuracy: 0.3768
Epoch 3/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3358 - accuracy: 0.6451
Epoch 3: val_accuracy did not improve from 0.43861
1986/1986 [==============================] - 445s 224ms/step - loss: 1.3358 - accuracy: 0.6451 - val_loss: 1.6720 - val_accuracy: 0.3813
Epoch 4/10
1986/1986 [==============================] - ETA: 0s - loss: 1.3452 - accuracy: 0.642

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.3250812703175794
f1 score 0.2389010079804302
average accuracy 0.325


### λ = 0 

Με την πρώτη έκδοση για το happy:

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### lr = 10^-4

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 1.2138 - accuracy: 0.6303
Epoch 1: val_accuracy improved from -inf to 0.45886, saving model to /content/weights0.4589.hdf5
1985/1985 [==============================] - 473s 228ms/step - loss: 1.2138 - accuracy: 0.6303 - val_loss: 1.4770 - val_accuracy: 0.4589
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 1.0665 - accuracy: 0.6831
Epoch 2: val_accuracy did not improve from 0.45886
1985/1985 [==============================] - 431s 217ms/step - loss: 1.0665 - accuracy: 0.6831 - val_loss: 1.5854 - val_accuracy: 0.4431
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.9803 - accuracy: 0.7160
Epoch 3: val_accuracy improved from 0.45886 to 0.48462, saving model to /content/weights0.4846.hdf5
1985/1985 [==============================] - 432s 218ms/step - loss: 0.9803 - accuracy: 0.7160 - val_loss: 1.4912 - val_accuracy: 0.4846
Epoch 4/10
1985/1985 [=========================

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.4964.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.49637409352338085
f1 score 0.4731670693401351
average accuracy 0.49628557114228455


#### Δοκιμή με kl divergence

###### lr = 10^-6 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="kl_divergence" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


2400/2400 [==============================] - ETA: 0s - loss: 1.5620 - accuracy: 0.3875
Epoch 1: val_accuracy improved from -inf to 0.21905, saving model to /content/weights0.2191.hdf5
2400/2400 [==============================] - 526s 217ms/step - loss: 1.5620 - accuracy: 0.3875 - val_loss: 2.3862 - val_accuracy: 0.2191
Epoch 2/5
2400/2400 [==============================] - ETA: 0s - loss: 1.1476 - accuracy: 0.5437
Epoch 2: val_accuracy improved from 0.21905 to 0.25306, saving model to /content/weights0.2531.hdf5
2400/2400 [==============================] - 519s 216ms/step - loss: 1.1476 - accuracy: 0.5437 - val_loss: 2.2276 - val_accuracy: 0.2531
Epoch 3/5
2400/2400 [==============================] - ETA: 0s - loss: 1.0263 - accuracy: 0.5847
Epoch 3: val_accuracy improved from 0.25306 to 0.28407, saving model to /content/weights0.2841.hdf5
2400/2400 [==============================] - 519s 216ms/step - loss: 1.0263 - accuracy: 0.5847 - val_loss: 2.1124 - val_accuracy: 0.2841
Epoch 4/5
2

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=5,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
2400/2400 [==============================] - ETA: 0s - loss: 0.3018 - accuracy: 0.8667
Epoch 1: val_accuracy improved from -inf to 0.41685, saving model to /content/weights0.4169.hdf5
2400/2400 [==============================] - 520s 216ms/step - loss: 0.3018 - accuracy: 0.8667 - val_loss: 1.9593 - val_accuracy: 0.4169
Epoch 2/5
2400/2400 [==============================] - ETA: 0s - loss: 0.2304 - accuracy: 0.8989
Epoch 2: val_accuracy improved from 0.41685 to 0.42686, saving model to /content/weights0.4269.hdf5
2400/2400 [==============================] - 520s 217ms/step - loss: 0.2304 - accuracy: 0.8989 - val_loss: 1.9912 - val_accuracy: 0.4269
Epoch 3/5
2400/2400 [==============================] - ETA: 0s - loss: 0.1829 - accuracy: 0.9184
Epoch 3: val_accuracy did not improve from 0.42686
2400/2400 [==============================] - 519s 216ms/step - loss: 0.1829 - accuracy: 0.9184 - val_loss: 2.1162 - val_accuracy: 0.4179
Epoch 4/5
2400/2400 [=============================

In [ ]:
# model_resnet = tf.keras.models.load_model("/content/weights0.4269.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.4211052763190798
f1 score 0.3880895895314857
average accuracy 0.42100951903807615


###### lr = 10^-5 

best 04291

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) 
model_resnet.compile( loss="kl_divergence" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 1.1351 - accuracy: 0.5299
Epoch 1: val_accuracy improved from -inf to 0.34959, saving model to /content/weights0.3496.hdf5
1985/1985 [==============================] - 496s 218ms/step - loss: 1.1351 - accuracy: 0.5299 - val_loss: 1.7927 - val_accuracy: 0.3496
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8304 - accuracy: 0.6388
Epoch 2: val_accuracy improved from 0.34959 to 0.39460, saving model to /content/weights0.3946.hdf5
1985/1985 [==============================] - 429s 216ms/step - loss: 0.8304 - accuracy: 0.6388 - val_loss: 1.6886 - val_accuracy: 0.3946
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.7008 - accuracy: 0.6885
Epoch 3: val_accuracy improved from 0.39460 to 0.41085, saving model to /content/weights0.4109.hdf5
1985/1985 [==============================] - 428s 215ms/step - loss: 0.7008 - accuracy: 0.6885 - val_loss: 1.6632 - val_accuracy: 0.410

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.4118529632408102
f1 score 0.3841560886211395
average accuracy 0.41176653306613226


### λ = 0.9

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset_2,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy, lr = 10^-4 (best results)

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 0.9735 - accuracy: 0.6741
Epoch 1: val_accuracy improved from -inf to 0.45011, saving model to /content/weights0.4501.hdf5
1985/1985 [==============================] - 440s 219ms/step - loss: 0.9735 - accuracy: 0.6741 - val_loss: 1.5447 - val_accuracy: 0.4501
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.7926 - accuracy: 0.7369
Epoch 2: val_accuracy improved from 0.45011 to 0.48137, saving model to /content/weights0.4814.hdf5
1985/1985 [==============================] - 430s 216ms/step - loss: 0.7926 - accuracy: 0.7369 - val_loss: 1.4903 - val_accuracy: 0.4814
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.6876 - accuracy: 0.7751
Epoch 3: val_accuracy improved from 0.48137 to 0.53363, saving model to /content/weights0.5336.hdf5
1985/1985 [==============================] - 432s 217ms/step - loss: 0.6876 - accuracy: 0.7751 - val_loss: 1.3795 - val_accuracy: 0.5336
Epoch 4/1

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5336.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5336334083520881
f1 score 0.518335141256822
average accuracy 0.5335490981963927


In [ ]:
print("f1 score per class \n",f1_score(y_true, y_pred_max,average=None))
print("accuracy per class \n",confusion_matrix(y_true, y_pred_max, normalize='true').diagonal())

f1 score per class 
 [0.48773449 0.67188724 0.57570423 0.49830125 0.58109685 0.47375505
 0.55050973 0.30769231]
accuracy per class 
 [0.676      0.858      0.654      0.44       0.498      0.352
 0.594      0.19639279]


### λ = 0.5

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset_2,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy, lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 1.1066 - accuracy: 0.6631
Epoch 1: val_accuracy improved from -inf to 0.47462, saving model to /content/weights0.4746.hdf5
1985/1985 [==============================] - 443s 218ms/step - loss: 1.1066 - accuracy: 0.6631 - val_loss: 1.4597 - val_accuracy: 0.4746
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.9445 - accuracy: 0.7253
Epoch 2: val_accuracy did not improve from 0.47462
1985/1985 [==============================] - 431s 217ms/step - loss: 0.9445 - accuracy: 0.7253 - val_loss: 1.4802 - val_accuracy: 0.4716
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8511 - accuracy: 0.7633
Epoch 3: val_accuracy improved from 0.47462 to 0.48662, saving model to /content/weights0.4866.hdf5
1985/1985 [==============================] - 431s 217ms/step - loss: 0.8511 - accuracy: 0.7633 - val_loss: 1.5238 - val_accuracy: 0.4866
Epoch 4/10
1985/1985 [=========================

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5084.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5083770942735684
f1 score 0.4862854502982517
average accuracy 0.5082765531062124


### λ = 0.3

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=min_train_dataset_2,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy, lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 1.1582 - accuracy: 0.6434
Epoch 1: val_accuracy improved from -inf to 0.45336, saving model to /content/weights0.4534.hdf5
1985/1985 [==============================] - 427s 212ms/step - loss: 1.1582 - accuracy: 0.6434 - val_loss: 1.5456 - val_accuracy: 0.4534
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 1.0013 - accuracy: 0.7020
Epoch 2: val_accuracy improved from 0.45336 to 0.47262, saving model to /content/weights0.4726.hdf5
1985/1985 [==============================] - 426s 214ms/step - loss: 1.0013 - accuracy: 0.7020 - val_loss: 1.4778 - val_accuracy: 0.4726
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.9111 - accuracy: 0.7372
Epoch 3: val_accuracy improved from 0.47262 to 0.49737, saving model to /content/weights0.4974.hdf5
1985/1985 [==============================] - 426s 214ms/step - loss: 0.9111 - accuracy: 0.7372 - val_loss: 1.4825 - val_accuracy: 0.4974
Epoch 4/1

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.4974.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.4973743435858965
f1 score 0.4663104105703995
average accuracy 0.49725801603206415


## Στατιστικά

### λ = 0.7

Είδαμε ότι η βέλτιστη απόδοση είναι 0.5166.

In [ ]:
model = tf.keras.models.load_model("/content/weights0.5166.hdf5")

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
y_pred_max = np.argmax(y_pred, axis=1)

In [ ]:
y_true = test_dataset["expr"].to_numpy()

In [ ]:
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5166291572893223
f1 score 0.5008811244897844
average accuracy 0.5165546092184369


In [ ]:
preds = pd.DataFrame(y_pred_max)
preds

,0
0,2
1,0
2,4
3,6
4,0
...,...
3994,1
3995,1
3996,6
3997,3


In [ ]:
true = pd.DataFrame(y_true)
true

,0
0,0
1,0
2,4
3,5
4,2
...,...
3994,1
3995,1
3996,6
3997,3


#### happy

In [ ]:
happy_true = true.loc[true[0] == 1]
happy_true

,0
10,1
17,1
40,1
48,1
74,1
...,...
3986,1
3988,1
3991,1
3994,1


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
happy_preds = preds.iloc[preds.index.isin(happy_true.index.tolist())]
happy_preds

,0
10,1
17,5
40,1
48,1
74,1
...,...
3986,1
3988,1
3991,1
3994,1


Πόσα είχαν συναίσθημα happy και μετά προβλεύθηκε κάτι διαφορετικό

In [ ]:
diff = happy_true.values != happy_preds.values
result = diff.flatten().sum()
print(result)

76


In [ ]:
del one_hot_train_dataset['valence']
del one_hot_train_dataset['arousal']
happy_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_1'] == 1]
happy_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg
8,0,1,0,0,0,0,0,0,/content/train_set/images/100011.jpg
...,...,...,...,...,...,...,...,...,...
287639,0,1,0,0,0,0,0,0,/content/train_set/images/9998.jpg
287640,0,1,0,0,0,0,0,0,/content/train_set/images/99990.jpg
287644,0,1,0,0,0,0,0,0,/content/train_set/images/99995.jpg
287645,0,1,0,0,0,0,0,0,/content/train_set/images/99996.jpg


In [ ]:
del final_train_dataset['valence']
del final_train_dataset['arousal']
happy = final_train_dataset.loc[final_train_dataset['file_loc'].isin(happy_old['file_loc'])]
happy

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
0,/content/train_set/images/0.jpg,2.437835e-04,0.998968,1.146756e-04,1.971191e-04,1.381485e-04,1.048359e-04,1.193568e-04,1.136759e-04
1,/content/train_set/images/100000.jpg,2.451585e-03,0.989951,1.162915e-03,1.803783e-03,1.318015e-03,1.034472e-03,1.165473e-03,1.112466e-03
2,/content/train_set/images/100001.jpg,2.472930e-06,0.999989,1.167000e-06,2.483886e-06,1.582615e-06,1.120584e-06,1.301686e-06,1.234581e-06
4,/content/train_set/images/100004.jpg,1.689112e-02,0.950533,4.490666e-03,8.701712e-03,5.708760e-03,4.156205e-03,4.917010e-03,4.601738e-03
8,/content/train_set/images/100011.jpg,1.584995e-04,0.999461,5.409319e-05,9.880894e-05,6.644019e-05,4.937113e-05,5.739521e-05,5.412199e-05
...,...,...,...,...,...,...,...,...,...
287639,/content/train_set/images/9998.jpg,3.717088e-02,0.903182,8.621447e-03,1.510484e-02,1.037715e-02,7.795307e-03,9.166490e-03,8.581549e-03
287640,/content/train_set/images/99990.jpg,7.803417e-07,0.999996,3.638917e-07,8.108128e-07,5.046033e-07,3.521656e-07,4.110461e-07,3.893358e-07
287644,/content/train_set/images/99995.jpg,1.991339e-02,0.928910,6.039021e-03,1.613062e-02,9.042737e-03,5.961578e-03,7.256225e-03,6.746785e-03
287645,/content/train_set/images/99996.jpg,9.770302e-06,0.999965,3.475150e-06,6.721902e-06,4.408158e-06,3.217587e-06,3.755454e-06,3.539860e-06


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [happy['expr_0'].max(),happy['expr_1'].max(),happy['expr_2'].max(),happy['expr_3'].max(),happy['expr_4'].max(),happy['expr_5'].max(),happy['expr_6'].max(),happy['expr_7'].max()]
print("Max values:\n",maxs)
mins = [happy['expr_0'].min(),happy['expr_1'].min(),happy['expr_2'].min(),happy['expr_3'].min(),happy['expr_4'].min(),happy['expr_5'].min(),happy['expr_6'].min(),happy['expr_7'].min()]
print("Min values:\n",mins)
means = [happy['expr_0'].mean(),happy['expr_1'].mean(),happy['expr_2'].mean(),happy['expr_3'].mean(),happy['expr_4'].mean(),happy['expr_5'].mean(),happy['expr_6'].mean(),happy['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2999898644745638, 1.0, 0.29416699884821146, 0.3, 0.29999946605336675, 0.0287108455866289, 0.18835695094687252, 0.09351359148114731]
Min values:
 [0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.0071955289131718225, 0.9745195631472361, 0.0023813945091821836, 0.005734770604767698, 0.0030879751874817625, 0.0021657784465883094, 0.0025291441084934483, 0.0023858450830784653]


Επίσης θέλουμε να δούμε από τις 134415 εικόνες με το συναίσθημα happy, σε πόσες το νέο label είναι διαφορετικό του happy.

In [ ]:
del happy['file_loc']

In [ ]:
happy

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
0,2.437835e-04,0.998968,1.146756e-04,1.971191e-04,1.381485e-04,1.048359e-04,1.193568e-04,1.136759e-04
1,2.451585e-03,0.989951,1.162915e-03,1.803783e-03,1.318015e-03,1.034472e-03,1.165473e-03,1.112466e-03
2,2.472930e-06,0.999989,1.167000e-06,2.483886e-06,1.582615e-06,1.120584e-06,1.301686e-06,1.234581e-06
4,1.689112e-02,0.950533,4.490666e-03,8.701712e-03,5.708760e-03,4.156205e-03,4.917010e-03,4.601738e-03
8,1.584995e-04,0.999461,5.409319e-05,9.880894e-05,6.644019e-05,4.937113e-05,5.739521e-05,5.412199e-05
...,...,...,...,...,...,...,...,...
287639,3.717088e-02,0.903182,8.621447e-03,1.510484e-02,1.037715e-02,7.795307e-03,9.166490e-03,8.581549e-03
287640,7.803417e-07,0.999996,3.638917e-07,8.108128e-07,5.046033e-07,3.521656e-07,4.110461e-07,3.893358e-07
287644,1.991339e-02,0.928910,6.039021e-03,1.613062e-02,9.042737e-03,5.961578e-03,7.256225e-03,6.746785e-03
287645,9.770302e-06,0.999965,3.475150e-06,6.721902e-06,4.408158e-06,3.217587e-06,3.755454e-06,3.539860e-06


In [ ]:
happy["max"]=happy.idxmax(axis=1)
happy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
0,2.437835e-04,0.998968,1.146756e-04,1.971191e-04,1.381485e-04,1.048359e-04,1.193568e-04,1.136759e-04,expr_1
1,2.451585e-03,0.989951,1.162915e-03,1.803783e-03,1.318015e-03,1.034472e-03,1.165473e-03,1.112466e-03,expr_1
2,2.472930e-06,0.999989,1.167000e-06,2.483886e-06,1.582615e-06,1.120584e-06,1.301686e-06,1.234581e-06,expr_1
4,1.689112e-02,0.950533,4.490666e-03,8.701712e-03,5.708760e-03,4.156205e-03,4.917010e-03,4.601738e-03,expr_1
8,1.584995e-04,0.999461,5.409319e-05,9.880894e-05,6.644019e-05,4.937113e-05,5.739521e-05,5.412199e-05,expr_1
...,...,...,...,...,...,...,...,...,...
287639,3.717088e-02,0.903182,8.621447e-03,1.510484e-02,1.037715e-02,7.795307e-03,9.166490e-03,8.581549e-03,expr_1
287640,7.803417e-07,0.999996,3.638917e-07,8.108128e-07,5.046033e-07,3.521656e-07,4.110461e-07,3.893358e-07,expr_1
287644,1.991339e-02,0.928910,6.039021e-03,1.613062e-02,9.042737e-03,5.961578e-03,7.256225e-03,6.746785e-03,expr_1
287645,9.770302e-06,0.999965,3.475150e-06,6.721902e-06,4.408158e-06,3.217587e-06,3.755454e-06,3.539860e-06,expr_1


In [ ]:
print(happy.loc[happy['max'] != "expr_1"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


Άρα όλες οι happy έχουν προβλεφθεί σωστά.

#### neutral

In [ ]:
neutral_true = true.loc[true[0] == 0]
neutral_true

,0
0,0
1,0
14,0
16,0
23,0
...,...
3953,0
3963,0
3982,0
3984,0


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
neutral_preds = preds.iloc[preds.index.isin(neutral_true.index.tolist())]
neutral_preds

,0
0,2
1,0
14,0
16,0
23,0
...,...
3953,2
3963,6
3982,0
3984,0


In [ ]:
diff = neutral_true.values != neutral_preds.values
result = diff.flatten().sum()
print(result)

185


In [ ]:
neutral_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_0'] == 1]
neutral_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg
10,1,0,0,0,0,0,0,0,/content/train_set/images/100013.jpg
13,1,0,0,0,0,0,0,0,/content/train_set/images/100018.jpg
19,1,0,0,0,0,0,0,0,/content/train_set/images/100025.jpg
23,1,0,0,0,0,0,0,0,/content/train_set/images/100031.jpg
...,...,...,...,...,...,...,...,...,...
287636,1,0,0,0,0,0,0,0,/content/train_set/images/99982.jpg
287637,1,0,0,0,0,0,0,0,/content/train_set/images/99984.jpg
287642,1,0,0,0,0,0,0,0,/content/train_set/images/99993.jpg
287643,1,0,0,0,0,0,0,0,/content/train_set/images/99994.jpg


In [ ]:
neutral = final_train_dataset.loc[final_train_dataset['file_loc'].isin(neutral_old['file_loc'])]
neutral

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
3,/content/train_set/images/100002.jpg,0.920783,1.442063e-02,9.325899e-03,1.421924e-02,1.137069e-02,8.891433e-03,1.097870e-02,1.001077e-02
10,/content/train_set/images/100013.jpg,1.000000,3.663053e-15,4.654616e-15,3.496348e-15,3.328615e-15,3.370818e-15,3.771811e-15,3.517095e-15
13,/content/train_set/images/100018.jpg,0.811172,8.788051e-02,1.721587e-02,2.155836e-02,1.686057e-02,1.408970e-02,1.608744e-02,1.513549e-02
19,/content/train_set/images/100025.jpg,0.999974,3.572045e-06,5.281556e-06,3.374697e-06,3.255115e-06,3.430589e-06,3.765962e-06,3.524337e-06
23,/content/train_set/images/100031.jpg,0.999999,1.643402e-07,2.012386e-07,1.487532e-07,1.401988e-07,1.412695e-07,1.569496e-07,1.468820e-07
...,...,...,...,...,...,...,...,...,...
287636,/content/train_set/images/99982.jpg,0.999954,6.716988e-06,8.774943e-06,6.024221e-06,5.708882e-06,5.851239e-06,6.442937e-06,6.041192e-06
287637,/content/train_set/images/99984.jpg,1.000000,8.610171e-10,1.158711e-09,8.207480e-10,7.859765e-10,8.086228e-10,8.987418e-10,8.389720e-10
287642,/content/train_set/images/99993.jpg,0.937453,6.969728e-03,1.945787e-02,6.528719e-03,6.500514e-03,7.678670e-03,7.912551e-03,7.499191e-03
287643,/content/train_set/images/99994.jpg,0.999979,3.349229e-06,3.236768e-06,3.098817e-06,2.815349e-06,2.624580e-06,3.035477e-06,2.811249e-06


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [neutral['expr_0'].max(),neutral['expr_1'].max(),neutral['expr_2'].max(),neutral['expr_3'].max(),neutral['expr_4'].max(),neutral['expr_5'].max(),neutral['expr_6'].max(),neutral['expr_7'].max()]
print("Max values:\n",maxs)
mins = [neutral['expr_0'].min(),neutral['expr_1'].min(),neutral['expr_2'].min(),neutral['expr_3'].min(),neutral['expr_4'].min(),neutral['expr_5'].min(),neutral['expr_6'].min(),neutral['expr_7'].min()]
print("Min values:\n",mins)
means = [neutral['expr_0'].mean(),neutral['expr_1'].mean(),neutral['expr_2'].mean(),neutral['expr_3'].mean(),neutral['expr_4'].mean(),neutral['expr_5'].mean(),neutral['expr_6'].mean(),neutral['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [1.0, 0.3, 0.28755267101668025, 0.20856772083255742, 0.2999997079085731, 0.10176474967548348, 0.14552838325288162, 0.07253953010495366]
Min values:
 [0.7, 6.131021835505047e-117, 7.370727608149487e-117, 5.963043734183301e-117, 5.650789524431911e-117, 5.6175705587246315e-117, 6.357500193414278e-117, 5.9108234399430465e-117]
Mean values:
 [0.9257567744877918, 0.010163304137175594, 0.01733893333625754, 0.0060607577464307045, 0.006331877872597657, 0.010818262552424615, 0.012840580954692293, 0.010689508912629767]


Επίσης θέλουμε να δούμε από τις 74874 εικόνες με το συναίσθημα neutral, σε πόσες το νέο label είναι διαφορετικό του neutral.

In [ ]:
del neutral['file_loc']

In [ ]:
neutral

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
3,0.920783,1.442063e-02,9.325899e-03,1.421924e-02,1.137069e-02,8.891433e-03,1.097870e-02,1.001077e-02
10,1.000000,3.663053e-15,4.654616e-15,3.496348e-15,3.328615e-15,3.370818e-15,3.771811e-15,3.517095e-15
13,0.811172,8.788051e-02,1.721587e-02,2.155836e-02,1.686057e-02,1.408970e-02,1.608744e-02,1.513549e-02
19,0.999974,3.572045e-06,5.281556e-06,3.374697e-06,3.255115e-06,3.430589e-06,3.765962e-06,3.524337e-06
23,0.999999,1.643402e-07,2.012386e-07,1.487532e-07,1.401988e-07,1.412695e-07,1.569496e-07,1.468820e-07
...,...,...,...,...,...,...,...,...
287636,0.999954,6.716988e-06,8.774943e-06,6.024221e-06,5.708882e-06,5.851239e-06,6.442937e-06,6.041192e-06
287637,1.000000,8.610171e-10,1.158711e-09,8.207480e-10,7.859765e-10,8.086228e-10,8.987418e-10,8.389720e-10
287642,0.937453,6.969728e-03,1.945787e-02,6.528719e-03,6.500514e-03,7.678670e-03,7.912551e-03,7.499191e-03
287643,0.999979,3.349229e-06,3.236768e-06,3.098817e-06,2.815349e-06,2.624580e-06,3.035477e-06,2.811249e-06


In [ ]:
neutral["max"]=neutral.idxmax(axis=1)
neutral

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
3,0.920783,1.442063e-02,9.325899e-03,1.421924e-02,1.137069e-02,8.891433e-03,1.097870e-02,1.001077e-02,expr_0
10,1.000000,3.663053e-15,4.654616e-15,3.496348e-15,3.328615e-15,3.370818e-15,3.771811e-15,3.517095e-15,expr_0
13,0.811172,8.788051e-02,1.721587e-02,2.155836e-02,1.686057e-02,1.408970e-02,1.608744e-02,1.513549e-02,expr_0
19,0.999974,3.572045e-06,5.281556e-06,3.374697e-06,3.255115e-06,3.430589e-06,3.765962e-06,3.524337e-06,expr_0
23,0.999999,1.643402e-07,2.012386e-07,1.487532e-07,1.401988e-07,1.412695e-07,1.569496e-07,1.468820e-07,expr_0
...,...,...,...,...,...,...,...,...,...
287636,0.999954,6.716988e-06,8.774943e-06,6.024221e-06,5.708882e-06,5.851239e-06,6.442937e-06,6.041192e-06,expr_0
287637,1.000000,8.610171e-10,1.158711e-09,8.207480e-10,7.859765e-10,8.086228e-10,8.987418e-10,8.389720e-10,expr_0
287642,0.937453,6.969728e-03,1.945787e-02,6.528719e-03,6.500514e-03,7.678670e-03,7.912551e-03,7.499191e-03,expr_0
287643,0.999979,3.349229e-06,3.236768e-06,3.098817e-06,2.815349e-06,2.624580e-06,3.035477e-06,2.811249e-06,expr_0


In [ ]:
print(neutral.loc[neutral['max'] != "expr_0"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


Άρα όλες οι neutral έχουν προβλεφθεί σωστά.

#### sad

In [ ]:
sad_true = true.loc[true[0] == 2]
sad_true

,0
4,2
8,2
13,2
20,2
21,2
...,...
3939,2
3958,2
3973,2
3976,2


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
sad_preds = preds.iloc[preds.index.isin(sad_true.index.tolist())]
sad_preds

,0
4,0
8,0
13,0
20,5
21,3
...,...
3939,2
3958,2
3973,2
3976,2


Πόσα είχαν συναίσθημα happy και μετά προβλεύθηκε κάτι διαφορετικό

In [ ]:
diff = sad_true.values != sad_preds.values
result = diff.flatten().sum()
print(result)

195


In [ ]:
sad_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_2'] == 1]
sad_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
17,0,0,1,0,0,0,0,0,/content/train_set/images/100023.jpg
29,0,0,1,0,0,0,0,0,/content/train_set/images/100039.jpg
36,0,0,1,0,0,0,0,0,/content/train_set/images/100045.jpg
43,0,0,1,0,0,0,0,0,/content/train_set/images/100052.jpg
47,0,0,1,0,0,0,0,0,/content/train_set/images/100058.jpg
...,...,...,...,...,...,...,...,...,...
287580,0,0,1,0,0,0,0,0,/content/train_set/images/99899.jpg
287629,0,0,1,0,0,0,0,0,/content/train_set/images/99973.jpg
287631,0,0,1,0,0,0,0,0,/content/train_set/images/99975.jpg
287632,0,0,1,0,0,0,0,0,/content/train_set/images/99977.jpg


In [ ]:
sad = final_train_dataset.loc[final_train_dataset['file_loc'].isin(sad_old['file_loc'])]
del sad['file_loc']
sad

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
17,6.778093e-03,3.122324e-03,0.965943,3.293537e-03,3.623434e-03,6.902464e-03,5.140602e-03,5.196966e-03
29,8.677693e-04,2.117690e-04,0.997450,2.171743e-04,2.320688e-04,3.798364e-04,3.250099e-04,3.167889e-04
36,9.673756e-03,4.177756e-03,0.934516,4.930075e-03,5.849033e-03,1.965794e-02,1.035737e-02,1.083787e-02
43,2.135116e-02,3.577072e-03,0.947520,3.854762e-03,4.186475e-03,7.229028e-03,6.264955e-03,6.016406e-03
47,5.755618e-04,1.295199e-04,0.998290,1.382647e-04,1.506869e-04,2.726494e-04,2.247626e-04,2.186275e-04
...,...,...,...,...,...,...,...,...
287580,1.262533e-04,5.001703e-05,0.999425,5.311065e-05,5.855317e-05,1.153974e-04,8.563046e-05,8.594941e-05
287629,4.475008e-06,1.646032e-06,0.999979,1.808010e-06,2.032874e-06,4.589335e-06,3.177843e-06,3.198508e-06
287631,2.227290e-03,9.638766e-04,0.988850,1.035087e-03,1.150600e-03,2.362927e-03,1.694688e-03,1.715063e-03
287632,1.127389e-07,3.028553e-08,1.000000,3.294489e-08,3.649412e-08,7.376145e-08,5.654963e-08,5.559420e-08


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [sad['expr_0'].max(),sad['expr_1'].max(),sad['expr_2'].max(),sad['expr_3'].max(),sad['expr_4'].max(),sad['expr_5'].max(),sad['expr_6'].max(),sad['expr_7'].max()]
print("Max values:\n",maxs)
mins = [sad['expr_0'].min(),sad['expr_1'].min(),sad['expr_2'].min(),sad['expr_3'].min(),sad['expr_4'].min(),sad['expr_5'].min(),sad['expr_6'].min(),sad['expr_7'].min()]
print("Min values:\n",mins)
means = [sad['expr_0'].mean(),sad['expr_1'].mean(),sad['expr_2'].mean(),sad['expr_3'].mean(),sad['expr_4'].mean(),sad['expr_5'].mean(),sad['expr_6'].mean(),sad['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29999999999976446, 0.26185167301597817, 1.0, 0.06141478050685742, 0.3, 0.3, 0.3, 0.3]
Min values:
 [4.6225260194e-314, 1.453685626e-314, 0.7, 1.5648216017e-314, 1.730332314e-314, 3.4924720325e-314, 2.626987926e-314, 2.605185075e-314]
Mean values:
 [0.022254812812134022, 0.0032901270063358406, 0.9490361376955995, 0.003075601551646815, 0.00334297400801039, 0.008037030363325558, 0.004791744623244813, 0.006171571939703062]


In [ ]:
sad["max"]=sad.idxmax(axis=1)
sad

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
17,6.778093e-03,3.122324e-03,0.965943,3.293537e-03,3.623434e-03,6.902464e-03,5.140602e-03,5.196966e-03,expr_2
29,8.677693e-04,2.117690e-04,0.997450,2.171743e-04,2.320688e-04,3.798364e-04,3.250099e-04,3.167889e-04,expr_2
36,9.673756e-03,4.177756e-03,0.934516,4.930075e-03,5.849033e-03,1.965794e-02,1.035737e-02,1.083787e-02,expr_2
43,2.135116e-02,3.577072e-03,0.947520,3.854762e-03,4.186475e-03,7.229028e-03,6.264955e-03,6.016406e-03,expr_2
47,5.755618e-04,1.295199e-04,0.998290,1.382647e-04,1.506869e-04,2.726494e-04,2.247626e-04,2.186275e-04,expr_2
...,...,...,...,...,...,...,...,...,...
287580,1.262533e-04,5.001703e-05,0.999425,5.311065e-05,5.855317e-05,1.153974e-04,8.563046e-05,8.594941e-05,expr_2
287629,4.475008e-06,1.646032e-06,0.999979,1.808010e-06,2.032874e-06,4.589335e-06,3.177843e-06,3.198508e-06,expr_2
287631,2.227290e-03,9.638766e-04,0.988850,1.035087e-03,1.150600e-03,2.362927e-03,1.694688e-03,1.715063e-03,expr_2
287632,1.127389e-07,3.028553e-08,1.000000,3.294489e-08,3.649412e-08,7.376145e-08,5.654963e-08,5.559420e-08,expr_2


In [ ]:
print(sad.loc[sad['max'] != "expr_2"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### surprise

In [ ]:
surprise_true = true.loc[true[0] == 3]
surprise_true

,0
6,3
7,3
22,3
25,3
38,3
...,...
3959,3
3962,3
3967,3
3989,3


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
surprise_preds = preds.iloc[preds.index.isin(surprise_true.index.tolist())]
surprise_preds

,0
6,0
7,6
22,1
25,3
38,0
...,...
3959,6
3962,3
3967,3
3989,1


In [ ]:
diff = surprise_true.values != surprise_preds.values
result = diff.flatten().sum()
print(result)

291


In [ ]:
surprise_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_3'] == 1]
surprise_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
6,0,0,0,1,0,0,0,0,/content/train_set/images/100008.jpg
31,0,0,0,1,0,0,0,0,/content/train_set/images/100040.jpg
33,0,0,0,1,0,0,0,0,/content/train_set/images/100042.jpg
62,0,0,0,1,0,0,0,0,/content/train_set/images/100077.jpg
86,0,0,0,1,0,0,0,0,/content/train_set/images/100113.jpg
...,...,...,...,...,...,...,...,...,...
287523,0,0,0,1,0,0,0,0,/content/train_set/images/99817.jpg
287533,0,0,0,1,0,0,0,0,/content/train_set/images/9982.jpg
287573,0,0,0,1,0,0,0,0,/content/train_set/images/9988.jpg
287604,0,0,0,1,0,0,0,0,/content/train_set/images/99925.jpg


In [ ]:
surprise = final_train_dataset.loc[final_train_dataset['file_loc'].isin(surprise_old['file_loc'])]
del surprise['file_loc']
surprise

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
6,1.550669e-03,1.187763e-03,6.431337e-04,0.987734,5.237972e-03,8.830110e-04,1.501479e-03,1.262072e-03
31,3.054729e-06,3.500341e-06,1.561853e-06,0.999969,1.403178e-05,2.144801e-06,3.387839e-06,2.975715e-06
33,7.157088e-05,5.621381e-05,4.662997e-05,0.701335,2.980140e-01,9.020941e-05,2.120069e-04,1.748228e-04
62,3.145408e-13,3.068060e-13,1.554297e-13,1.000000,1.959726e-12,2.198801e-13,3.683469e-13,3.165251e-13
86,1.794928e-04,2.310078e-04,9.169557e-05,0.998395,6.293903e-04,1.223746e-04,1.860830e-04,1.652683e-04
...,...,...,...,...,...,...,...,...
287523,5.346451e-12,4.633681e-12,2.624208e-12,1.000000,4.882729e-11,3.838935e-12,6.819984e-12,5.741355e-12
287533,6.813540e-04,5.245038e-04,2.910121e-04,0.994030,2.777218e-03,4.058957e-04,7.023461e-04,5.880201e-04
287573,8.669501e-04,1.218600e-03,4.279853e-04,0.992991,2.381343e-03,5.551821e-04,8.241777e-04,7.350288e-04
287604,5.919838e-04,4.461632e-04,4.267212e-04,0.702800,2.896924e-01,9.896648e-04,2.739714e-03,2.313533e-03


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [surprise['expr_0'].max(),surprise['expr_1'].max(),surprise['expr_2'].max(),surprise['expr_3'].max(),surprise['expr_4'].max(),surprise['expr_5'].max(),surprise['expr_6'].max(),surprise['expr_7'].max()]
print("Max values:\n",maxs)
mins = [surprise['expr_0'].min(),surprise['expr_1'].min(),surprise['expr_2'].min(),surprise['expr_3'].min(),surprise['expr_4'].min(),surprise['expr_5'].min(),surprise['expr_6'].min(),surprise['expr_7'].min()]
print("Min values:\n",mins)
means = [surprise['expr_0'].mean(),surprise['expr_1'].mean(),surprise['expr_2'].mean(),surprise['expr_3'].mean(),surprise['expr_4'].mean(),surprise['expr_5'].mean(),surprise['expr_6'].mean(),surprise['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29996936612014147, 0.29999929614717086, 0.29999700883968805, 1.0, 0.3, 0.2999999999995289, 0.2992812177874664, 0.2997833146766381]
Min values:
 [0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.006049006647601376, 0.005054313927120835, 0.002393642398653626, 0.9187481914871507, 0.05438562085191312, 0.003310540836660877, 0.005539800657694652, 0.0045188831932047886]


In [ ]:
surprise["max"]=surprise.idxmax(axis=1)
surprise

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
6,1.550669e-03,1.187763e-03,6.431337e-04,0.987734,5.237972e-03,8.830110e-04,1.501479e-03,1.262072e-03,expr_3
31,3.054729e-06,3.500341e-06,1.561853e-06,0.999969,1.403178e-05,2.144801e-06,3.387839e-06,2.975715e-06,expr_3
33,7.157088e-05,5.621381e-05,4.662997e-05,0.701335,2.980140e-01,9.020941e-05,2.120069e-04,1.748228e-04,expr_3
62,3.145408e-13,3.068060e-13,1.554297e-13,1.000000,1.959726e-12,2.198801e-13,3.683469e-13,3.165251e-13,expr_3
86,1.794928e-04,2.310078e-04,9.169557e-05,0.998395,6.293903e-04,1.223746e-04,1.860830e-04,1.652683e-04,expr_3
...,...,...,...,...,...,...,...,...,...
287523,5.346451e-12,4.633681e-12,2.624208e-12,1.000000,4.882729e-11,3.838935e-12,6.819984e-12,5.741355e-12,expr_3
287533,6.813540e-04,5.245038e-04,2.910121e-04,0.994030,2.777218e-03,4.058957e-04,7.023461e-04,5.880201e-04,expr_3
287573,8.669501e-04,1.218600e-03,4.279853e-04,0.992991,2.381343e-03,5.551821e-04,8.241777e-04,7.350288e-04,expr_3
287604,5.919838e-04,4.461632e-04,4.267212e-04,0.702800,2.896924e-01,9.896648e-04,2.739714e-03,2.313533e-03,expr_3


In [ ]:
print(surprise.loc[surprise['max'] != "expr_3"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### fear

In [ ]:
fear_true = true.loc[true[0] == 4]
fear_true

,0
2,4
9,4
19,4
28,4
31,4
...,...
3970,4
3972,4
3977,4
3990,4


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
fear_preds = preds.iloc[preds.index.isin(fear_true.index.tolist())]
fear_preds

,0
2,4
9,4
19,4
28,4
31,6
...,...
3970,3
3972,1
3977,2
3990,4


In [ ]:
diff = fear_true.values != fear_preds.values
result = diff.flatten().sum()
print(result)

293


In [ ]:
fear_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_4'] == 1]
fear_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
7,0,0,0,0,1,0,0,0,/content/train_set/images/100009.jpg
85,0,0,0,0,1,0,0,0,/content/train_set/images/100112.jpg
178,0,0,0,0,1,0,0,0,/content/train_set/images/10022.jpg
251,0,0,0,0,1,0,0,0,/content/train_set/images/100315.jpg
333,0,0,0,0,1,0,0,0,/content/train_set/images/100424.jpg
...,...,...,...,...,...,...,...,...,...
287492,0,0,0,0,1,0,0,0,/content/train_set/images/99783.jpg
287544,0,0,0,0,1,0,0,0,/content/train_set/images/99852.jpg
287574,0,0,0,0,1,0,0,0,/content/train_set/images/99891.jpg
287627,0,0,0,0,1,0,0,0,/content/train_set/images/99970.jpg


In [ ]:
fear = final_train_dataset.loc[final_train_dataset['file_loc'].isin(fear_old['file_loc'])]
del fear['file_loc']
fear

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
7,6.433659e-65,4.436553e-65,4.077989e-65,5.564383e-64,1.000000,8.804869e-65,3.022920e-64,2.161619e-64
85,4.823128e-34,3.332300e-34,3.026580e-34,4.719267e-33,1.000000,6.418004e-34,2.130395e-33,1.532957e-33
178,4.118464e-08,2.733007e-08,2.696553e-08,1.948621e-07,0.999999,6.492393e-08,3.041789e-07,1.974114e-07
251,3.668187e-04,2.271032e-04,2.046203e-04,3.223441e-03,0.993344,4.024282e-04,1.332620e-03,8.984905e-04
333,3.914014e-07,2.683806e-07,2.303030e-07,7.553062e-06,0.999989,4.486925e-07,1.307778e-06,9.563257e-07
...,...,...,...,...,...,...,...,...
287492,1.758551e-05,1.317214e-05,1.207020e-05,1.252373e-04,0.999672,2.636665e-05,7.372646e-05,5.968189e-05
287544,1.174309e-03,9.427655e-04,8.180463e-04,9.766757e-03,0.978800,1.649151e-03,3.673264e-03,3.175233e-03
287574,6.311218e-04,4.205064e-04,3.407396e-04,2.354503e-02,0.971702,6.101712e-04,1.577747e-03,1.172594e-03
287627,3.715060e-16,2.648438e-16,2.336676e-16,4.857134e-15,1.000000,4.803229e-16,1.433107e-15,1.074635e-15


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [fear['expr_0'].max(),fear['expr_1'].max(),fear['expr_2'].max(),fear['expr_3'].max(),fear['expr_4'].max(),fear['expr_5'].max(),fear['expr_6'].max(),fear['expr_7'].max()]
print("Max values:\n",maxs)
mins = [fear['expr_0'].min(),fear['expr_1'].min(),fear['expr_2'].min(),fear['expr_3'].min(),fear['expr_4'].min(),fear['expr_5'].min(),fear['expr_6'].min(),fear['expr_7'].min()]
print("Min values:\n",mins)
means = [fear['expr_0'].mean(),fear['expr_1'].mean(),fear['expr_2'].mean(),fear['expr_3'].mean(),fear['expr_4'].mean(),fear['expr_5'].mean(),fear['expr_6'].mean(),fear['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.18572341723268185, 0.032828241495403886, 0.2999698637391902, 0.2999973326830519, 1.0, 0.3, 0.2990882268879533, 0.3]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0]
Mean values:
 [0.002272215085340342, 0.0008750417455321186, 0.001269053664212939, 0.006392926069128361, 0.9790323537847652, 0.002223704639730988, 0.004595448689693627, 0.0033392563215964687]


In [ ]:
fear["max"]=fear.idxmax(axis=1)
fear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
7,6.433659e-65,4.436553e-65,4.077989e-65,5.564383e-64,1.000000,8.804869e-65,3.022920e-64,2.161619e-64,expr_4
85,4.823128e-34,3.332300e-34,3.026580e-34,4.719267e-33,1.000000,6.418004e-34,2.130395e-33,1.532957e-33,expr_4
178,4.118464e-08,2.733007e-08,2.696553e-08,1.948621e-07,0.999999,6.492393e-08,3.041789e-07,1.974114e-07,expr_4
251,3.668187e-04,2.271032e-04,2.046203e-04,3.223441e-03,0.993344,4.024282e-04,1.332620e-03,8.984905e-04,expr_4
333,3.914014e-07,2.683806e-07,2.303030e-07,7.553062e-06,0.999989,4.486925e-07,1.307778e-06,9.563257e-07,expr_4
...,...,...,...,...,...,...,...,...,...
287492,1.758551e-05,1.317214e-05,1.207020e-05,1.252373e-04,0.999672,2.636665e-05,7.372646e-05,5.968189e-05,expr_4
287544,1.174309e-03,9.427655e-04,8.180463e-04,9.766757e-03,0.978800,1.649151e-03,3.673264e-03,3.175233e-03,expr_4
287574,6.311218e-04,4.205064e-04,3.407396e-04,2.354503e-02,0.971702,6.101712e-04,1.577747e-03,1.172594e-03,expr_4
287627,3.715060e-16,2.648438e-16,2.336676e-16,4.857134e-15,1.000000,4.803229e-16,1.433107e-15,1.074635e-15,expr_4


In [ ]:
print(fear.loc[fear['max'] != "expr_4"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### disgust

In [ ]:
disgust_true = true.loc[true[0] == 5]
disgust_true

,0
3,5
15,5
57,5
64,5
107,5
...,...
3960,5
3964,5
3979,5
3980,5


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
disgust_preds = preds.iloc[preds.index.isin(disgust_true.index.tolist())]
disgust_preds

,0
3,6
15,5
57,3
64,5
107,6
...,...
3960,2
3964,5
3979,6
3980,1


In [ ]:
diff = disgust_true.values != disgust_preds.values
result = diff.flatten().sum()
print(result)

342


In [ ]:
disgust_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_5'] == 1]
disgust_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
35,0,0,0,0,0,1,0,0,/content/train_set/images/100044.jpg
91,0,0,0,0,0,1,0,0,/content/train_set/images/100119.jpg
99,0,0,0,0,0,1,0,0,/content/train_set/images/100128.jpg
145,0,0,0,0,0,1,0,0,/content/train_set/images/100186.jpg
174,0,0,0,0,0,1,0,0,/content/train_set/images/100223.jpg
...,...,...,...,...,...,...,...,...,...
287201,0,0,0,0,0,1,0,0,/content/train_set/images/99421.jpg
287359,0,0,0,0,0,1,0,0,/content/train_set/images/99612.jpg
287480,0,0,0,0,0,1,0,0,/content/train_set/images/99770.jpg
287493,0,0,0,0,0,1,0,0,/content/train_set/images/99784.jpg


In [ ]:
disgust = final_train_dataset.loc[final_train_dataset['file_loc'].isin(disgust_old['file_loc'])]
del disgust['file_loc']
disgust

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
35,5.812960e-09,2.539166e-09,8.250235e-09,4.169545e-09,7.024822e-09,1.000000,7.609379e-08,1.402009e-07
91,4.366095e-05,2.178978e-05,6.332488e-05,3.555846e-05,5.978082e-05,0.998665,3.901911e-04,7.202647e-04
99,7.736507e-06,3.442388e-06,7.190050e-06,6.793607e-06,1.535910e-05,0.736598,1.399265e-02,2.493687e-01
145,6.693756e-10,3.182294e-10,8.920345e-10,5.377839e-10,9.509853e-10,1.000000,1.077192e-08,2.482798e-08
174,4.577718e-10,2.235212e-10,5.610338e-10,3.922442e-10,7.367687e-10,1.000000,1.159483e-08,3.574608e-08
...,...,...,...,...,...,...,...,...
287201,2.837760e-13,1.349701e-13,3.593202e-13,2.329329e-13,4.254246e-13,1.000000,6.260005e-12,1.694888e-11
287359,8.073203e-05,3.958038e-05,1.246451e-04,6.324882e-05,1.032347e-04,0.997946,6.077477e-04,1.034835e-03
287480,6.102991e-09,2.894992e-09,8.326716e-09,4.847331e-09,8.453010e-09,1.000000,8.698254e-08,1.887293e-07
287493,4.797237e-20,2.141760e-20,6.157208e-20,3.648553e-20,6.495007e-20,1.000000,1.063523e-18,2.490887e-18


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [disgust['expr_0'].max(),disgust['expr_1'].max(),disgust['expr_2'].max(),disgust['expr_3'].max(),disgust['expr_4'].max(),disgust['expr_5'].max(),disgust['expr_6'].max(),disgust['expr_7'].max()]
print("Max values:\n",maxs)
mins = [disgust['expr_0'].min(),disgust['expr_1'].min(),disgust['expr_2'].min(),disgust['expr_3'].min(),disgust['expr_4'].min(),disgust['expr_5'].min(),disgust['expr_6'].min(),disgust['expr_7'].min()]
print("Min values:\n",mins)
means = [disgust['expr_0'].mean(),disgust['expr_1'].mean(),disgust['expr_2'].mean(),disgust['expr_3'].mean(),disgust['expr_4'].mean(),disgust['expr_5'].mean(),disgust['expr_6'].mean(),disgust['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2970180252447872, 0.019801942148057233, 0.2999999999674287, 0.299948439224495, 0.2999999999999936, 1.0, 0.2999999999982009, 0.3]
Min values:
 [1.1007309023860154e-303, 5.007068258195963e-304, 1.3260524802850418e-303, 8.770349246564958e-304, 1.6324206577024738e-303, 0.7, 3.8066010799624533e-302, 1.1536832656220979e-301]
Mean values:
 [0.002378199875204804, 0.0006187037329595057, 0.0021700195938340445, 0.0018383839504132096, 0.003916821475999894, 0.9479860032665796, 0.015516574828141241, 0.025575293276867774]


In [ ]:
disgust["max"]=disgust.idxmax(axis=1)
disgust

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
35,5.812960e-09,2.539166e-09,8.250235e-09,4.169545e-09,7.024822e-09,1.000000,7.609379e-08,1.402009e-07,expr_5
91,4.366095e-05,2.178978e-05,6.332488e-05,3.555846e-05,5.978082e-05,0.998665,3.901911e-04,7.202647e-04,expr_5
99,7.736507e-06,3.442388e-06,7.190050e-06,6.793607e-06,1.535910e-05,0.736598,1.399265e-02,2.493687e-01,expr_5
145,6.693756e-10,3.182294e-10,8.920345e-10,5.377839e-10,9.509853e-10,1.000000,1.077192e-08,2.482798e-08,expr_5
174,4.577718e-10,2.235212e-10,5.610338e-10,3.922442e-10,7.367687e-10,1.000000,1.159483e-08,3.574608e-08,expr_5
...,...,...,...,...,...,...,...,...,...
287201,2.837760e-13,1.349701e-13,3.593202e-13,2.329329e-13,4.254246e-13,1.000000,6.260005e-12,1.694888e-11,expr_5
287359,8.073203e-05,3.958038e-05,1.246451e-04,6.324882e-05,1.032347e-04,0.997946,6.077477e-04,1.034835e-03,expr_5
287480,6.102991e-09,2.894992e-09,8.326716e-09,4.847331e-09,8.453010e-09,1.000000,8.698254e-08,1.887293e-07,expr_5
287493,4.797237e-20,2.141760e-20,6.157208e-20,3.648553e-20,6.495007e-20,1.000000,1.063523e-18,2.490887e-18,expr_5


In [ ]:
print(disgust.loc[disgust['max'] != "expr_5"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### anger

In [ ]:
anger_true = true.loc[true[0] == 6]
anger_true

,0
5,6
18,6
44,6
59,6
82,6
...,...
3968,6
3969,6
3978,6
3983,6


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
anger_preds = preds.iloc[preds.index.isin(anger_true.index.tolist())]
anger_preds

,0
5,6
18,6
44,6
59,6
82,0
...,...
3968,0
3969,6
3978,6
3983,6


In [ ]:
diff = anger_true.values != anger_preds.values
result = diff.flatten().sum()
print(result)

161


In [ ]:
anger_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_6'] == 1]
anger_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
5,0,0,0,0,0,0,1,0,/content/train_set/images/100006.jpg
14,0,0,0,0,0,0,1,0,/content/train_set/images/100019.jpg
20,0,0,0,0,0,0,1,0,/content/train_set/images/100026.jpg
21,0,0,0,0,0,0,1,0,/content/train_set/images/100028.jpg
24,0,0,0,0,0,0,1,0,/content/train_set/images/100032.jpg
...,...,...,...,...,...,...,...,...,...
287605,0,0,0,0,0,0,1,0,/content/train_set/images/99926.jpg
287633,0,0,0,0,0,0,1,0,/content/train_set/images/99978.jpg
287638,0,0,0,0,0,0,1,0,/content/train_set/images/99988.jpg
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg


In [ ]:
anger = final_train_dataset.loc[final_train_dataset['file_loc'].isin(anger_old['file_loc'])]
del anger['file_loc']
anger

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
5,2.771369e-03,1.844927e-03,2.074687e-03,6.992254e-03,1.820512e-01,0.007861,0.753599,4.280568e-02
14,1.690952e-03,1.043902e-03,1.488568e-03,2.521832e-03,9.243834e-03,0.019272,0.778530,1.862088e-01
20,1.344677e-04,6.631567e-05,8.606208e-05,2.049120e-04,1.318915e-03,0.000466,0.985832,1.189139e-02
21,4.977347e-03,2.681464e-03,2.863460e-03,1.254660e-02,1.854905e-01,0.007300,0.758082,2.605935e-02
24,3.039174e-02,1.164128e-02,1.273308e-02,3.470068e-02,6.299744e-02,0.024030,0.776278,4.722802e-02
...,...,...,...,...,...,...,...,...
287605,1.420083e-03,8.811800e-04,1.137244e-03,2.542816e-03,1.740632e-02,0.008500,0.817242,1.508712e-01
287633,2.643163e-08,1.312144e-08,3.210058e-08,2.315265e-08,4.390756e-08,0.299997,0.700001,2.092142e-06
287638,8.903572e-10,4.005322e-10,1.253561e-09,6.612638e-10,1.125664e-09,0.300000,0.700000,2.338481e-08
287646,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,0.300000,0.700000,1.726580e-09


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [anger['expr_0'].max(),anger['expr_1'].max(),anger['expr_2'].max(),anger['expr_3'].max(),anger['expr_4'].max(),anger['expr_5'].max(),anger['expr_6'].max(),anger['expr_7'].max()]
print("Max values:\n",maxs)
mins = [anger['expr_0'].min(),anger['expr_1'].min(),anger['expr_2'].min(),anger['expr_3'].min(),anger['expr_4'].min(),anger['expr_5'].min(),anger['expr_6'].min(),anger['expr_7'].min()]
print("Min values:\n",mins)
means = [anger['expr_0'].mean(),anger['expr_1'].mean(),anger['expr_2'].mean(),anger['expr_3'].mean(),anger['expr_4'].mean(),anger['expr_5'].mean(),anger['expr_6'].mean(),anger['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2999999999893749, 0.023403256260000806, 0.2999999999733653, 0.29999999905764874, 0.3, 0.3, 1.0, 0.3]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0]
Mean values:
 [0.00895019754032473, 0.0024886750373191656, 0.005098688990172643, 0.006038926439817829, 0.02515281120869782, 0.07475110668800462, 0.8022336883729507, 0.07528590572271235]


In [ ]:
anger

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
5,2.771369e-03,1.844927e-03,2.074687e-03,6.992254e-03,1.820512e-01,0.007861,0.753599,4.280568e-02
14,1.690952e-03,1.043902e-03,1.488568e-03,2.521832e-03,9.243834e-03,0.019272,0.778530,1.862088e-01
20,1.344677e-04,6.631567e-05,8.606208e-05,2.049120e-04,1.318915e-03,0.000466,0.985832,1.189139e-02
21,4.977347e-03,2.681464e-03,2.863460e-03,1.254660e-02,1.854905e-01,0.007300,0.758082,2.605935e-02
24,3.039174e-02,1.164128e-02,1.273308e-02,3.470068e-02,6.299744e-02,0.024030,0.776278,4.722802e-02
...,...,...,...,...,...,...,...,...
287605,1.420083e-03,8.811800e-04,1.137244e-03,2.542816e-03,1.740632e-02,0.008500,0.817242,1.508712e-01
287633,2.643163e-08,1.312144e-08,3.210058e-08,2.315265e-08,4.390756e-08,0.299997,0.700001,2.092142e-06
287638,8.903572e-10,4.005322e-10,1.253561e-09,6.612638e-10,1.125664e-09,0.300000,0.700000,2.338481e-08
287646,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,0.300000,0.700000,1.726580e-09


In [ ]:
anger["max"]=anger.idxmax(axis=1)
anger

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
5,2.771369e-03,1.844927e-03,2.074687e-03,6.992254e-03,1.820512e-01,0.007861,0.753599,4.280568e-02,expr_6
14,1.690952e-03,1.043902e-03,1.488568e-03,2.521832e-03,9.243834e-03,0.019272,0.778530,1.862088e-01,expr_6
20,1.344677e-04,6.631567e-05,8.606208e-05,2.049120e-04,1.318915e-03,0.000466,0.985832,1.189139e-02,expr_6
21,4.977347e-03,2.681464e-03,2.863460e-03,1.254660e-02,1.854905e-01,0.007300,0.758082,2.605935e-02,expr_6
24,3.039174e-02,1.164128e-02,1.273308e-02,3.470068e-02,6.299744e-02,0.024030,0.776278,4.722802e-02,expr_6
...,...,...,...,...,...,...,...,...,...
287605,1.420083e-03,8.811800e-04,1.137244e-03,2.542816e-03,1.740632e-02,0.008500,0.817242,1.508712e-01,expr_6
287633,2.643163e-08,1.312144e-08,3.210058e-08,2.315265e-08,4.390756e-08,0.299997,0.700001,2.092142e-06,expr_6
287638,8.903572e-10,4.005322e-10,1.253561e-09,6.612638e-10,1.125664e-09,0.300000,0.700000,2.338481e-08,expr_6
287646,4.768414e-11,2.231422e-11,6.401001e-11,3.757260e-11,6.601766e-11,0.300000,0.700000,1.726580e-09,expr_6


In [ ]:
print(anger.loc[anger['max'] != "expr_6"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### contempt

In [ ]:
contempt_true = true.loc[true[0] == 7]
contempt_true

,0
11,7
12,7
29,7
42,7
47,7
...,...
3931,7
3971,7
3974,7
3985,7


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
contempt_preds = preds.iloc[preds.index.isin(contempt_true.index.tolist())]
contempt_preds

,0
11,2
12,6
29,0
42,2
47,6
...,...
3931,7
3971,7
3974,0
3985,6


In [ ]:
diff = contempt_true.values != contempt_preds.values
result = diff.flatten().sum()
print(result)

390


In [ ]:
contempt_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_7'] == 1]
contempt_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
9,0,0,0,0,0,0,0,1,/content/train_set/images/100012.jpg
38,0,0,0,0,0,0,0,1,/content/train_set/images/100047.jpg
123,0,0,0,0,0,0,0,1,/content/train_set/images/10015.jpg
196,0,0,0,0,0,0,0,1,/content/train_set/images/100251.jpg
294,0,0,0,0,0,0,0,1,/content/train_set/images/100368.jpg
...,...,...,...,...,...,...,...,...,...
286493,0,0,0,0,0,0,0,1,/content/train_set/images/98490.jpg
286534,0,0,0,0,0,0,0,1,/content/train_set/images/98540.jpg
286847,0,0,0,0,0,0,0,1,/content/train_set/images/98971.jpg
286978,0,0,0,0,0,0,0,1,/content/train_set/images/99129.jpg


In [ ]:
contempt = final_train_dataset.loc[final_train_dataset['file_loc'].isin(contempt_old['file_loc'])]
del contempt['file_loc']
contempt

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
9,1.074046e-18,5.129836e-19,8.575738e-19,1.180242e-18,3.742558e-18,4.987394e-17,0.000003,0.999997
38,1.040545e-06,5.554533e-07,9.433664e-07,1.219592e-06,3.606639e-06,1.326885e-04,0.001289,0.998571
123,1.114681e-04,6.161441e-05,1.058289e-04,1.310076e-04,3.583201e-04,1.368592e-02,0.018726,0.966820
196,9.677008e-05,3.792632e-05,6.559208e-05,8.304677e-05,2.042673e-04,9.515113e-04,0.278618,0.719943
294,4.551100e-06,2.415312e-06,4.241924e-06,5.154272e-06,1.423857e-05,1.100738e-03,0.002898,0.995970
...,...,...,...,...,...,...,...,...
286493,7.899143e-07,3.349723e-07,5.429558e-07,7.832008e-07,2.288773e-06,8.005577e-06,0.299052,0.700935
286534,1.352335e-03,7.125314e-04,8.542943e-04,2.569357e-03,3.101044e-02,3.459104e-03,0.221891,0.738151
286847,2.987249e-05,1.613781e-05,2.823789e-05,3.421409e-05,9.310859e-05,6.001276e-03,0.008955,0.984842
286978,4.076321e-06,1.703537e-06,2.756714e-06,3.974492e-06,1.137780e-05,3.642009e-05,0.297506,0.702433


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [contempt['expr_0'].max(),contempt['expr_1'].max(),contempt['expr_2'].max(),contempt['expr_3'].max(),contempt['expr_4'].max(),contempt['expr_5'].max(),contempt['expr_6'].max(),contempt['expr_7'].max()]
print("Max values:\n",maxs)
mins = [contempt['expr_0'].min(),contempt['expr_1'].min(),contempt['expr_2'].min(),contempt['expr_3'].min(),contempt['expr_4'].min(),contempt['expr_5'].min(),contempt['expr_6'].min(),contempt['expr_7'].min()]
print("Min values:\n",mins)
means = [contempt['expr_0'].mean(),contempt['expr_1'].mean(),contempt['expr_2'].mean(),contempt['expr_3'].mean(),contempt['expr_4'].mean(),contempt['expr_5'].mean(),contempt['expr_6'].mean(),contempt['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29446635651634273, 0.18323972660896648, 0.08830696205737015, 0.3, 0.3, 0.29999999999997656, 0.3, 1.0]
Min values:
 [4.87466561722284e-133, 2.413077125250347e-133, 4.0403487915088436e-133, 5.516124139574978e-133, 1.7606666790811774e-132, 3.3197967409522445e-131, 1.6488446320051497e-125, 0.7]
Mean values:
 [0.0050019628198967854, 0.0014206547625206405, 0.0016582218828120451, 0.007830505629593483, 0.012148604745619174, 0.021305184046200926, 0.07416172343276281, 0.8764731426805941]


In [ ]:
contempt

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
9,1.074046e-18,5.129836e-19,8.575738e-19,1.180242e-18,3.742558e-18,4.987394e-17,0.000003,0.999997
38,1.040545e-06,5.554533e-07,9.433664e-07,1.219592e-06,3.606639e-06,1.326885e-04,0.001289,0.998571
123,1.114681e-04,6.161441e-05,1.058289e-04,1.310076e-04,3.583201e-04,1.368592e-02,0.018726,0.966820
196,9.677008e-05,3.792632e-05,6.559208e-05,8.304677e-05,2.042673e-04,9.515113e-04,0.278618,0.719943
294,4.551100e-06,2.415312e-06,4.241924e-06,5.154272e-06,1.423857e-05,1.100738e-03,0.002898,0.995970
...,...,...,...,...,...,...,...,...
286493,7.899143e-07,3.349723e-07,5.429558e-07,7.832008e-07,2.288773e-06,8.005577e-06,0.299052,0.700935
286534,1.352335e-03,7.125314e-04,8.542943e-04,2.569357e-03,3.101044e-02,3.459104e-03,0.221891,0.738151
286847,2.987249e-05,1.613781e-05,2.823789e-05,3.421409e-05,9.310859e-05,6.001276e-03,0.008955,0.984842
286978,4.076321e-06,1.703537e-06,2.756714e-06,3.974492e-06,1.137780e-05,3.642009e-05,0.297506,0.702433


In [ ]:
contempt["max"]=contempt.idxmax(axis=1)
contempt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
9,1.074046e-18,5.129836e-19,8.575738e-19,1.180242e-18,3.742558e-18,4.987394e-17,0.000003,0.999997,expr_7
38,1.040545e-06,5.554533e-07,9.433664e-07,1.219592e-06,3.606639e-06,1.326885e-04,0.001289,0.998571,expr_7
123,1.114681e-04,6.161441e-05,1.058289e-04,1.310076e-04,3.583201e-04,1.368592e-02,0.018726,0.966820,expr_7
196,9.677008e-05,3.792632e-05,6.559208e-05,8.304677e-05,2.042673e-04,9.515113e-04,0.278618,0.719943,expr_7
294,4.551100e-06,2.415312e-06,4.241924e-06,5.154272e-06,1.423857e-05,1.100738e-03,0.002898,0.995970,expr_7
...,...,...,...,...,...,...,...,...,...
286493,7.899143e-07,3.349723e-07,5.429558e-07,7.832008e-07,2.288773e-06,8.005577e-06,0.299052,0.700935,expr_7
286534,1.352335e-03,7.125314e-04,8.542943e-04,2.569357e-03,3.101044e-02,3.459104e-03,0.221891,0.738151,expr_7
286847,2.987249e-05,1.613781e-05,2.823789e-05,3.421409e-05,9.310859e-05,6.001276e-03,0.008955,0.984842,expr_7
286978,4.076321e-06,1.703537e-06,2.756714e-06,3.974492e-06,1.137780e-05,3.642009e-05,0.297506,0.702433,expr_7


In [ ]:
print(contempt.loc[contempt['max'] != "expr_7"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


## Validation set check

Όσον αφορά το validation set θέλουμε απλά να εξετάσουμε τις τιμές των ετικετών με τρόπο παρόμοιο με τον παραπάνω:

In [ ]:
test_dataset_va

,file_loc,expr,valence,arousal
0,/content/val_set/images/0.jpg,0,-0.176846,-0.077640
1,/content/val_set/images/1001.jpg,0,-0.135501,0.004839
2,/content/val_set/images/1002.jpg,4,-0.114510,0.857748
3,/content/val_set/images/1003.jpg,5,-0.642857,0.484127
4,/content/val_set/images/1007.jpg,2,-0.796971,-0.229129
...,...,...,...,...
3994,/content/val_set/images/994.jpg,1,0.468254,0.119048
3995,/content/val_set/images/995.jpg,1,0.531746,0.015873
3996,/content/val_set/images/996.jpg,6,-0.416183,0.701703
3997,/content/val_set/images/999.jpg,3,0.176846,0.599551


Θέλουμε να εφαρμόσουμε μία μορφή data cleaning. Αρχικά παίρνουμε τα true labels και κρατάμε όσα είναι happy.

### happy

#### va > 0

In [ ]:
happy = test_dataset_va.loc[test_dataset_va['expr'] == 1]
happy

,file_loc,expr,valence,arousal
10,/content/val_set/images/1012.jpg,1,0.785714,-0.103175
17,/content/val_set/images/101.jpg,1,0.357143,0.079365
40,/content/val_set/images/1043.jpg,1,0.875919,0.053233
48,/content/val_set/images/1052.jpg,1,0.650794,-0.119048
74,/content/val_set/images/1085.jpg,1,0.556418,-0.090580
...,...,...,...,...
3986,/content/val_set/images/984.jpg,1,0.600506,0.131690
3988,/content/val_set/images/986.jpg,1,0.510957,0.105352
3991,/content/val_set/images/990.jpg,1,0.920635,0.134921
3994,/content/val_set/images/994.jpg,1,0.468254,0.119048


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι και οι δύο θετικές. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
happy_wrong_1 = happy.loc[(happy['valence'] <= 0) | (happy['arousal'] <= 0)]
happy_wrong_1

,file_loc,expr,valence,arousal
10,/content/val_set/images/1012.jpg,1,0.785714,-0.103175
48,/content/val_set/images/1052.jpg,1,0.650794,-0.119048
74,/content/val_set/images/1085.jpg,1,0.556418,-0.090580
86,/content/val_set/images/1099.jpg,1,0.595238,-0.131690
124,/content/val_set/images/114.jpg,1,0.515873,-0.007937
...,...,...,...,...
3869,/content/val_set/images/838.jpg,1,0.871080,-0.120983
3915,/content/val_set/images/895.jpg,1,0.542562,-0.210704
3947,/content/val_set/images/939.jpg,1,0.801587,-0.071429
3951,/content/val_set/images/945.jpg,1,0.879688,-0.094817


Άρα έχουμε 109 εικόνες που είναι annotated ως happy ενώ δεν θα έπρεπε.

#### va > 0.1

In [ ]:
happy = test_dataset_va.loc[test_dataset_va['expr'] == 1]
happy

,file_loc,expr,valence,arousal
10,/content/val_set/images/1012.jpg,1,0.785714,-0.103175
17,/content/val_set/images/101.jpg,1,0.357143,0.079365
40,/content/val_set/images/1043.jpg,1,0.875919,0.053233
48,/content/val_set/images/1052.jpg,1,0.650794,-0.119048
74,/content/val_set/images/1085.jpg,1,0.556418,-0.090580
...,...,...,...,...
3986,/content/val_set/images/984.jpg,1,0.600506,0.131690
3988,/content/val_set/images/986.jpg,1,0.510957,0.105352
3991,/content/val_set/images/990.jpg,1,0.920635,0.134921
3994,/content/val_set/images/994.jpg,1,0.468254,0.119048


In [ ]:
happy_wrong_2 = happy.loc[(happy['valence'] <= 0.1) | (happy['arousal'] <= 0.1)]
happy_wrong_2

,file_loc,expr,valence,arousal
10,/content/val_set/images/1012.jpg,1,0.785714,-0.103175
17,/content/val_set/images/101.jpg,1,0.357143,0.079365
40,/content/val_set/images/1043.jpg,1,0.875919,0.053233
48,/content/val_set/images/1052.jpg,1,0.650794,-0.119048
74,/content/val_set/images/1085.jpg,1,0.556418,-0.090580
...,...,...,...,...
3951,/content/val_set/images/945.jpg,1,0.879688,-0.094817
3961,/content/val_set/images/95.jpg,1,0.547619,0.015873
3966,/content/val_set/images/964.jpg,1,0.793651,-0.253968
3975,/content/val_set/images/972.jpg,1,0.809524,0.087302


Άρα έχουμε 287 εικόνες που είναι annotated ως happy ενώ δεν θα έπρεπε.

### neutral

In [ ]:
neutral = test_dataset_va.loc[test_dataset_va['expr'] == 0]
neutral

,file_loc,expr,valence,arousal
0,/content/val_set/images/0.jpg,0,-0.176846,-0.077640
1,/content/val_set/images/1001.jpg,0,-0.135501,0.004839
14,/content/val_set/images/1017.jpg,0,0.010535,0.005268
16,/content/val_set/images/1019.jpg,0,0.099621,0.039849
23,/content/val_set/images/1025.jpg,0,0.000000,-0.007937
...,...,...,...,...
3953,/content/val_set/images/947.jpg,0,0.109584,0.059773
3963,/content/val_set/images/961.jpg,0,-0.131690,-0.410872
3982,/content/val_set/images/979.jpg,0,-0.044830,0.059773
3984,/content/val_set/images/981.jpg,0,-0.023810,-0.071429


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι ανάμεσα στο -0.1 και στο 0.1. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
neutral_wrong = neutral.loc[((neutral['valence'] > 0.1) | (neutral['valence'] < -0.1)) | ((neutral['arousal'] > 0.1) | (neutral['arousal'] < -0.1))]
neutral_wrong

,file_loc,expr,valence,arousal
0,/content/val_set/images/0.jpg,0,-0.176846,-0.077640
1,/content/val_set/images/1001.jpg,0,-0.135501,0.004839
24,/content/val_set/images/1026.jpg,0,-0.111111,-0.071429
26,/content/val_set/images/1029.jpg,0,-0.044504,-0.148346
90,/content/val_set/images/1103.jpg,0,-0.284450,-0.121155
...,...,...,...,...
3894,/content/val_set/images/867.jpg,0,-0.150794,0.150794
3924,/content/val_set/images/906.jpg,0,0.053233,-0.387147
3953,/content/val_set/images/947.jpg,0,0.109584,0.059773
3963,/content/val_set/images/961.jpg,0,-0.131690,-0.410872


Άρα έχουμε 151 εικόνες που είναι annotated ως neutral ενώ δεν θα έπρεπε.

### sad

In [ ]:
sad = test_dataset_va.loc[test_dataset_va['expr'] == 2]
sad

,file_loc,expr,valence,arousal
4,/content/val_set/images/1007.jpg,2,-0.796971,-0.229129
8,/content/val_set/images/1010.jpg,2,-0.539165,-0.181159
13,/content/val_set/images/1016.jpg,2,-0.405757,-0.279436
20,/content/val_set/images/1022.jpg,2,-0.111111,-0.333333
21,/content/val_set/images/1023.jpg,2,-0.119048,-0.357143
...,...,...,...,...
3939,/content/val_set/images/928.jpg,2,-0.431332,-0.086266
3958,/content/val_set/images/957.jpg,2,-0.851763,-0.179319
3973,/content/val_set/images/970.jpg,2,-0.690476,-0.404762
3976,/content/val_set/images/973.jpg,2,-0.658730,-0.317460


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
sad_wrong = sad.loc[sad['valence'] >= 0]
sad_wrong

,file_loc,expr,valence,arousal


Άρα δεν έχουμε καμία που είναι annotated ως sad ενώ δεν θα έπρεπε.

### fear

In [ ]:
fear = test_dataset_va.loc[test_dataset_va['expr'] == 4]
fear

,file_loc,expr,valence,arousal
2,/content/val_set/images/1002.jpg,4,-0.114510,0.857748
9,/content/val_set/images/1011.jpg,4,-0.150794,0.603175
19,/content/val_set/images/1021.jpg,4,-0.119546,0.831839
28,/content/val_set/images/1030.jpg,4,-0.067751,0.958188
31,/content/val_set/images/1033.jpg,4,-0.048393,0.604917
...,...,...,...,...
3970,/content/val_set/images/968.jpg,4,-0.212931,0.788811
3972,/content/val_set/images/96.jpg,4,-0.158730,0.825397
3977,/content/val_set/images/974.jpg,4,-0.103520,0.672878
3990,/content/val_set/images/98.jpg,4,-0.101626,0.629113


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
fear_wrong = fear.loc[fear['valence'] >= 0]
fear_wrong

,file_loc,expr,valence,arousal
1657,/content/val_set/images/3054.jpg,4,0.246806,0.754936
1862,/content/val_set/images/3303.jpg,4,0.212931,0.798490
2290,/content/val_set/images/3828.jpg,4,0.246806,0.764615


Άρα έχουμε 3 εικόνες που είναι annotated ως fear ενώ δεν θα έπρεπε.

### disgust

In [ ]:
disgust = test_dataset_va.loc[test_dataset_va['expr'] == 5]
disgust

,file_loc,expr,valence,arousal
3,/content/val_set/images/1003.jpg,5,-0.642857,0.484127
15,/content/val_set/images/1018.jpg,5,-0.793651,0.512969
57,/content/val_set/images/1062.jpg,5,-0.674603,0.484127
64,/content/val_set/images/1071.jpg,5,-0.415100,0.642231
107,/content/val_set/images/1130.jpg,5,-0.764615,0.314557
...,...,...,...,...
3960,/content/val_set/images/959.jpg,5,-0.904762,0.380952
3964,/content/val_set/images/962.jpg,5,-0.738095,0.420635
3979,/content/val_set/images/976.jpg,5,-0.912698,0.357143
3980,/content/val_set/images/977.jpg,5,-0.764615,0.440379


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
disgust_wrong = disgust.loc[disgust['valence'] >= 0]
disgust_wrong

,file_loc,expr,valence,arousal
294,/content/val_set/images/1372.jpg,5,0.212931,0.822687


Άρα έχουμε 1 εικόνα που είναι annotated ως disgust ενώ δεν θα έπρεπε.

### anger

In [ ]:
anger = test_dataset_va.loc[test_dataset_va['expr'] == 6]
anger

,file_loc,expr,valence,arousal
5,/content/val_set/images/1008.jpg,6,-0.336439,0.655625
18,/content/val_set/images/1020.jpg,6,-0.325397,0.404762
44,/content/val_set/images/1049.jpg,6,-0.516224,0.732195
59,/content/val_set/images/1064.jpg,6,-0.396825,0.785714
82,/content/val_set/images/1094.jpg,6,-0.563492,0.261905
...,...,...,...,...
3968,/content/val_set/images/966.jpg,6,-0.146653,0.345066
3969,/content/val_set/images/967.jpg,6,-0.401665,0.754936
3978,/content/val_set/images/975.jpg,6,-0.464576,0.856562
3983,/content/val_set/images/980.jpg,6,-0.700723,0.290084


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
anger_wrong = anger.loc[anger['valence'] >= 0]
anger_wrong

,file_loc,expr,valence,arousal


Άρα δεν έχουμε καμία εικόνα που είναι annotated ως anger ενώ δεν θα έπρεπε.

### contempt

In [ ]:
contempt = test_dataset_va.loc[test_dataset_va['expr'] == 7]
contempt

,file_loc,expr,valence,arousal
11,/content/val_set/images/1013.jpg,7,-0.619048,0.642857
12,/content/val_set/images/1014.jpg,7,-0.284392,0.608466
29,/content/val_set/images/1031.jpg,7,-0.227449,0.271003
42,/content/val_set/images/1046.jpg,7,-0.624274,0.662989
47,/content/val_set/images/1051.jpg,7,-0.569358,0.631470
...,...,...,...,...
3931,/content/val_set/images/918.jpg,7,-0.493612,0.512969
3971,/content/val_set/images/969.jpg,7,-0.421022,0.464576
3974,/content/val_set/images/971.jpg,7,-0.648471,0.648471
3985,/content/val_set/images/982.jpg,7,-0.304878,0.324235


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
contempt_wrong = contempt.loc[contempt['valence'] >= 0]
contempt_wrong

,file_loc,expr,valence,arousal
52,/content/val_set/images/1058.jpg,7,0.261324,0.774293
218,/content/val_set/images/1271.jpg,7,0.150794,0.468254
1122,/content/val_set/images/2392.jpg,7,0.253968,0.404762
1135,/content/val_set/images/2415.jpg,7,0.166667,0.396825
1188,/content/val_set/images/2475.jpg,7,0.253968,0.420635
1319,/content/val_set/images/2629.jpg,7,0.469415,0.358111
1349,/content/val_set/images/2668.jpg,7,0.142857,0.333333
1518,/content/val_set/images/2874.jpg,7,0.038715,0.793651
1606,/content/val_set/images/298.jpg,7,0.329075,0.033875
2560,/content/val_set/images/4168.jpg,7,0.087108,0.614595


Άρα έχουμε 13 εικόνες που είναι annotated ως contempt ενώ δεν θα έπρεπε.

# Data cleaning + Label smoothing

Θα κάνουμε το relabelling για λ=0.9 που έβγαλε τα καλύτερα αποτελέσματα. Επομένως τώρα θέλουμε να διαγράψουμε από το train_dataset τις εικόνες που έχουν λάθος annotation.

Τώρα θέλουμε να δημιουργήσουμε 7 clusters όπου για κάθε συναίσθημα έχουμε την μέση τιμή valence και arousal.

0: Neutral, 1: Happiness, 2: Sadness, 3: Surprise, 4: Fear, 5: Disgust, 6: Anger, 7: Contempt

In [ ]:
train_dataset = pd.read_pickle("./train_dataset.pkl")  

In [ ]:
train_dataset

,file_loc,expr,valence,arousal
0,/content/train_set/images/0.jpg,1,0.785714,-0.055556
1,/content/train_set/images/100000.jpg,1,0.784476,-0.137627
2,/content/train_set/images/100001.jpg,1,0.774293,0.091947
3,/content/train_set/images/100002.jpg,0,0.089659,0.114565
4,/content/train_set/images/100004.jpg,1,0.430701,0.048393
...,...,...,...,...
287646,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,3,0.406504,0.609756


In [ ]:
new_min_train_dataset = train_dataset.loc[~train_dataset['file_loc'].isin(happy_wrong_2['file_loc']) & ~train_dataset['file_loc'].isin(neutral_wrong['file_loc']) & ~train_dataset['file_loc'].isin(sad_wrong['file_loc']) & ~train_dataset['file_loc'].isin(fear_wrong['file_loc']) & ~train_dataset['file_loc'].isin(disgust_wrong['file_loc']) & ~train_dataset['file_loc'].isin(anger_wrong['file_loc']) & ~train_dataset['file_loc'].isin(contempt_wrong['file_loc'])]
new_min_train_dataset

,file_loc,expr,valence,arousal
5,/content/train_set/images/100006.jpg,6,-0.341270,0.833333
6,/content/train_set/images/100008.jpg,3,0.150966,0.534852
7,/content/train_set/images/100009.jpg,4,-0.126422,0.758533
9,/content/train_set/images/100012.jpg,7,-0.493612,0.561363
10,/content/train_set/images/100013.jpg,0,-0.067751,-0.043554
...,...,...,...,...
287646,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
287647,/content/train_set/images/99998.jpg,0,0.017253,0.012940
287648,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
287649,/content/train_set/images/9999.jpg,3,0.406504,0.609756


In [ ]:
mean_va = []

In [ ]:
neutral_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 0]["valence"].mean()
neutral_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 0]["arousal"].mean()
mean_va.append((neutral_mean_valence,neutral_mean_arousal))

happy_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 1]["valence"].mean()
happy_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 1]["arousal"].mean()
mean_va.append((happy_mean_valence,happy_mean_arousal))

sad_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 2]["valence"].mean()
sad_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 2]["arousal"].mean()
mean_va.append((sad_mean_valence,sad_mean_arousal))

surprise_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 3]["valence"].mean()
surprise_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 3]["arousal"].mean()
mean_va.append((surprise_mean_valence,surprise_mean_arousal))

fear_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 4]["valence"].mean()
fear_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 4]["arousal"].mean()
mean_va.append((fear_mean_valence,fear_mean_arousal))

disgust_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 5]["valence"].mean()
disgust_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 5]["arousal"].mean()
mean_va.append((disgust_mean_valence,disgust_mean_arousal))

anger_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 6]["valence"].mean()
anger_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 6]["arousal"].mean()
mean_va.append((anger_mean_valence,anger_mean_arousal))

contempt_mean_valence = new_min_train_dataset[new_min_train_dataset["expr"] == 7]["valence"].mean()
contempt_mean_arousal = new_min_train_dataset[new_min_train_dataset["expr"] == 7]["arousal"].mean()
mean_va.append((contempt_mean_valence,contempt_mean_arousal))

In [ ]:
print("Mean valence,arousal: \n",mean_va)

Mean valence,arousal: 
 [(0.00024209404136427959, -0.004937479616035614), (0.6929492590550383, 0.20108133050047217), (-0.6371660958395537, -0.25706700242240904), (0.1804439995542938, 0.6893279744258338), (-0.1266877410285084, 0.7662884124744054), (-0.6955601125691698, 0.4568032009934124), (-0.45289340818010054, 0.5667930499931659), (-0.5356136801730295, 0.5867818174485032)]


In [ ]:
print(len(mean_va))

8


Με βάση αυτά τα 7 clusters θέλουμε για κάθε εικόνα να βρούμε την ευκλείδια απόσταση των τιμών valence και arousal της από τα 7 clusters. Για την ευκλείδια απόσταση χρησιμοποιούμε την συνάρτηση `numpy.linalg.norm(a-b)`.

In [ ]:
new_one_hot_train_dataset = pd.get_dummies(new_min_train_dataset.expr, prefix='expr')
new_one_hot_train_dataset["file_loc"]=new_min_train_dataset["file_loc"]
new_one_hot_train_dataset["valence"]=new_min_train_dataset["valence"]
new_one_hot_train_dataset["arousal"]=new_min_train_dataset["arousal"]
new_one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
5,0,0,0,0,0,0,1,0,/content/train_set/images/100006.jpg,-0.341270,0.833333
6,0,0,0,1,0,0,0,0,/content/train_set/images/100008.jpg,0.150966,0.534852
7,0,0,0,0,1,0,0,0,/content/train_set/images/100009.jpg,-0.126422,0.758533
9,0,0,0,0,0,0,0,1,/content/train_set/images/100012.jpg,-0.493612,0.561363
10,1,0,0,0,0,0,0,0,/content/train_set/images/100013.jpg,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
index = new_one_hot_train_dataset.index
print(list(index))

[5, 6, 7, 9, 10, 11, 14, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 31, 32, 33, 35, 36, 38, 39, 41, 43, 44, 46, 47, 49, 51, 56, 57, 58, 59, 60, 61, 62, 64, 68, 70, 71, 72, 74, 75, 76, 78, 79, 81, 82, 83, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 99, 100, 102, 106, 113, 114, 115, 116, 120, 121, 123, 127, 128, 131, 135, 137, 138, 139, 140, 142, 144, 145, 146, 150, 151, 152, 153, 154, 155, 160, 163, 164, 166, 167, 168, 170, 172, 173, 174, 177, 178, 179, 181, 182, 183, 185, 186, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 211, 213, 214, 215, 216, 218, 219, 222, 223, 227, 228, 229, 230, 233, 234, 235, 236, 238, 239, 240, 241, 242, 244, 246, 248, 249, 251, 253, 254, 256, 257, 259, 260, 261, 263, 264, 265, 267, 270, 271, 273, 274, 275, 276, 277, 280, 282, 283, 284, 287, 288, 293, 294, 296, 297, 299, 300, 301, 306, 307, 308, 309, 310, 311, 313, 314, 315, 316, 318, 319, 320, 321, 323, 324, 325, 326, 328, 329, 330, 332, 333, 335, 336, 337, 33

In [ ]:
list_of_distances = []
for j in list(new_one_hot_train_dataset.index): #for every image
  # print(j)
  distances = []
  for i in mean_va: #for every cluster`
    # print(i)
    a = np.array((float(new_one_hot_train_dataset.loc[j]["valence"]),float(new_one_hot_train_dataset.loc[j]["arousal"])))
    # print(a)
    i = np.array(i)
    # print(i)
    distance = np.linalg.norm(a-i)
    # print(distance)
    distances.append(distance)
  list_of_distances.append(distances)

In [ ]:
print(list_of_distances[:100])

[[0.9051673366688703, 1.2121681605228265, 1.1298347953642958, 0.5412235626074996, 0.22481219402377933, 0.5170069374817096, 0.2889694278005667, 0.31393781492636125], [0.5604376665888707, 0.6365129322441799, 1.1172680550745873, 0.15726340684512025, 0.3614614957725776, 0.8501165063024253, 0.6047035765747439, 0.6885407490967651], [0.7739063030913103, 0.9910204962528524, 1.1367950107012963, 0.314572848871572, 0.007759963978163498, 0.6441731621135367, 0.3786129802155772, 0.4437750554398753], [0.7513907767698301, 1.24005262100753, 0.8309244534236889, 0.686095128400532, 0.4202711464038228, 0.22741106336004868, 0.041079059905551205, 0.04909437256792056], [0.07819370619248406, 0.7990687533844708, 0.6081296697436719, 0.7737677928972972, 0.8119841795549719, 0.8028088110480652, 0.721705221748923, 0.7849961853250379], [0.8576784153454463, 0.17859512597909924, 1.5424299332686293, 0.6821229073323419, 0.9914212809064787, 1.475875438852069, 1.2474302981822232, 1.3323393407840705], [0.9973644636781784, 1

In [ ]:
len(list_of_distances)

180637

Τώρα θέλουμε να δημιουργήσουμε μία νέα λίστα όπου κάθε στοιχείο κάθε υπολίστας θα υπολογίζεται σύμφωνα με τον τύπο 
`max(υπολίστας)/στοιχείο υπολίστας`

In [ ]:
list_of_distances[2]

[0.7739063030913103,
 0.9910204962528524,
 1.1367950107012963,
 0.314572848871572,
 0.007759963978163498,
 0.6441731621135367,
 0.3786129802155772,
 0.4437750554398753]

In [ ]:
new_list_of_distances = []
for lists in list_of_distances: #for every list
  new_distances = []
  # print(lists)
  maxim = max(lists)
  # print(maxim)
  for element in lists:
    new_distances.append(maxim/element)
  new_list_of_distances.append(new_distances)

In [ ]:
print(new_list_of_distances[:100])

[[1.339164717303828, 1.0, 1.0728720389001507, 2.2396810565357854, 5.39191464140335, 2.3445878046185986, 4.194797248099922, 3.8611728275141317], [1.993563462418381, 1.7552951377364623, 1.0, 7.10443756426389, 3.0909739160088687, 1.314252866273748, 1.8476293151814824, 1.6226607597883365], [1.4689052229713784, 1.1470953577646799, 1.0, 3.613773454318068, 146.49488243762883, 1.764735132664428, 3.002525190906081, 2.5616469352349944], [1.6503431494573553, 1.0, 1.4923770938478162, 1.8074062468544536, 2.9506013715631334, 5.452912460306366, 30.186976621632862, 25.258549119697072], [10.384265167789419, 1.016163097500431, 1.3352155304266362, 1.04938999401174, 1.0, 1.0114290829655048, 1.1250911800073637, 1.0343797775510963], [1.7983779300862857, 8.636461520507217, 1.0, 2.2612199600520544, 1.5557765028589572, 1.045094926485345, 1.2364858665981455, 1.157685498021039], [1.4120233556903063, 1.0, 1.320619538103619, 1.8345950206440662, 3.107326075569705, 3.8057201769055933, 5.188098511480564, 6.2608908510

Στην συνέχεια τις περνάμε από την συνάρτηση softmax.

In [ ]:
list_of_softmax = []
for i in new_list_of_distances:
  list_of_softmax.append(list(softmax(i)))

In [ ]:
print(list_of_softmax[:100])

[[0.010518892781647199, 0.007493292114241966, 0.008059731751184842, 0.025885663745357383, 0.6054202693671521, 0.02874880037231439, 0.18287547992561204, 0.13099786994248974], [0.005779305682155915, 0.004554042165457387, 0.0021398165238852105, 0.958296906392291, 0.017317092246082823, 0.0029299136603272127, 0.0049945604190684495, 0.0039883629107317535], [1.0375722860194786e-63, 7.520697390458793e-64, 6.491953675445935e-64, 8.861817467161227e-63, 1.0, 1.39474772086458e-63, 4.8090694886602053e-63, 3.094496948928965e-63], [4.013890331732245e-13, 2.094715572208663e-13, 3.427375722800242e-13, 4.696530412301515e-13, 1.4731958044531424e-12, 1.7988724526360237e-11, 0.9928141344985777, 0.0071858654805387915], [0.9993579664098803, 8.53504949164593e-05, 0.0001174272133537039, 8.823406777651332e-05, 8.398205544714737e-05, 8.494739931598812e-05, 9.517281367754781e-05, 8.691954563173586e-05], [0.0010659945662127835, 0.9942531166827718, 0.0004797598073307697, 0.0016934201788592475, 0.0008363624569958449

In [ ]:
print(list_of_softmax[0])
print(type(list_of_softmax[0]))

[0.010518892781647199, 0.007493292114241966, 0.008059731751184842, 0.025885663745357383, 0.6054202693671521, 0.02874880037231439, 0.18287547992561204, 0.13099786994248974]
<class 'list'>


Τώρα θέλουμε να δημιουργήσουμε έναν γραμμικό συνδυασμό των πραγματικών labels και αυτών που προέκυψαν από label smoothing με βάση τον τύπο

`new_labels = λ*real_labels + (1-λ)*smoothed_labels`



In [ ]:
new_one_hot_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc,valence,arousal
5,0,0,0,0,0,0,1,0,/content/train_set/images/100006.jpg,-0.341270,0.833333
6,0,0,0,1,0,0,0,0,/content/train_set/images/100008.jpg,0.150966,0.534852
7,0,0,0,0,1,0,0,0,/content/train_set/images/100009.jpg,-0.126422,0.758533
9,0,0,0,0,0,0,0,1,/content/train_set/images/100012.jpg,-0.493612,0.561363
10,1,0,0,0,0,0,0,0,/content/train_set/images/100013.jpg,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg,-0.753266,0.447745
287647,1,0,0,0,0,0,0,0,/content/train_set/images/99998.jpg,0.017253,0.012940
287648,0,0,0,0,0,0,1,0,/content/train_set/images/99999.jpg,-0.695322,0.400337
287649,0,0,0,1,0,0,0,0,/content/train_set/images/9999.jpg,0.406504,0.609756


In [ ]:
labels = list(zip(new_one_hot_train_dataset["expr_0"],new_one_hot_train_dataset["expr_1"],new_one_hot_train_dataset["expr_2"],new_one_hot_train_dataset["expr_3"],new_one_hot_train_dataset["expr_4"],new_one_hot_train_dataset["expr_5"],new_one_hot_train_dataset["expr_6"],new_one_hot_train_dataset["expr_7"]))
labels = [list(x) for x in labels]

### λ = 0.9

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.9 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.1 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.0010518892781647199, 0.0007493292114241967, 0.0008059731751184843, 0.0025885663745357384, 0.06054202693671521, 0.002874880037231439, 0.9182875479925612, 0.013099786994248975], [0.0005779305682155916, 0.0004554042165457387, 0.00021398165238852105, 0.9958296906392291, 0.0017317092246082824, 0.0002929913660327213, 0.000499456041906845, 0.0003988362910731754], [1.0375722860194786e-64, 7.5206973904587935e-65, 6.491953675445935e-65, 8.861817467161227e-64, 1.0, 1.39474772086458e-64, 4.809069488660206e-64, 3.0944969489289655e-64], [4.0138903317322456e-14, 2.094715572208663e-14, 3.4273757228002423e-14, 4.696530412301516e-14, 1.4731958044531425e-13, 1.7988724526360239e-12, 0.09928141344985777, 0.9007185865480539], [0.9999357966409881, 8.53504949164593e-06, 1.174272133537039e-05, 8.823406777651332e-06, 8.398205544714738e-06, 8.494739931598813e-06, 9.517281367754783e-06, 8.691954563173587e-06], [0.00010659945662127836, 0.9994253116682772, 4.797598073307697e-05, 0.00016934201788592476, 8.363624

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
valence_list = new_one_hot_train_dataset["valence"]
arousal_list = new_one_hot_train_dataset["arousal"]
image_locations = new_one_hot_train_dataset["file_loc"]
new_final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
new_final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/100006.jpg,1.051889e-03,7.493292e-04,8.059732e-04,2.588566e-03,6.054203e-02,2.874880e-03,9.182875e-01,1.309979e-02,-0.341270,0.833333
1,/content/train_set/images/100008.jpg,5.779306e-04,4.554042e-04,2.139817e-04,9.958297e-01,1.731709e-03,2.929914e-04,4.994560e-04,3.988363e-04,0.150966,0.534852
2,/content/train_set/images/100009.jpg,1.037572e-64,7.520697e-65,6.491954e-65,8.861817e-64,1.000000e+00,1.394748e-64,4.809069e-64,3.094497e-64,-0.126422,0.758533
3,/content/train_set/images/100012.jpg,4.013890e-14,2.094716e-14,3.427376e-14,4.696530e-14,1.473196e-13,1.798872e-12,9.928141e-02,9.007186e-01,-0.493612,0.561363
4,/content/train_set/images/100013.jpg,9.999358e-01,8.535049e-06,1.174272e-05,8.823407e-06,8.398206e-06,8.494740e-06,9.517281e-06,8.691955e-06,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
180632,/content/train_set/images/99997.jpg,6.561629e-12,3.361014e-12,9.641838e-12,5.659920e-12,9.968684e-12,1.000000e-01,9.000000e-01,3.623878e-10,-0.753266,0.447745
180633,/content/train_set/images/99998.jpg,1.000000e+00,5.521700e-16,5.461038e-16,5.575203e-16,4.984465e-16,4.533485e-16,5.298036e-16,4.784074e-16,0.017253,0.012940
180634,/content/train_set/images/99999.jpg,9.327996e-12,4.440779e-12,1.367978e-11,7.475426e-12,1.299902e-11,1.000000e-01,9.000000e-01,4.945945e-10,-0.695322,0.400337
180635,/content/train_set/images/9999.jpg,1.878880e-03,4.517033e-03,8.100785e-04,9.856618e-01,3.424049e-03,1.008778e-03,1.442054e-03,1.257341e-03,0.406504,0.609756


In [ ]:
new_final_train_dataset.to_pickle("./new_final_train_dataset_l_0.9.pkl")  

In [ ]:
new_final_train_dataset = pd.read_pickle("./new_final_train_dataset_l_0.9.pkl")  

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=new_final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy, lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 0.9727 - accuracy: 0.6709
Epoch 1: val_accuracy improved from -inf to 0.46337, saving model to /content/weights0.4634.hdf5
1985/1985 [==============================] - 445s 222ms/step - loss: 0.9727 - accuracy: 0.6709 - val_loss: 1.5350 - val_accuracy: 0.4634
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.7890 - accuracy: 0.7358
Epoch 2: val_accuracy improved from 0.46337 to 0.46737, saving model to /content/weights0.4674.hdf5
1985/1985 [==============================] - 439s 221ms/step - loss: 0.7890 - accuracy: 0.7358 - val_loss: 1.6298 - val_accuracy: 0.4674
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.6816 - accuracy: 0.7741
Epoch 3: val_accuracy improved from 0.46737 to 0.49962, saving model to /content/weights0.4996.hdf5
1985/1985 [==============================] - 439s 221ms/step - loss: 0.6816 - accuracy: 0.7741 - val_loss: 1.5234 - val_accuracy: 0.4996
Epoch 4/1

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 0.9679 - accuracy: 0.6727
Epoch 1: val_accuracy improved from -inf to 0.45986, saving model to /content/weights0.4599.hdf5
1985/1985 [==============================] - 484s 234ms/step - loss: 0.9679 - accuracy: 0.6727 - val_loss: 1.5000 - val_accuracy: 0.4599
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.7855 - accuracy: 0.7374
Epoch 2: val_accuracy improved from 0.45986 to 0.48787, saving model to /content/weights0.4879.hdf5
1985/1985 [==============================] - 426s 214ms/step - loss: 0.7855 - accuracy: 0.7374 - val_loss: 1.5022 - val_accuracy: 0.4879
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.6786 - accuracy: 0.7755
Epoch 3: val_accuracy improved from 0.48787 to 0.50663, saving model to /content/weights0.5066.hdf5
1985/1985 [==============================] - 426s 215ms/step - loss: 0.6786 - accuracy: 0.7755 - val_loss: 1.4687 - val_accuracy: 0.506

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5131.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5131282820705176
f1 score 0.4896864531310743
average accuracy 0.5130360721442886


### λ = 0.7

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.7 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.3 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.0031556678344941596, 0.0022479876342725896, 0.0024179195253554524, 0.007765699123607215, 0.1816260808101456, 0.008624640111694316, 0.7548626439776835, 0.039299360982746924], [0.0017337917046467745, 0.001366212649637216, 0.0006419449571655632, 0.9874890719176872, 0.0051951276738248465, 0.0008789740980981637, 0.0014983681257205348, 0.001196508873219526], [3.1127168580584356e-64, 2.256209217137638e-64, 1.9475861026337806e-64, 2.6585452401483676e-63, 1.0, 4.18424316259374e-64, 1.4427208465980617e-63, 9.283490846786895e-64], [1.2041670995196736e-13, 6.284146716625989e-14, 1.0282127168400725e-13, 1.4089591236904545e-13, 4.419587413359427e-13, 5.396617357908071e-12, 0.2978442403495733, 0.7021557596441615], [0.999807389922964, 2.560514847493779e-05, 3.522816400611117e-05, 2.6470220332953995e-05, 2.519461663414421e-05, 2.5484219794796437e-05, 2.855184410326434e-05, 2.6075863689520758e-05], [0.000319798369863835, 0.9982759350048315, 0.0001439279421992309, 0.0005080260536577743, 0.00025090873

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
valence_list = new_one_hot_train_dataset["valence"]
arousal_list = new_one_hot_train_dataset["arousal"]
image_locations = new_one_hot_train_dataset["file_loc"]
new_final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
new_final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/100006.jpg,3.155668e-03,2.247988e-03,2.417920e-03,7.765699e-03,1.816261e-01,8.624640e-03,7.548626e-01,3.929936e-02,-0.341270,0.833333
1,/content/train_set/images/100008.jpg,1.733792e-03,1.366213e-03,6.419450e-04,9.874891e-01,5.195128e-03,8.789741e-04,1.498368e-03,1.196509e-03,0.150966,0.534852
2,/content/train_set/images/100009.jpg,3.112717e-64,2.256209e-64,1.947586e-64,2.658545e-63,1.000000e+00,4.184243e-64,1.442721e-63,9.283491e-64,-0.126422,0.758533
3,/content/train_set/images/100012.jpg,1.204167e-13,6.284147e-14,1.028213e-13,1.408959e-13,4.419587e-13,5.396617e-12,2.978442e-01,7.021558e-01,-0.493612,0.561363
4,/content/train_set/images/100013.jpg,9.998074e-01,2.560515e-05,3.522816e-05,2.647022e-05,2.519462e-05,2.548422e-05,2.855184e-05,2.607586e-05,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
180632,/content/train_set/images/99997.jpg,1.968489e-11,1.008304e-11,2.892551e-11,1.697976e-11,2.990605e-11,3.000000e-01,7.000000e-01,1.087163e-09,-0.753266,0.447745
180633,/content/train_set/images/99998.jpg,1.000000e+00,1.656510e-15,1.638311e-15,1.672561e-15,1.495339e-15,1.360045e-15,1.589411e-15,1.435222e-15,0.017253,0.012940
180634,/content/train_set/images/99999.jpg,2.798399e-11,1.332234e-11,4.103933e-11,2.242628e-11,3.899706e-11,3.000000e-01,7.000000e-01,1.483784e-09,-0.695322,0.400337
180635,/content/train_set/images/9999.jpg,5.636639e-03,1.355110e-02,2.430235e-03,9.569854e-01,1.027215e-02,3.026335e-03,4.326163e-03,3.772024e-03,0.406504,0.609756


In [ ]:
new_final_train_dataset.to_pickle("./new_final_train_dataset_l_0.7.pkl")  

In [ ]:
new_final_train_dataset = pd.read_pickle("./new_final_train_dataset_l_0.7.pkl")  

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=new_final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy, lr = 10^-4 (best results)

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


1985/1985 [==============================] - ETA: 0s - loss: 1.0300 - accuracy: 0.6727
Epoch 1: val_accuracy improved from -inf to 0.47262, saving model to /content/weights0.4726.hdf5
1985/1985 [==============================] - 440s 219ms/step - loss: 1.0300 - accuracy: 0.6727 - val_loss: 1.4580 - val_accuracy: 0.4726
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8607 - accuracy: 0.7359
Epoch 2: val_accuracy did not improve from 0.47262
1985/1985 [==============================] - 432s 218ms/step - loss: 0.8607 - accuracy: 0.7359 - val_loss: 1.5908 - val_accuracy: 0.4524
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.7628 - accuracy: 0.7755
Epoch 3: val_accuracy improved from 0.47262 to 0.48587, saving model to /content/weights0.4859.hdf5
1985/1985 [==============================] - 434s 219ms/step - loss: 0.7628 - accuracy: 0.7755 - val_loss: 1.5293 - val_accuracy: 0.4859
Epoch 4/10
1985/1985 [==============================] - ET

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5201.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5201300325081271
f1 score 0.5025272152449066
average accuracy 0.5200365731462926


###### 2nd version of happy, lr = 10^-4, kl divergence

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="kl_divergence" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8581 - accuracy: 0.6725
Epoch 1: val_accuracy improved from -inf to 0.47112, saving model to /content/weights0.4711.hdf5
1985/1985 [==============================] - 440s 219ms/step - loss: 0.8581 - accuracy: 0.6725 - val_loss: 1.4819 - val_accuracy: 0.4711
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.6860 - accuracy: 0.7366
Epoch 2: val_accuracy improved from 0.47112 to 0.48837, saving model to /content/weights0.4884.hdf5
1985/1985 [==============================] - 432s 217ms/step - loss: 0.6860 - accuracy: 0.7366 - val_loss: 1.4578 - val_accuracy: 0.4884
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.5865 - accuracy: 0.7747
Epoch 3: val_accuracy improved from 0.48837 to 0.51488, saving model to /content/weights0.5149.hdf5
1985/1985 [==============================] - 432s 217ms/step - loss: 0.5865 - accuracy: 0.7747 - val_loss: 1.4414 - val_accuracy: 0.514

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5206.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5206301575393848
f1 score 0.5099315004037301
average accuracy 0.520557114228457


### λ = 0.5

In [ ]:
new_labels = []
for h in range(len(labels)):
  real_labels = [i * 0.5 for i in labels[h]]
  # print(real_labels)
  smoothed_labels = [j * 0.5 for j in list_of_softmax[h]]
  # print(smoothed_labels)
  new_labels.append([sum(x) for x in zip(real_labels, smoothed_labels)])

In [ ]:
print(new_labels[:100])

[[0.005259446390823599, 0.003746646057120983, 0.004029865875592421, 0.012942831872678692, 0.30271013468357605, 0.014374400186157195, 0.591437739962806, 0.06549893497124487], [0.0028896528410779576, 0.0022770210827286934, 0.0010699082619426053, 0.9791484531961455, 0.008658546123041412, 0.0014649568301636064, 0.0024972802095342248, 0.0019941814553658768], [5.187861430097393e-64, 3.7603486952293966e-64, 3.2459768377229676e-64, 4.430908733580613e-63, 1.0, 6.9737386043229e-64, 2.4045347443301027e-63, 1.5472484744644826e-63], [2.0069451658661226e-13, 1.0473577861043315e-13, 1.713687861400121e-13, 2.3482652061507577e-13, 7.365979022265712e-13, 8.994362263180119e-12, 0.49640706724928885, 0.5035929327402694], [0.9996789832049402, 4.267524745822965e-05, 5.871360667685195e-05, 4.411703388825666e-05, 4.1991027723573686e-05, 4.247369965799406e-05, 4.758640683877391e-05, 4.345977281586793e-05], [0.0005329972831063917, 0.997126558341386, 0.00023987990366538484, 0.0008467100894296237, 0.00041818122849

Πλέον με τα new_labels δημιουργούμε ένα νέο dataframe:

In [ ]:
expr_0 = [item[0] for item in new_labels]
expr_1 = [item[1] for item in new_labels]
expr_2 = [item[2] for item in new_labels]
expr_3 = [item[3] for item in new_labels]
expr_4 = [item[4] for item in new_labels]
expr_5 = [item[5] for item in new_labels]
expr_6 = [item[6] for item in new_labels]
expr_7 = [item[7] for item in new_labels]

In [ ]:
valence_list = new_one_hot_train_dataset["valence"]
arousal_list = new_one_hot_train_dataset["arousal"]
image_locations = new_one_hot_train_dataset["file_loc"]
new_final_train_dataset = pd.DataFrame(list(zip(image_locations, expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, valence_list, arousal_list)),
              columns=['file_loc','expr_0','expr_1','expr_2','expr_3','expr_4','expr_5','expr_6','expr_7','valence','arousal'])
new_final_train_dataset

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,valence,arousal
0,/content/train_set/images/100006.jpg,5.259446e-03,3.746646e-03,4.029866e-03,1.294283e-02,3.027101e-01,1.437440e-02,5.914377e-01,6.549893e-02,-0.341270,0.833333
1,/content/train_set/images/100008.jpg,2.889653e-03,2.277021e-03,1.069908e-03,9.791485e-01,8.658546e-03,1.464957e-03,2.497280e-03,1.994181e-03,0.150966,0.534852
2,/content/train_set/images/100009.jpg,5.187861e-64,3.760349e-64,3.245977e-64,4.430909e-63,1.000000e+00,6.973739e-64,2.404535e-63,1.547248e-63,-0.126422,0.758533
3,/content/train_set/images/100012.jpg,2.006945e-13,1.047358e-13,1.713688e-13,2.348265e-13,7.365979e-13,8.994362e-12,4.964071e-01,5.035929e-01,-0.493612,0.561363
4,/content/train_set/images/100013.jpg,9.996790e-01,4.267525e-05,5.871361e-05,4.411703e-05,4.199103e-05,4.247370e-05,4.758641e-05,4.345977e-05,-0.067751,-0.043554
...,...,...,...,...,...,...,...,...,...,...,...
180632,/content/train_set/images/99997.jpg,3.280814e-11,1.680507e-11,4.820919e-11,2.829960e-11,4.984342e-11,5.000000e-01,5.000000e-01,1.811939e-09,-0.753266,0.447745
180633,/content/train_set/images/99998.jpg,1.000000e+00,2.760850e-15,2.730519e-15,2.787601e-15,2.492232e-15,2.266742e-15,2.649018e-15,2.392037e-15,0.017253,0.012940
180634,/content/train_set/images/99999.jpg,4.663998e-11,2.220389e-11,6.839889e-11,3.737713e-11,6.499510e-11,5.000000e-01,5.000000e-01,2.472973e-09,-0.695322,0.400337
180635,/content/train_set/images/9999.jpg,9.394399e-03,2.258516e-02,4.050392e-03,9.283089e-01,1.712024e-02,5.043891e-03,7.210272e-03,6.286706e-03,0.406504,0.609756


In [ ]:
new_final_train_dataset.to_pickle("./new_final_train_dataset_l_0.5.pkl")  

In [ ]:
new_final_train_dataset = pd.read_pickle("./new_final_train_dataset_l_0.5.pkl")  

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=img_gen.flow_from_dataframe(dataframe=new_final_train_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=91,class_mode="raw")

Found 180637 validated image filenames.


In [ ]:
test_generator=img_gen.flow_from_dataframe(dataframe=one_hot_test_dataset,x_col="file_loc",y_col=["expr_0","expr_1","expr_2","expr_3","expr_4","expr_5","expr_6","expr_7"],target_size=(96, 96),batch_size=93,shuffle=False,class_mode="raw")

Found 3999 validated image filenames.


In [ ]:
STEP_SIZE_TRAIN_resnet = train_generator.n//train_generator.batch_size
print(STEP_SIZE_TRAIN_resnet)
STEP_SIZE_TEST_resnet = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TEST_resnet)

1985
43


###### 2nd version of happy resnet lr = 10^-4 

In [ ]:
resnet = tf.keras.applications.ResNet50( 
    include_top=False ,
    weights='imagenet' ,
    input_shape=(96, 96, 3) ,
    pooling="avg"
)
for layer in resnet.layers:
    layer.trainable = True

model_resnet = tf.keras.Sequential()
model_resnet.add(resnet)
model_resnet.add(tf.keras.layers.Flatten())
model_resnet.add(tf.keras.layers.Dense(8, activation='softmax',name='output') ) #we want 7 expressions so we put 7 in order to have a valid range of [0, 8)
model_resnet.compile( loss="categorical_crossentropy" , optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)  ,metrics=['accuracy'])

94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
model_path = f"/content/weights" + "{val_accuracy:.4f}.hdf5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model_resnet.fit_generator(
                   train_generator,
                   epochs=10,
                   steps_per_epoch=STEP_SIZE_TRAIN_resnet,
                   validation_data=test_generator,
                   validation_steps=STEP_SIZE_TEST_resnet,
                   max_queue_size=1,
                   shuffle=True,
                   callbacks=[checkpoint],
                   verbose=1
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
1985/1985 [==============================] - ETA: 0s - loss: 1.0848 - accuracy: 0.6622
Epoch 1: val_accuracy improved from -inf to 0.46512, saving model to /content/weights0.4651.hdf5
1985/1985 [==============================] - 485s 234ms/step - loss: 1.0848 - accuracy: 0.6622 - val_loss: 1.4584 - val_accuracy: 0.4651
Epoch 2/10
1985/1985 [==============================] - ETA: 0s - loss: 0.9210 - accuracy: 0.7244
Epoch 2: val_accuracy improved from 0.46512 to 0.48612, saving model to /content/weights0.4861.hdf5
1985/1985 [==============================] - 433s 218ms/step - loss: 0.9210 - accuracy: 0.7244 - val_loss: 1.4431 - val_accuracy: 0.4861
Epoch 3/10
1985/1985 [==============================] - ETA: 0s - loss: 0.8252 - accuracy: 0.7620
Epoch 3: val_accuracy improved from 0.48612 to 0.51213, saving model to /content/weights0.5121.hdf5
1985/1985 [==============================] - 434s 219ms/step - loss: 0.8252 - accuracy: 0.7620 - val_loss: 1.3962 - val_accuracy: 0.512

In [ ]:
model_resnet = tf.keras.models.load_model("/content/weights0.5121.hdf5")

In [ ]:
y_pred = model_resnet.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from statistics import mean
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.512128032008002
f1 score 0.49079014113570424
average accuracy 0.5120375751503006


## Ξανά στατιστικά

### λ = 0.7

In [ ]:
model = tf.keras.models.load_model("/content/weights0.5141.hdf5")

In [ ]:
y_pred = model.predict(test_generator)
y_pred_max = np.argmax(y_pred, axis=1)
y_true = test_dataset["expr"].to_numpy()

In [ ]:
print("accuracy",accuracy_score(y_true, y_pred_max))
print("f1 score",f1_score(y_true, y_pred_max,average='macro'))
print("average accuracy",mean(confusion_matrix(y_true, y_pred_max, normalize='true').diagonal()))

accuracy 0.5141285321330332
f1 score 0.4967340264009073
average accuracy 0.5140355711422846


In [ ]:
preds = pd.DataFrame(y_pred_max)
preds

,0
0,0
1,0
2,4
3,6
4,0
...,...
3994,1
3995,1
3996,6
3997,3


In [ ]:
true = pd.DataFrame(y_true)
true

,0
0,0
1,0
2,4
3,5
4,2
...,...
3994,1
3995,1
3996,6
3997,3


#### happy

In [ ]:
happy_true = true.loc[true[0] == 1]
happy_true

,0
10,1
17,1
40,1
48,1
74,1
...,...
3986,1
3988,1
3991,1
3994,1


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
happy_preds = preds.iloc[preds.index.isin(happy_true.index.tolist())]
happy_preds

,0
10,1
17,0
40,1
48,1
74,1
...,...
3986,1
3988,1
3991,1
3994,1


Πόσα είχαν συναίσθημα happy και μετά προβλεύθηκε κάτι διαφορετικό

In [ ]:
diff = happy_true.values != happy_preds.values
result = diff.flatten().sum()
print(result)

83


In [ ]:
del one_hot_train_dataset['valence']
del one_hot_train_dataset['arousal']
happy_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_1'] == 1]
happy_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
0,0,1,0,0,0,0,0,0,/content/train_set/images/0.jpg
1,0,1,0,0,0,0,0,0,/content/train_set/images/100000.jpg
2,0,1,0,0,0,0,0,0,/content/train_set/images/100001.jpg
4,0,1,0,0,0,0,0,0,/content/train_set/images/100004.jpg
8,0,1,0,0,0,0,0,0,/content/train_set/images/100011.jpg
...,...,...,...,...,...,...,...,...,...
287639,0,1,0,0,0,0,0,0,/content/train_set/images/9998.jpg
287640,0,1,0,0,0,0,0,0,/content/train_set/images/99990.jpg
287644,0,1,0,0,0,0,0,0,/content/train_set/images/99995.jpg
287645,0,1,0,0,0,0,0,0,/content/train_set/images/99996.jpg


In [ ]:
del new_final_train_dataset['valence']
del new_final_train_dataset['arousal']
happy = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(happy_old['file_loc'])]
happy

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
5,/content/train_set/images/100015.jpg,3.197984e-04,0.998276,1.439279e-04,5.080261e-04,2.509087e-04,1.505669e-04,1.823264e-04,1.685105e-04
8,/content/train_set/images/100024.jpg,3.761421e-03,0.987569,1.029420e-03,2.749388e-03,1.534095e-03,1.013366e-03,1.223416e-03,1.119725e-03
14,/content/train_set/images/100033.jpg,7.749931e-03,0.976914,1.966146e-03,4.478708e-03,2.712220e-03,1.879748e-03,2.241016e-03,2.058022e-03
15,/content/train_set/images/100034.jpg,3.706898e-09,1.000000,1.568345e-09,4.288438e-09,2.411644e-09,1.576484e-09,1.871366e-09,1.736976e-09
16,/content/train_set/images/100038.jpg,8.943781e-03,0.973419,2.239894e-03,5.202001e-03,3.120252e-03,2.149475e-03,2.568926e-03,2.356729e-03
...,...,...,...,...,...,...,...,...,...
180614,/content/train_set/images/99962.jpg,1.613115e-73,1.000000,6.242165e-74,1.652274e-73,9.372519e-74,6.202864e-74,7.369161e-74,6.823771e-74
180616,/content/train_set/images/9996.jpg,8.994323e-03,0.972584,2.269442e-03,5.611910e-03,3.261057e-03,2.203696e-03,2.649721e-03,2.425885e-03
180628,/content/train_set/images/99990.jpg,1.478595e-06,0.999994,5.918815e-07,1.320159e-06,8.203152e-07,5.722157e-07,6.686609e-07,6.227592e-07
180631,/content/train_set/images/99995.jpg,3.127598e-02,0.909762,6.975608e-03,1.866103e-02,1.043973e-02,6.877341e-03,8.383485e-03,7.624323e-03


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

Μειώθηκαν οι max values,αυξηθηκαν οι min values

In [ ]:
maxs = [happy['expr_0'].max(),happy['expr_1'].max(),happy['expr_2'].max(),happy['expr_3'].max(),happy['expr_4'].max(),happy['expr_5'].max(),happy['expr_6'].max(),happy['expr_7'].max()]
print("Max values:\n",maxs)
mins = [happy['expr_0'].min(),happy['expr_1'].min(),happy['expr_2'].min(),happy['expr_3'].min(),happy['expr_4'].min(),happy['expr_5'].min(),happy['expr_6'].min(),happy['expr_7'].min()]
print("Min values:\n",mins)
means = [happy['expr_0'].mean(),happy['expr_1'].mean(),happy['expr_2'].mean(),happy['expr_3'].mean(),happy['expr_4'].mean(),happy['expr_5'].mean(),happy['expr_6'].mean(),happy['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.23332964175090531, 1.0, 0.014975808499528906, 0.3, 0.0448779369107215, 0.01693035599164408, 0.024575897412645587, 0.020574197336118285]
Min values:
 [1.573130355978286e-276, 0.7, 6.101846203796695e-277, 1.647652802847909e-276, 9.247707411785468e-277, 6.083344735445131e-277, 7.238760805847267e-277, 6.699973599618338e-277]
Mean values:
 [0.0045248218858437396, 0.9834657368815021, 0.0011675455574912926, 0.004940118776311298, 0.0019527487235060468, 0.001180720662151238, 0.0014475494869719558, 0.0013207580262220944]


Το πριν:

In [ ]:
maxs = [happy['expr_0'].max(),happy['expr_1'].max(),happy['expr_2'].max(),happy['expr_3'].max(),happy['expr_4'].max(),happy['expr_5'].max(),happy['expr_6'].max(),happy['expr_7'].max()]
print("Max values:\n",maxs)
mins = [happy['expr_0'].min(),happy['expr_1'].min(),happy['expr_2'].min(),happy['expr_3'].min(),happy['expr_4'].min(),happy['expr_5'].min(),happy['expr_6'].min(),happy['expr_7'].min()]
print("Min values:\n",mins)
means = [happy['expr_0'].mean(),happy['expr_1'].mean(),happy['expr_2'].mean(),happy['expr_3'].mean(),happy['expr_4'].mean(),happy['expr_5'].mean(),happy['expr_6'].mean(),happy['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2999898644745638, 1.0, 0.29416699884821146, 0.3, 0.29999946605336675, 0.0287108455866289, 0.18835695094687252, 0.09351359148114731]
Min values:
 [0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.0071955289131718225, 0.9745195631472361, 0.0023813945091821836, 0.005734770604767698, 0.0030879751874817625, 0.0021657784465883094, 0.0025291441084934483, 0.0023858450830784653]


Επίσης θέλουμε να δούμε από τις 134415 εικόνες με το συναίσθημα happy, σε πόσες το νέο label είναι διαφορετικό του happy.

In [ ]:
del happy['file_loc']

In [ ]:
happy

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
5,3.197984e-04,0.998276,1.439279e-04,5.080261e-04,2.509087e-04,1.505669e-04,1.823264e-04,1.685105e-04
8,3.761421e-03,0.987569,1.029420e-03,2.749388e-03,1.534095e-03,1.013366e-03,1.223416e-03,1.119725e-03
14,7.749931e-03,0.976914,1.966146e-03,4.478708e-03,2.712220e-03,1.879748e-03,2.241016e-03,2.058022e-03
15,3.706898e-09,1.000000,1.568345e-09,4.288438e-09,2.411644e-09,1.576484e-09,1.871366e-09,1.736976e-09
16,8.943781e-03,0.973419,2.239894e-03,5.202001e-03,3.120252e-03,2.149475e-03,2.568926e-03,2.356729e-03
...,...,...,...,...,...,...,...,...
180614,1.613115e-73,1.000000,6.242165e-74,1.652274e-73,9.372519e-74,6.202864e-74,7.369161e-74,6.823771e-74
180616,8.994323e-03,0.972584,2.269442e-03,5.611910e-03,3.261057e-03,2.203696e-03,2.649721e-03,2.425885e-03
180628,1.478595e-06,0.999994,5.918815e-07,1.320159e-06,8.203152e-07,5.722157e-07,6.686609e-07,6.227592e-07
180631,3.127598e-02,0.909762,6.975608e-03,1.866103e-02,1.043973e-02,6.877341e-03,8.383485e-03,7.624323e-03


In [ ]:
happy["max"]=happy.idxmax(axis=1)
happy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
5,3.197984e-04,0.998276,1.439279e-04,5.080261e-04,2.509087e-04,1.505669e-04,1.823264e-04,1.685105e-04,expr_1
8,3.761421e-03,0.987569,1.029420e-03,2.749388e-03,1.534095e-03,1.013366e-03,1.223416e-03,1.119725e-03,expr_1
14,7.749931e-03,0.976914,1.966146e-03,4.478708e-03,2.712220e-03,1.879748e-03,2.241016e-03,2.058022e-03,expr_1
15,3.706898e-09,1.000000,1.568345e-09,4.288438e-09,2.411644e-09,1.576484e-09,1.871366e-09,1.736976e-09,expr_1
16,8.943781e-03,0.973419,2.239894e-03,5.202001e-03,3.120252e-03,2.149475e-03,2.568926e-03,2.356729e-03,expr_1
...,...,...,...,...,...,...,...,...,...
180614,1.613115e-73,1.000000,6.242165e-74,1.652274e-73,9.372519e-74,6.202864e-74,7.369161e-74,6.823771e-74,expr_1
180616,8.994323e-03,0.972584,2.269442e-03,5.611910e-03,3.261057e-03,2.203696e-03,2.649721e-03,2.425885e-03,expr_1
180628,1.478595e-06,0.999994,5.918815e-07,1.320159e-06,8.203152e-07,5.722157e-07,6.686609e-07,6.227592e-07,expr_1
180631,3.127598e-02,0.909762,6.975608e-03,1.866103e-02,1.043973e-02,6.877341e-03,8.383485e-03,7.624323e-03,expr_1


In [ ]:
print(happy.loc[happy['max'] != "expr_1"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


Άρα όλες οι happy έχουν προβλεφθεί σωστά.

#### neutral

In [ ]:
neutral_true = true.loc[true[0] == 0]
neutral_true

,0
0,0
1,0
14,0
16,0
23,0
...,...
3953,0
3963,0
3982,0
3984,0


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
neutral_preds = preds.iloc[preds.index.isin(neutral_true.index.tolist())]
neutral_preds

,0
0,0
1,0
14,0
16,0
23,0
...,...
3953,2
3963,0
3982,0
3984,0


In [ ]:
diff = neutral_true.values != neutral_preds.values
result = diff.flatten().sum()
print(result)

165


In [ ]:
neutral_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_0'] == 1]
neutral_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
3,1,0,0,0,0,0,0,0,/content/train_set/images/100002.jpg
10,1,0,0,0,0,0,0,0,/content/train_set/images/100013.jpg
13,1,0,0,0,0,0,0,0,/content/train_set/images/100018.jpg
19,1,0,0,0,0,0,0,0,/content/train_set/images/100025.jpg
23,1,0,0,0,0,0,0,0,/content/train_set/images/100031.jpg
...,...,...,...,...,...,...,...,...,...
287636,1,0,0,0,0,0,0,0,/content/train_set/images/99982.jpg
287637,1,0,0,0,0,0,0,0,/content/train_set/images/99984.jpg
287642,1,0,0,0,0,0,0,0,/content/train_set/images/99993.jpg
287643,1,0,0,0,0,0,0,0,/content/train_set/images/99994.jpg


In [ ]:
neutral = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(neutral_old['file_loc'])]
neutral

,file_loc,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
4,/content/train_set/images/100013.jpg,0.999807,2.560515e-05,3.522816e-05,2.647022e-05,2.519462e-05,2.548422e-05,2.855184e-05,2.607586e-05
9,/content/train_set/images/100025.jpg,0.994152,7.448933e-04,1.194533e-03,7.635401e-04,7.363556e-04,7.751621e-04,8.519627e-04,7.816690e-04
12,/content/train_set/images/100031.jpg,0.999782,3.015335e-05,4.050224e-05,2.994796e-05,2.821878e-05,2.840382e-05,3.159398e-05,2.898892e-05
24,/content/train_set/images/100048.jpg,1.000000,4.169641e-15,4.168373e-15,4.037892e-15,3.614084e-15,3.333979e-15,3.847971e-15,3.493718e-15
30,/content/train_set/images/10005.jpg,0.999917,1.408616e-05,1.340641e-05,1.241223e-05,1.099701e-05,1.020976e-05,1.160086e-05,1.061909e-05
...,...,...,...,...,...,...,...,...,...
180624,/content/train_set/images/99981.jpg,0.999986,1.961344e-06,1.980427e-06,2.144618e-06,1.935563e-06,1.736236e-06,2.069193e-06,1.850960e-06
180625,/content/train_set/images/99982.jpg,0.998285,2.336858e-04,3.355468e-04,2.304365e-04,2.183255e-04,2.235345e-04,2.464227e-04,2.266578e-04
180626,/content/train_set/images/99984.jpg,0.998978,1.334107e-04,1.944022e-04,1.377466e-04,1.318839e-04,1.355253e-04,1.508168e-04,1.379310e-04
180630,/content/train_set/images/99994.jpg,1.000000,7.429987e-74,7.872514e-74,7.538777e-74,6.845722e-74,6.373948e-74,7.383166e-74,6.683463e-74


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [neutral['expr_0'].max(),neutral['expr_1'].max(),neutral['expr_2'].max(),neutral['expr_3'].max(),neutral['expr_4'].max(),neutral['expr_5'].max(),neutral['expr_6'].max(),neutral['expr_7'].max()]
print("Max values:\n",maxs)
mins = [neutral['expr_0'].min(),neutral['expr_1'].min(),neutral['expr_2'].min(),neutral['expr_3'].min(),neutral['expr_4'].min(),neutral['expr_5'].min(),neutral['expr_6'].min(),neutral['expr_7'].min()]
print("Min values:\n",mins)
means = [neutral['expr_0'].mean(),neutral['expr_1'].mean(),neutral['expr_2'].mean(),neutral['expr_3'].mean(),neutral['expr_4'].mean(),neutral['expr_5'].mean(),neutral['expr_6'].mean(),neutral['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [1.0, 0.002716781552163899, 0.0033956572678706097, 0.0033865156589639486, 0.003284519547614194, 0.003112420797203643, 0.0038508601625592415, 0.0033465617813156863]
Min values:
 [0.9769066832323088, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.9997484339864268, 3.416302553465581e-05, 4.078692048023812e-05, 3.6935244546669107e-05, 3.436745686311427e-05, 3.26758358329147e-05, 3.831913249104605e-05, 3.431839782472151e-05]


To πριν

In [ ]:
maxs = [neutral['expr_0'].max(),neutral['expr_1'].max(),neutral['expr_2'].max(),neutral['expr_3'].max(),neutral['expr_4'].max(),neutral['expr_5'].max(),neutral['expr_6'].max(),neutral['expr_7'].max()]
print("Max values:\n",maxs)
mins = [neutral['expr_0'].min(),neutral['expr_1'].min(),neutral['expr_2'].min(),neutral['expr_3'].min(),neutral['expr_4'].min(),neutral['expr_5'].min(),neutral['expr_6'].min(),neutral['expr_7'].min()]
print("Min values:\n",mins)
means = [neutral['expr_0'].mean(),neutral['expr_1'].mean(),neutral['expr_2'].mean(),neutral['expr_3'].mean(),neutral['expr_4'].mean(),neutral['expr_5'].mean(),neutral['expr_6'].mean(),neutral['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [1.0, 0.3, 0.28755267101668025, 0.20856772083255742, 0.2999997079085731, 0.10176474967548348, 0.14552838325288162, 0.07253953010495366]
Min values:
 [0.7, 6.131021835505047e-117, 7.370727608149487e-117, 5.963043734183301e-117, 5.650789524431911e-117, 5.6175705587246315e-117, 6.357500193414278e-117, 5.9108234399430465e-117]
Mean values:
 [0.9257567744877918, 0.010163304137175594, 0.01733893333625754, 0.0060607577464307045, 0.006331877872597657, 0.010818262552424615, 0.012840580954692293, 0.010689508912629767]


Επίσης θέλουμε να δούμε από τις 74874 εικόνες με το συναίσθημα neutral, σε πόσες το νέο label είναι διαφορετικό του neutral.

In [ ]:
del neutral['file_loc']

In [ ]:
neutral

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
4,0.999807,2.560515e-05,3.522816e-05,2.647022e-05,2.519462e-05,2.548422e-05,2.855184e-05,2.607586e-05
9,0.994152,7.448933e-04,1.194533e-03,7.635401e-04,7.363556e-04,7.751621e-04,8.519627e-04,7.816690e-04
12,0.999782,3.015335e-05,4.050224e-05,2.994796e-05,2.821878e-05,2.840382e-05,3.159398e-05,2.898892e-05
24,1.000000,4.169641e-15,4.168373e-15,4.037892e-15,3.614084e-15,3.333979e-15,3.847971e-15,3.493718e-15
30,0.999917,1.408616e-05,1.340641e-05,1.241223e-05,1.099701e-05,1.020976e-05,1.160086e-05,1.061909e-05
...,...,...,...,...,...,...,...,...
180624,0.999986,1.961344e-06,1.980427e-06,2.144618e-06,1.935563e-06,1.736236e-06,2.069193e-06,1.850960e-06
180625,0.998285,2.336858e-04,3.355468e-04,2.304365e-04,2.183255e-04,2.235345e-04,2.464227e-04,2.266578e-04
180626,0.998978,1.334107e-04,1.944022e-04,1.377466e-04,1.318839e-04,1.355253e-04,1.508168e-04,1.379310e-04
180630,1.000000,7.429987e-74,7.872514e-74,7.538777e-74,6.845722e-74,6.373948e-74,7.383166e-74,6.683463e-74


In [ ]:
neutral["max"]=neutral.idxmax(axis=1)
neutral

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
4,0.999807,2.560515e-05,3.522816e-05,2.647022e-05,2.519462e-05,2.548422e-05,2.855184e-05,2.607586e-05,expr_0
9,0.994152,7.448933e-04,1.194533e-03,7.635401e-04,7.363556e-04,7.751621e-04,8.519627e-04,7.816690e-04,expr_0
12,0.999782,3.015335e-05,4.050224e-05,2.994796e-05,2.821878e-05,2.840382e-05,3.159398e-05,2.898892e-05,expr_0
24,1.000000,4.169641e-15,4.168373e-15,4.037892e-15,3.614084e-15,3.333979e-15,3.847971e-15,3.493718e-15,expr_0
30,0.999917,1.408616e-05,1.340641e-05,1.241223e-05,1.099701e-05,1.020976e-05,1.160086e-05,1.061909e-05,expr_0
...,...,...,...,...,...,...,...,...,...
180624,0.999986,1.961344e-06,1.980427e-06,2.144618e-06,1.935563e-06,1.736236e-06,2.069193e-06,1.850960e-06,expr_0
180625,0.998285,2.336858e-04,3.355468e-04,2.304365e-04,2.183255e-04,2.235345e-04,2.464227e-04,2.266578e-04,expr_0
180626,0.998978,1.334107e-04,1.944022e-04,1.377466e-04,1.318839e-04,1.355253e-04,1.508168e-04,1.379310e-04,expr_0
180630,1.000000,7.429987e-74,7.872514e-74,7.538777e-74,6.845722e-74,6.373948e-74,7.383166e-74,6.683463e-74,expr_0


In [ ]:
print(neutral.loc[neutral['max'] != "expr_0"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


Άρα όλες οι neutral έχουν προβλεφθεί σωστά.

#### sad

In [ ]:
sad_true = true.loc[true[0] == 2]
sad_true

,0
4,2
8,2
13,2
20,2
21,2
...,...
3939,2
3958,2
3973,2
3976,2


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
sad_preds = preds.iloc[preds.index.isin(sad_true.index.tolist())]
sad_preds

,0
4,0
8,0
13,6
20,4
21,3
...,...
3939,2
3958,2
3973,2
3976,2


Πόσα είχαν συναίσθημα happy και μετά προβλεύθηκε κάτι διαφορετικό

In [ ]:
diff = sad_true.values != sad_preds.values
result = diff.flatten().sum()
print(result)

208


In [ ]:
sad_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_2'] == 1]
sad_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
17,0,0,1,0,0,0,0,0,/content/train_set/images/100023.jpg
29,0,0,1,0,0,0,0,0,/content/train_set/images/100039.jpg
36,0,0,1,0,0,0,0,0,/content/train_set/images/100045.jpg
43,0,0,1,0,0,0,0,0,/content/train_set/images/100052.jpg
47,0,0,1,0,0,0,0,0,/content/train_set/images/100058.jpg
...,...,...,...,...,...,...,...,...,...
287580,0,0,1,0,0,0,0,0,/content/train_set/images/99899.jpg
287629,0,0,1,0,0,0,0,0,/content/train_set/images/99973.jpg
287631,0,0,1,0,0,0,0,0,/content/train_set/images/99975.jpg
287632,0,0,1,0,0,0,0,0,/content/train_set/images/99977.jpg


In [ ]:
sad = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(sad_old['file_loc'])]
del sad['file_loc']
sad

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
7,5.211701e-03,2.502095e-03,0.971599,2.764372e-03,3.055121e-03,5.996231e-03,4.400242e-03,4.471627e-03
17,4.720350e-04,1.324620e-04,0.998402,1.442180e-04,1.547405e-04,2.606190e-04,2.209625e-04,2.133124e-04
22,7.868293e-03,3.657761e-03,0.940088,4.473797e-03,5.339014e-03,1.870540e-02,9.617316e-03,1.025013e-02
26,1.323912e-02,2.851492e-03,0.960225,3.255347e-03,3.552085e-03,6.311215e-03,5.430285e-03,5.135535e-03
29,3.086420e-04,8.263710e-05,0.998917,9.333612e-05,1.022151e-04,1.908032e-04,1.556799e-04,1.498770e-04
...,...,...,...,...,...,...,...,...
180591,7.480124e-05,3.158192e-05,0.999623,3.522406e-05,3.902335e-05,7.946646e-05,5.806664e-05,5.841758e-05
180619,2.297281e-06,9.192713e-07,0.999988,1.058270e-06,1.196392e-06,2.801407e-06,1.905787e-06,1.924590e-06
180620,1.588746e-03,7.257274e-04,0.991271,8.159292e-04,9.114541e-04,1.933676e-03,1.364483e-03,1.388626e-03
180621,4.433591e-08,1.371049e-08,1.000000,1.572116e-08,1.750772e-08,3.662841e-08,2.770369e-08,2.707116e-08


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [sad['expr_0'].max(),sad['expr_1'].max(),sad['expr_2'].max(),sad['expr_3'].max(),sad['expr_4'].max(),sad['expr_5'].max(),sad['expr_6'].max(),sad['expr_7'].max()]
print("Max values:\n",maxs)
mins = [sad['expr_0'].min(),sad['expr_1'].min(),sad['expr_2'].min(),sad['expr_3'].min(),sad['expr_4'].min(),sad['expr_5'].min(),sad['expr_6'].min(),sad['expr_7'].min()]
print("Min values:\n",mins)
means = [sad['expr_0'].mean(),sad['expr_1'].mean(),sad['expr_2'].mean(),sad['expr_3'].mean(),sad['expr_4'].mean(),sad['expr_5'].mean(),sad['expr_6'].mean(),sad['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29943607025299107, 0.031664078003283656, 1.0, 0.042811809707761724, 0.3, 0.3, 0.3, 0.3]
Min values:
 [0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.01758540499695774, 0.0031887334665366015, 0.953062007985231, 0.003224939908888843, 0.003484776201108738, 0.008171741651113534, 0.0050657620113128245, 0.006216633778850598]


Πριν

In [ ]:
maxs = [sad['expr_0'].max(),sad['expr_1'].max(),sad['expr_2'].max(),sad['expr_3'].max(),sad['expr_4'].max(),sad['expr_5'].max(),sad['expr_6'].max(),sad['expr_7'].max()]
print("Max values:\n",maxs)
mins = [sad['expr_0'].min(),sad['expr_1'].min(),sad['expr_2'].min(),sad['expr_3'].min(),sad['expr_4'].min(),sad['expr_5'].min(),sad['expr_6'].min(),sad['expr_7'].min()]
print("Min values:\n",mins)
means = [sad['expr_0'].mean(),sad['expr_1'].mean(),sad['expr_2'].mean(),sad['expr_3'].mean(),sad['expr_4'].mean(),sad['expr_5'].mean(),sad['expr_6'].mean(),sad['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29999999999976446, 0.26185167301597817, 1.0, 0.06141478050685742, 0.3, 0.3, 0.3, 0.3]
Min values:
 [4.6225260194e-314, 1.453685626e-314, 0.7, 1.5648216017e-314, 1.730332314e-314, 3.4924720325e-314, 2.626987926e-314, 2.605185075e-314]
Mean values:
 [0.022254812812134022, 0.0032901270063358406, 0.9490361376955995, 0.003075601551646815, 0.00334297400801039, 0.008037030363325558, 0.004791744623244813, 0.006171571939703062]


In [ ]:
sad["max"]=sad.idxmax(axis=1)
sad

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
7,5.211701e-03,2.502095e-03,0.971599,2.764372e-03,3.055121e-03,5.996231e-03,4.400242e-03,4.471627e-03,expr_2
17,4.720350e-04,1.324620e-04,0.998402,1.442180e-04,1.547405e-04,2.606190e-04,2.209625e-04,2.133124e-04,expr_2
22,7.868293e-03,3.657761e-03,0.940088,4.473797e-03,5.339014e-03,1.870540e-02,9.617316e-03,1.025013e-02,expr_2
26,1.323912e-02,2.851492e-03,0.960225,3.255347e-03,3.552085e-03,6.311215e-03,5.430285e-03,5.135535e-03,expr_2
29,3.086420e-04,8.263710e-05,0.998917,9.333612e-05,1.022151e-04,1.908032e-04,1.556799e-04,1.498770e-04,expr_2
...,...,...,...,...,...,...,...,...,...
180591,7.480124e-05,3.158192e-05,0.999623,3.522406e-05,3.902335e-05,7.946646e-05,5.806664e-05,5.841758e-05,expr_2
180619,2.297281e-06,9.192713e-07,0.999988,1.058270e-06,1.196392e-06,2.801407e-06,1.905787e-06,1.924590e-06,expr_2
180620,1.588746e-03,7.257274e-04,0.991271,8.159292e-04,9.114541e-04,1.933676e-03,1.364483e-03,1.388626e-03,expr_2
180621,4.433591e-08,1.371049e-08,1.000000,1.572116e-08,1.750772e-08,3.662841e-08,2.770369e-08,2.707116e-08,expr_2


In [ ]:
print(sad.loc[sad['max'] != "expr_2"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### surprise

In [ ]:
surprise_true = true.loc[true[0] == 3]
surprise_true

,0
6,3
7,3
22,3
25,3
38,3
...,...
3959,3
3962,3
3967,3
3989,3


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
surprise_preds = preds.iloc[preds.index.isin(surprise_true.index.tolist())]
surprise_preds

,0
6,0
7,6
22,0
25,3
38,2
...,...
3959,3
3962,3
3967,3
3989,1


In [ ]:
diff = surprise_true.values != surprise_preds.values
result = diff.flatten().sum()
print(result)

269


In [ ]:
surprise_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_3'] == 1]
surprise_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
6,0,0,0,1,0,0,0,0,/content/train_set/images/100008.jpg
31,0,0,0,1,0,0,0,0,/content/train_set/images/100040.jpg
33,0,0,0,1,0,0,0,0,/content/train_set/images/100042.jpg
62,0,0,0,1,0,0,0,0,/content/train_set/images/100077.jpg
86,0,0,0,1,0,0,0,0,/content/train_set/images/100113.jpg
...,...,...,...,...,...,...,...,...,...
287523,0,0,0,1,0,0,0,0,/content/train_set/images/99817.jpg
287533,0,0,0,1,0,0,0,0,/content/train_set/images/9982.jpg
287573,0,0,0,1,0,0,0,0,/content/train_set/images/9988.jpg
287604,0,0,0,1,0,0,0,0,/content/train_set/images/99925.jpg


In [ ]:
surprise = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(surprise_old['file_loc'])]
del surprise['file_loc']
surprise

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
1,1.733792e-03,1.366213e-03,6.419450e-04,0.987489,5.195128e-03,8.789741e-04,1.498368e-03,1.196509e-03
18,3.325956e-06,4.498772e-06,1.559419e-06,0.999968,1.391313e-05,2.136414e-06,3.382035e-06,2.853307e-06
20,7.571035e-05,6.343459e-05,4.825509e-05,0.701382,2.979492e-01,9.299830e-05,2.193245e-04,1.694206e-04
38,3.415644e-13,3.726902e-13,1.548678e-13,1.000000,1.935418e-12,2.185145e-13,3.669428e-13,3.010199e-13
53,1.964154e-04,3.117123e-04,9.157169e-05,0.998309,6.249966e-04,1.219409e-04,1.858072e-04,1.590344e-04
...,...,...,...,...,...,...,...,...
180555,5.772849e-12,5.423016e-12,2.615443e-12,1.000000,4.812552e-11,3.815244e-12,6.795565e-12,5.429974e-12
180562,7.561538e-04,6.031082e-04,2.905983e-04,0.993934,2.753553e-03,4.041881e-04,7.011818e-04,5.570034e-04
180587,9.575799e-04,1.718052e-03,4.268151e-04,0.992452,2.363355e-03,5.525001e-04,8.218259e-04,7.073732e-04
180603,5.832487e-04,4.646306e-04,4.181169e-04,0.702744,2.900215e-01,9.654004e-04,2.683752e-03,2.119319e-03


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [surprise['expr_0'].max(),surprise['expr_1'].max(),surprise['expr_2'].max(),surprise['expr_3'].max(),surprise['expr_4'].max(),surprise['expr_5'].max(),surprise['expr_6'].max(),surprise['expr_7'].max()]
print("Max values:\n",maxs)
mins = [surprise['expr_0'].min(),surprise['expr_1'].min(),surprise['expr_2'].min(),surprise['expr_3'].min(),surprise['expr_4'].min(),surprise['expr_5'].min(),surprise['expr_6'].min(),surprise['expr_7'].min()]
print("Min values:\n",mins)
means = [surprise['expr_0'].mean(),surprise['expr_1'].mean(),surprise['expr_2'].mean(),surprise['expr_3'].mean(),surprise['expr_4'].mean(),surprise['expr_5'].mean(),surprise['expr_6'].mean(),surprise['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29452768119352907, 0.2999999996458214, 0.29999857554465537, 1.0, 0.3, 0.2999999999998145, 0.2999955974359371, 0.2999956801223135]
Min values:
 [0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0]
Mean values:
 [0.0067316380379698025, 0.006471813035710477, 0.00233093450073137, 0.9175908481257357, 0.054034474813906226, 0.0032092005102293945, 0.005486518116848794, 0.004144572858868283]


In [ ]:
surprise["max"]=surprise.idxmax(axis=1)
surprise

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
1,1.733792e-03,1.366213e-03,6.419450e-04,0.987489,5.195128e-03,8.789741e-04,1.498368e-03,1.196509e-03,expr_3
18,3.325956e-06,4.498772e-06,1.559419e-06,0.999968,1.391313e-05,2.136414e-06,3.382035e-06,2.853307e-06,expr_3
20,7.571035e-05,6.343459e-05,4.825509e-05,0.701382,2.979492e-01,9.299830e-05,2.193245e-04,1.694206e-04,expr_3
38,3.415644e-13,3.726902e-13,1.548678e-13,1.000000,1.935418e-12,2.185145e-13,3.669428e-13,3.010199e-13,expr_3
53,1.964154e-04,3.117123e-04,9.157169e-05,0.998309,6.249966e-04,1.219409e-04,1.858072e-04,1.590344e-04,expr_3
...,...,...,...,...,...,...,...,...,...
180555,5.772849e-12,5.423016e-12,2.615443e-12,1.000000,4.812552e-11,3.815244e-12,6.795565e-12,5.429974e-12,expr_3
180562,7.561538e-04,6.031082e-04,2.905983e-04,0.993934,2.753553e-03,4.041881e-04,7.011818e-04,5.570034e-04,expr_3
180587,9.575799e-04,1.718052e-03,4.268151e-04,0.992452,2.363355e-03,5.525001e-04,8.218259e-04,7.073732e-04,expr_3
180603,5.832487e-04,4.646306e-04,4.181169e-04,0.702744,2.900215e-01,9.654004e-04,2.683752e-03,2.119319e-03,expr_3


In [ ]:
print(surprise.loc[surprise['max'] != "expr_3"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### fear

In [ ]:
fear_true = true.loc[true[0] == 4]
fear_true

,0
2,4
9,4
19,4
28,4
31,4
...,...
3970,4
3972,4
3977,4
3990,4


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
fear_preds = preds.iloc[preds.index.isin(fear_true.index.tolist())]
fear_preds

,0
2,4
9,4
19,4
28,4
31,0
...,...
3970,4
3972,4
3977,4
3990,4


In [ ]:
diff = fear_true.values != fear_preds.values
result = diff.flatten().sum()
print(result)

262


In [ ]:
fear_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_4'] == 1]
fear_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
7,0,0,0,0,1,0,0,0,/content/train_set/images/100009.jpg
85,0,0,0,0,1,0,0,0,/content/train_set/images/100112.jpg
178,0,0,0,0,1,0,0,0,/content/train_set/images/10022.jpg
251,0,0,0,0,1,0,0,0,/content/train_set/images/100315.jpg
333,0,0,0,0,1,0,0,0,/content/train_set/images/100424.jpg
...,...,...,...,...,...,...,...,...,...
287492,0,0,0,0,1,0,0,0,/content/train_set/images/99783.jpg
287544,0,0,0,0,1,0,0,0,/content/train_set/images/99852.jpg
287574,0,0,0,0,1,0,0,0,/content/train_set/images/99891.jpg
287627,0,0,0,0,1,0,0,0,/content/train_set/images/99970.jpg


In [ ]:
fear = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(fear_old['file_loc'])]
del fear['file_loc']
fear

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
2,3.112717e-64,2.256209e-64,1.947586e-64,2.658545e-63,1.000000,4.184243e-64,1.442721e-63,9.283491e-64
52,1.637198e-32,1.189092e-32,1.012382e-32,1.579251e-31,1.000000,2.136379e-32,7.121395e-32,4.622851e-32
103,3.297125e-08,2.294023e-08,2.151134e-08,1.555003e-07,0.999999,5.149244e-08,2.424254e-07,1.374454e-07
155,3.821931e-04,2.457310e-04,2.088473e-04,3.291632e-03,0.993285,4.087334e-04,1.359041e-03,8.188975e-04
209,4.682397e-07,3.369195e-07,2.689538e-07,8.825387e-06,0.999987,5.216479e-07,1.526323e-06,1.015296e-06
...,...,...,...,...,...,...,...,...
180539,1.737132e-05,1.377169e-05,1.179513e-05,1.224233e-04,0.999683,2.565228e-05,7.201960e-05,5.405611e-05
180570,1.194132e-03,1.022382e-03,8.193525e-04,9.785485e-03,0.978842,1.645746e-03,3.678324e-03,3.012521e-03
180588,6.771575e-04,4.703171e-04,3.525354e-04,2.437639e-02,0.970752,6.286806e-04,1.631463e-03,1.111252e-03
180617,9.304246e-16,6.994427e-16,5.748917e-16,1.195535e-14,1.000000,1.176364e-15,3.523913e-15,2.410978e-15


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [fear['expr_0'].max(),fear['expr_1'].max(),fear['expr_2'].max(),fear['expr_3'].max(),fear['expr_4'].max(),fear['expr_5'].max(),fear['expr_6'].max(),fear['expr_7'].max()]
print("Max values:\n",maxs)
mins = [fear['expr_0'].min(),fear['expr_1'].min(),fear['expr_2'].min(),fear['expr_3'].min(),fear['expr_4'].min(),fear['expr_5'].min(),fear['expr_6'].min(),fear['expr_7'].min()]
print("Min values:\n",mins)
means = [fear['expr_0'].mean(),fear['expr_1'].mean(),fear['expr_2'].mean(),fear['expr_3'].mean(),fear['expr_4'].mean(),fear['expr_5'].mean(),fear['expr_6'].mean(),fear['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.1662843963089614, 0.029630323356647455, 0.29998331352724034, 0.13895195703458366, 1.0, 0.3, 0.2996486135311684, 0.29999999471154126]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0]
Mean values:
 [0.0022665546220430686, 0.000915045004710352, 0.0013065581152835628, 0.005883378267836475, 0.979617551322658, 0.002195979438401041, 0.004669030042038384, 0.003145903187029077]


To prin

In [ ]:
maxs = [fear['expr_0'].max(),fear['expr_1'].max(),fear['expr_2'].max(),fear['expr_3'].max(),fear['expr_4'].max(),fear['expr_5'].max(),fear['expr_6'].max(),fear['expr_7'].max()]
print("Max values:\n",maxs)
mins = [fear['expr_0'].min(),fear['expr_1'].min(),fear['expr_2'].min(),fear['expr_3'].min(),fear['expr_4'].min(),fear['expr_5'].min(),fear['expr_6'].min(),fear['expr_7'].min()]
print("Min values:\n",mins)
means = [fear['expr_0'].mean(),fear['expr_1'].mean(),fear['expr_2'].mean(),fear['expr_3'].mean(),fear['expr_4'].mean(),fear['expr_5'].mean(),fear['expr_6'].mean(),fear['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.18572341723268185, 0.032828241495403886, 0.2999698637391902, 0.2999973326830519, 1.0, 0.3, 0.2990882268879533, 0.3]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0, 0.0]
Mean values:
 [0.002272215085340342, 0.0008750417455321186, 0.001269053664212939, 0.006392926069128361, 0.9790323537847652, 0.002223704639730988, 0.004595448689693627, 0.0033392563215964687]


In [ ]:
fear["max"]=fear.idxmax(axis=1)
fear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
2,3.112717e-64,2.256209e-64,1.947586e-64,2.658545e-63,1.000000,4.184243e-64,1.442721e-63,9.283491e-64,expr_4
52,1.637198e-32,1.189092e-32,1.012382e-32,1.579251e-31,1.000000,2.136379e-32,7.121395e-32,4.622851e-32,expr_4
103,3.297125e-08,2.294023e-08,2.151134e-08,1.555003e-07,0.999999,5.149244e-08,2.424254e-07,1.374454e-07,expr_4
155,3.821931e-04,2.457310e-04,2.088473e-04,3.291632e-03,0.993285,4.087334e-04,1.359041e-03,8.188975e-04,expr_4
209,4.682397e-07,3.369195e-07,2.689538e-07,8.825387e-06,0.999987,5.216479e-07,1.526323e-06,1.015296e-06,expr_4
...,...,...,...,...,...,...,...,...,...
180539,1.737132e-05,1.377169e-05,1.179513e-05,1.224233e-04,0.999683,2.565228e-05,7.201960e-05,5.405611e-05,expr_4
180570,1.194132e-03,1.022382e-03,8.193525e-04,9.785485e-03,0.978842,1.645746e-03,3.678324e-03,3.012521e-03,expr_4
180588,6.771575e-04,4.703171e-04,3.525354e-04,2.437639e-02,0.970752,6.286806e-04,1.631463e-03,1.111252e-03,expr_4
180617,9.304246e-16,6.994427e-16,5.748917e-16,1.195535e-14,1.000000,1.176364e-15,3.523913e-15,2.410978e-15,expr_4


In [ ]:
print(fear.loc[fear['max'] != "expr_4"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### disgust

In [ ]:
disgust_true = true.loc[true[0] == 5]
disgust_true

,0
3,5
15,5
57,5
64,5
107,5
...,...
3960,5
3964,5
3979,5
3980,5


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
disgust_preds = preds.iloc[preds.index.isin(disgust_true.index.tolist())]
disgust_preds

,0
3,6
15,3
57,3
64,6
107,6
...,...
3960,2
3964,5
3979,6
3980,1


In [ ]:
diff = disgust_true.values != disgust_preds.values
result = diff.flatten().sum()
print(result)

324


In [ ]:
disgust_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_5'] == 1]
disgust_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
35,0,0,0,0,0,1,0,0,/content/train_set/images/100044.jpg
91,0,0,0,0,0,1,0,0,/content/train_set/images/100119.jpg
99,0,0,0,0,0,1,0,0,/content/train_set/images/100128.jpg
145,0,0,0,0,0,1,0,0,/content/train_set/images/100186.jpg
174,0,0,0,0,0,1,0,0,/content/train_set/images/100223.jpg
...,...,...,...,...,...,...,...,...,...
287201,0,0,0,0,0,1,0,0,/content/train_set/images/99421.jpg
287359,0,0,0,0,0,1,0,0,/content/train_set/images/99612.jpg
287480,0,0,0,0,0,1,0,0,/content/train_set/images/99770.jpg
287493,0,0,0,0,0,1,0,0,/content/train_set/images/99784.jpg


In [ ]:
disgust = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(disgust_old['file_loc'])]
del disgust['file_loc']
disgust

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
21,3.473079e-09,1.673745e-09,5.476485e-09,2.761931e-09,4.671164e-09,1.000000,5.097266e-08,1.182001e-07
57,3.534696e-05,1.914684e-05,5.564146e-05,3.124549e-05,5.264298e-05,0.998648,3.433600e-04,8.143947e-04
63,4.152950e-06,2.036144e-06,4.205287e-06,3.975550e-06,8.973088e-06,0.717420,7.864256e-03,2.746921e-01
84,3.434784e-10,1.784241e-10,4.995233e-10,3.012642e-10,5.338375e-10,1.000000,6.036200e-09,1.962849e-08
101,3.264289e-10,1.737804e-10,4.333392e-10,3.033616e-10,5.701421e-10,1.000000,8.888252e-09,4.462169e-08
...,...,...,...,...,...,...,...,...
180354,1.070889e-13,5.567732e-14,1.477060e-13,9.583345e-14,1.752921e-13,1.000000,2.566921e-12,1.054937e-11
180448,6.461735e-05,3.442862e-05,1.088380e-04,5.516240e-05,9.029693e-05,0.997989,5.329074e-04,1.124479e-03
180530,3.423275e-09,1.774350e-09,5.103157e-09,2.971011e-09,5.193050e-09,1.000000,5.341512e-08,1.591046e-07
180540,7.058189e-21,3.469610e-21,9.986767e-21,5.916588e-21,1.056174e-20,1.000000,1.732348e-19,5.636933e-19


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [disgust['expr_0'].max(),disgust['expr_1'].max(),disgust['expr_2'].max(),disgust['expr_3'].max(),disgust['expr_4'].max(),disgust['expr_5'].max(),disgust['expr_6'].max(),disgust['expr_7'].max()]
print("Max values:\n",maxs)
mins = [disgust['expr_0'].min(),disgust['expr_1'].min(),disgust['expr_2'].min(),disgust['expr_3'].min(),disgust['expr_4'].min(),disgust['expr_5'].min(),disgust['expr_6'].min(),disgust['expr_7'].min()]
print("Min values:\n",mins)
means = [disgust['expr_0'].mean(),disgust['expr_1'].mean(),disgust['expr_2'].mean(),disgust['expr_3'].mean(),disgust['expr_4'].mean(),disgust['expr_5'].mean(),disgust['expr_6'].mean(),disgust['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2639419012645073, 0.01973139119711704, 0.2999999999913445, 0.05635170312881933, 0.2999999999999957, 1.0, 0.2999999999996909, 0.3]
Min values:
 [1.61160120387583e-204, 8.05714546926749e-205, 2.128069215642195e-204, 1.4084286822987947e-204, 2.62624472052532e-204, 0.7, 6.099918128031507e-203, 2.879797283425561e-202]
Mean values:
 [0.002074351240297199, 0.0006222383792053445, 0.002197474714160303, 0.0013215781712530874, 0.003927294235985446, 0.9461620363293741, 0.015900429894150234, 0.027794597035574253]


Πριν:

In [ ]:
maxs = [disgust['expr_0'].max(),disgust['expr_1'].max(),disgust['expr_2'].max(),disgust['expr_3'].max(),disgust['expr_4'].max(),disgust['expr_5'].max(),disgust['expr_6'].max(),disgust['expr_7'].max()]
print("Max values:\n",maxs)
mins = [disgust['expr_0'].min(),disgust['expr_1'].min(),disgust['expr_2'].min(),disgust['expr_3'].min(),disgust['expr_4'].min(),disgust['expr_5'].min(),disgust['expr_6'].min(),disgust['expr_7'].min()]
print("Min values:\n",mins)
means = [disgust['expr_0'].mean(),disgust['expr_1'].mean(),disgust['expr_2'].mean(),disgust['expr_3'].mean(),disgust['expr_4'].mean(),disgust['expr_5'].mean(),disgust['expr_6'].mean(),disgust['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2970180252447872, 0.019801942148057233, 0.2999999999674287, 0.299948439224495, 0.2999999999999936, 1.0, 0.2999999999982009, 0.3]
Min values:
 [1.1007309023860154e-303, 5.007068258195963e-304, 1.3260524802850418e-303, 8.770349246564958e-304, 1.6324206577024738e-303, 0.7, 3.8066010799624533e-302, 1.1536832656220979e-301]
Mean values:
 [0.002378199875204804, 0.0006187037329595057, 0.0021700195938340445, 0.0018383839504132096, 0.003916821475999894, 0.9479860032665796, 0.015516574828141241, 0.025575293276867774]


In [ ]:
disgust["max"]=disgust.idxmax(axis=1)
disgust

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
21,3.473079e-09,1.673745e-09,5.476485e-09,2.761931e-09,4.671164e-09,1.000000,5.097266e-08,1.182001e-07,expr_5
57,3.534696e-05,1.914684e-05,5.564146e-05,3.124549e-05,5.264298e-05,0.998648,3.433600e-04,8.143947e-04,expr_5
63,4.152950e-06,2.036144e-06,4.205287e-06,3.975550e-06,8.973088e-06,0.717420,7.864256e-03,2.746921e-01,expr_5
84,3.434784e-10,1.784241e-10,4.995233e-10,3.012642e-10,5.338375e-10,1.000000,6.036200e-09,1.962849e-08,expr_5
101,3.264289e-10,1.737804e-10,4.333392e-10,3.033616e-10,5.701421e-10,1.000000,8.888252e-09,4.462169e-08,expr_5
...,...,...,...,...,...,...,...,...,...
180354,1.070889e-13,5.567732e-14,1.477060e-13,9.583345e-14,1.752921e-13,1.000000,2.566921e-12,1.054937e-11,expr_5
180448,6.461735e-05,3.442862e-05,1.088380e-04,5.516240e-05,9.029693e-05,0.997989,5.329074e-04,1.124479e-03,expr_5
180530,3.423275e-09,1.774350e-09,5.103157e-09,2.971011e-09,5.193050e-09,1.000000,5.341512e-08,1.591046e-07,expr_5
180540,7.058189e-21,3.469610e-21,9.986767e-21,5.916588e-21,1.056174e-20,1.000000,1.732348e-19,5.636933e-19,expr_5


In [ ]:
print(disgust.loc[disgust['max'] != "expr_5"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### anger

In [ ]:
anger_true = true.loc[true[0] == 6]
anger_true

,0
5,6
18,6
44,6
59,6
82,6
...,...
3968,6
3969,6
3978,6
3983,6


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
anger_preds = preds.iloc[preds.index.isin(anger_true.index.tolist())]
anger_preds

,0
5,6
18,6
44,6
59,6
82,0
...,...
3968,0
3969,6
3978,5
3983,6


Πριν εκανε 161 λαθη τωρα 204

In [ ]:
diff = anger_true.values != anger_preds.values
result = diff.flatten().sum()
print(result)

204


In [ ]:
anger_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_6'] == 1]
anger_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
5,0,0,0,0,0,0,1,0,/content/train_set/images/100006.jpg
14,0,0,0,0,0,0,1,0,/content/train_set/images/100019.jpg
20,0,0,0,0,0,0,1,0,/content/train_set/images/100026.jpg
21,0,0,0,0,0,0,1,0,/content/train_set/images/100028.jpg
24,0,0,0,0,0,0,1,0,/content/train_set/images/100032.jpg
...,...,...,...,...,...,...,...,...,...
287605,0,0,0,0,0,0,1,0,/content/train_set/images/99926.jpg
287633,0,0,0,0,0,0,1,0,/content/train_set/images/99978.jpg
287638,0,0,0,0,0,0,1,0,/content/train_set/images/99988.jpg
287646,0,0,0,0,0,0,1,0,/content/train_set/images/99997.jpg


In [ ]:
anger = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(anger_old['file_loc'])]
del anger['file_loc']
anger

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
0,3.155668e-03,2.247988e-03,2.417920e-03,7.765699e-03,1.816261e-01,0.008625,0.754863,3.929936e-02
6,1.564592e-03,1.036243e-03,1.427924e-03,2.387381e-03,8.524360e-03,0.017138,0.768286,1.996354e-01
10,1.589019e-04,8.477984e-05,1.068244e-04,2.492992e-04,1.549959e-03,0.000548,0.990315,6.987089e-03
11,5.507155e-03,3.160482e-03,3.255102e-03,1.356957e-02,1.845335e-01,0.007982,0.759499,2.249313e-02
13,3.064386e-02,1.238303e-02,1.328349e-02,3.557101e-02,6.390507e-02,0.024650,0.777006,4.255771e-02
...,...,...,...,...,...,...,...,...
180604,1.626085e-03,1.082964e-03,1.345270e-03,2.935652e-03,1.916403e-02,0.009351,0.820554,1.439406e-01
180622,2.176183e-08,1.175875e-08,2.853949e-08,2.061400e-08,3.909669e-08,0.299996,0.700001,3.047396e-06
180627,4.430425e-10,2.191000e-10,6.889313e-10,3.629408e-10,6.199196e-10,0.300000,0.700000,1.679693e-08
180632,1.968489e-11,1.008304e-11,2.892551e-11,1.697976e-11,2.990605e-11,0.300000,0.700000,1.087163e-09


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [anger['expr_0'].max(),anger['expr_1'].max(),anger['expr_2'].max(),anger['expr_3'].max(),anger['expr_4'].max(),anger['expr_5'].max(),anger['expr_6'].max(),anger['expr_7'].max()]
print("Max values:\n",maxs)
mins = [anger['expr_0'].min(),anger['expr_1'].min(),anger['expr_2'].min(),anger['expr_3'].min(),anger['expr_4'].min(),anger['expr_5'].min(),anger['expr_6'].min(),anger['expr_7'].min()]
print("Min values:\n",mins)
means = [anger['expr_0'].mean(),anger['expr_1'].mean(),anger['expr_2'].mean(),anger['expr_3'].mean(),anger['expr_4'].mean(),anger['expr_5'].mean(),anger['expr_6'].mean(),anger['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.29937461727719233, 0.020140298299689413, 0.2999999999919648, 0.1360817369385703, 0.3, 0.3, 1.0, 0.3]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0]
Mean values:
 [0.008476868350241355, 0.0026443474184773955, 0.005343242926658091, 0.006292179353707805, 0.02611671441817608, 0.07498858089723051, 0.8061707670964844, 0.06996729953902447]


Πριν

In [ ]:
maxs = [anger['expr_0'].max(),anger['expr_1'].max(),anger['expr_2'].max(),anger['expr_3'].max(),anger['expr_4'].max(),anger['expr_5'].max(),anger['expr_6'].max(),anger['expr_7'].max()]
print("Max values:\n",maxs)
mins = [anger['expr_0'].min(),anger['expr_1'].min(),anger['expr_2'].min(),anger['expr_3'].min(),anger['expr_4'].min(),anger['expr_5'].min(),anger['expr_6'].min(),anger['expr_7'].min()]
print("Min values:\n",mins)
means = [anger['expr_0'].mean(),anger['expr_1'].mean(),anger['expr_2'].mean(),anger['expr_3'].mean(),anger['expr_4'].mean(),anger['expr_5'].mean(),anger['expr_6'].mean(),anger['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.2999999999893749, 0.023403256260000806, 0.2999999999733653, 0.29999999905764874, 0.3, 0.3, 1.0, 0.3]
Min values:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0]
Mean values:
 [0.00895019754032473, 0.0024886750373191656, 0.005098688990172643, 0.006038926439817829, 0.02515281120869782, 0.07475110668800462, 0.8022336883729507, 0.07528590572271235]


In [ ]:
anger

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
0,3.155668e-03,2.247988e-03,2.417920e-03,7.765699e-03,1.816261e-01,0.008625,0.754863,3.929936e-02
6,1.564592e-03,1.036243e-03,1.427924e-03,2.387381e-03,8.524360e-03,0.017138,0.768286,1.996354e-01
10,1.589019e-04,8.477984e-05,1.068244e-04,2.492992e-04,1.549959e-03,0.000548,0.990315,6.987089e-03
11,5.507155e-03,3.160482e-03,3.255102e-03,1.356957e-02,1.845335e-01,0.007982,0.759499,2.249313e-02
13,3.064386e-02,1.238303e-02,1.328349e-02,3.557101e-02,6.390507e-02,0.024650,0.777006,4.255771e-02
...,...,...,...,...,...,...,...,...
180604,1.626085e-03,1.082964e-03,1.345270e-03,2.935652e-03,1.916403e-02,0.009351,0.820554,1.439406e-01
180622,2.176183e-08,1.175875e-08,2.853949e-08,2.061400e-08,3.909669e-08,0.299996,0.700001,3.047396e-06
180627,4.430425e-10,2.191000e-10,6.889313e-10,3.629408e-10,6.199196e-10,0.300000,0.700000,1.679693e-08
180632,1.968489e-11,1.008304e-11,2.892551e-11,1.697976e-11,2.990605e-11,0.300000,0.700000,1.087163e-09


In [ ]:
anger["max"]=anger.idxmax(axis=1)
anger

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
0,3.155668e-03,2.247988e-03,2.417920e-03,7.765699e-03,1.816261e-01,0.008625,0.754863,3.929936e-02,expr_6
6,1.564592e-03,1.036243e-03,1.427924e-03,2.387381e-03,8.524360e-03,0.017138,0.768286,1.996354e-01,expr_6
10,1.589019e-04,8.477984e-05,1.068244e-04,2.492992e-04,1.549959e-03,0.000548,0.990315,6.987089e-03,expr_6
11,5.507155e-03,3.160482e-03,3.255102e-03,1.356957e-02,1.845335e-01,0.007982,0.759499,2.249313e-02,expr_6
13,3.064386e-02,1.238303e-02,1.328349e-02,3.557101e-02,6.390507e-02,0.024650,0.777006,4.255771e-02,expr_6
...,...,...,...,...,...,...,...,...,...
180604,1.626085e-03,1.082964e-03,1.345270e-03,2.935652e-03,1.916403e-02,0.009351,0.820554,1.439406e-01,expr_6
180622,2.176183e-08,1.175875e-08,2.853949e-08,2.061400e-08,3.909669e-08,0.299996,0.700001,3.047396e-06,expr_6
180627,4.430425e-10,2.191000e-10,6.889313e-10,3.629408e-10,6.199196e-10,0.300000,0.700000,1.679693e-08,expr_6
180632,1.968489e-11,1.008304e-11,2.892551e-11,1.697976e-11,2.990605e-11,0.300000,0.700000,1.087163e-09,expr_6


In [ ]:
print(anger.loc[anger['max'] != "expr_6"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


#### contempt

In [ ]:
contempt_true = true.loc[true[0] == 7]
contempt_true

,0
11,7
12,7
29,7
42,7
47,7
...,...
3931,7
3971,7
3974,7
3985,7


Το τι ετικέτα έχουν μετά την εκπαίδευση του νευρωνικού αυτές οι εικόνες

In [ ]:
contempt_preds = preds.iloc[preds.index.isin(contempt_true.index.tolist())]
contempt_preds

,0
11,6
12,0
29,0
42,7
47,7
...,...
3931,0
3971,0
3974,0
3985,6


Πριν 390 τωρα 428

In [ ]:
diff = contempt_true.values != contempt_preds.values
result = diff.flatten().sum()
print(result)

428


In [ ]:
contempt_old = one_hot_train_dataset.loc[one_hot_train_dataset['expr_7'] == 1]
contempt_old

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,file_loc
9,0,0,0,0,0,0,0,1,/content/train_set/images/100012.jpg
38,0,0,0,0,0,0,0,1,/content/train_set/images/100047.jpg
123,0,0,0,0,0,0,0,1,/content/train_set/images/10015.jpg
196,0,0,0,0,0,0,0,1,/content/train_set/images/100251.jpg
294,0,0,0,0,0,0,0,1,/content/train_set/images/100368.jpg
...,...,...,...,...,...,...,...,...,...
286493,0,0,0,0,0,0,0,1,/content/train_set/images/98490.jpg
286534,0,0,0,0,0,0,0,1,/content/train_set/images/98540.jpg
286847,0,0,0,0,0,0,0,1,/content/train_set/images/98971.jpg
286978,0,0,0,0,0,0,0,1,/content/train_set/images/99129.jpg


In [ ]:
contempt = new_final_train_dataset.loc[new_final_train_dataset['file_loc'].isin(contempt_old['file_loc'])]
del contempt['file_loc']
contempt

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
3,1.204167e-13,6.284147e-14,1.028213e-13,1.408959e-13,4.419587e-13,5.396617e-12,0.297844,0.702156
23,4.805190e-08,2.779208e-08,4.605788e-08,5.930754e-08,1.732795e-07,5.748458e-06,0.000057,0.999937
73,3.553352e-05,2.121732e-05,3.554916e-05,4.386097e-05,1.186061e-04,4.136149e-03,0.005815,0.989794
117,9.766776e-05,4.250044e-05,7.260668e-05,9.177949e-05,2.249662e-04,1.003276e-03,0.287650,0.710817
181,3.056404e-07,1.758160e-07,3.017302e-07,3.656202e-07,9.998355e-07,6.896684e-05,0.000189,0.999740
...,...,...,...,...,...,...,...,...
179911,8.670439e-07,4.047985e-07,6.447250e-07,9.262043e-07,2.684961e-06,8.959662e-06,0.299650,0.700336
179940,1.573948e-03,8.919728e-04,1.033779e-03,3.013280e-03,3.429433e-02,3.977681e-03,0.227738,0.727477
180128,5.158250e-06,3.015828e-06,5.153833e-06,6.227199e-06,1.677144e-05,9.736632e-04,0.001507,0.997483
180215,4.364739e-06,2.010112e-06,3.198117e-06,4.592754e-06,1.304853e-05,3.996883e-05,0.298932,0.701001


Θέλουμε να βρούμε για κάθε στήλη (για τα 8 συναισθήματα) την max,min,average τιμή της.

In [ ]:
maxs = [contempt['expr_0'].max(),contempt['expr_1'].max(),contempt['expr_2'].max(),contempt['expr_3'].max(),contempt['expr_4'].max(),contempt['expr_5'].max(),contempt['expr_6'].max(),contempt['expr_7'].max()]
print("Max values:\n",maxs)
mins = [contempt['expr_0'].min(),contempt['expr_1'].min(),contempt['expr_2'].min(),contempt['expr_3'].min(),contempt['expr_4'].min(),contempt['expr_5'].min(),contempt['expr_6'].min(),contempt['expr_7'].min()]
print("Min values:\n",mins)
means = [contempt['expr_0'].mean(),contempt['expr_1'].mean(),contempt['expr_2'].mean(),contempt['expr_3'].mean(),contempt['expr_4'].mean(),contempt['expr_5'].mean(),contempt['expr_6'].mean(),contempt['expr_7'].mean()]
print("Mean values:\n",means)

Max values:
 [0.26631837789590374, 0.020842614416546203, 0.08940806036890979, 0.06354773890774176, 0.3, 0.29999999999999144, 0.3, 1.0]
Min values:
 [8.573428890363822e-115, 4.660723517684566e-115, 7.727103878086789e-115, 1.0234508848436001e-114, 3.140581598337042e-114, 7.606519292788288e-113, 9.376089424144167e-109, 0.7]
Mean values:
 [0.003813873676664684, 0.000996350962188599, 0.001572662706486791, 0.0022314522813821056, 0.01183141813197387, 0.012683660510207246, 0.0729381601522172, 0.8939324215788794]


In [ ]:
contempt

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
3,1.204167e-13,6.284147e-14,1.028213e-13,1.408959e-13,4.419587e-13,5.396617e-12,0.297844,0.702156
23,4.805190e-08,2.779208e-08,4.605788e-08,5.930754e-08,1.732795e-07,5.748458e-06,0.000057,0.999937
73,3.553352e-05,2.121732e-05,3.554916e-05,4.386097e-05,1.186061e-04,4.136149e-03,0.005815,0.989794
117,9.766776e-05,4.250044e-05,7.260668e-05,9.177949e-05,2.249662e-04,1.003276e-03,0.287650,0.710817
181,3.056404e-07,1.758160e-07,3.017302e-07,3.656202e-07,9.998355e-07,6.896684e-05,0.000189,0.999740
...,...,...,...,...,...,...,...,...
179911,8.670439e-07,4.047985e-07,6.447250e-07,9.262043e-07,2.684961e-06,8.959662e-06,0.299650,0.700336
179940,1.573948e-03,8.919728e-04,1.033779e-03,3.013280e-03,3.429433e-02,3.977681e-03,0.227738,0.727477
180128,5.158250e-06,3.015828e-06,5.153833e-06,6.227199e-06,1.677144e-05,9.736632e-04,0.001507,0.997483
180215,4.364739e-06,2.010112e-06,3.198117e-06,4.592754e-06,1.304853e-05,3.996883e-05,0.298932,0.701001


In [ ]:
contempt["max"]=contempt.idxmax(axis=1)
contempt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7,max
3,1.204167e-13,6.284147e-14,1.028213e-13,1.408959e-13,4.419587e-13,5.396617e-12,0.297844,0.702156,expr_7
23,4.805190e-08,2.779208e-08,4.605788e-08,5.930754e-08,1.732795e-07,5.748458e-06,0.000057,0.999937,expr_7
73,3.553352e-05,2.121732e-05,3.554916e-05,4.386097e-05,1.186061e-04,4.136149e-03,0.005815,0.989794,expr_7
117,9.766776e-05,4.250044e-05,7.260668e-05,9.177949e-05,2.249662e-04,1.003276e-03,0.287650,0.710817,expr_7
181,3.056404e-07,1.758160e-07,3.017302e-07,3.656202e-07,9.998355e-07,6.896684e-05,0.000189,0.999740,expr_7
...,...,...,...,...,...,...,...,...,...
179911,8.670439e-07,4.047985e-07,6.447250e-07,9.262043e-07,2.684961e-06,8.959662e-06,0.299650,0.700336,expr_7
179940,1.573948e-03,8.919728e-04,1.033779e-03,3.013280e-03,3.429433e-02,3.977681e-03,0.227738,0.727477,expr_7
180128,5.158250e-06,3.015828e-06,5.153833e-06,6.227199e-06,1.677144e-05,9.736632e-04,0.001507,0.997483,expr_7
180215,4.364739e-06,2.010112e-06,3.198117e-06,4.592754e-06,1.304853e-05,3.996883e-05,0.298932,0.701001,expr_7


In [ ]:
print(contempt.loc[contempt['max'] != "expr_7"])

Empty DataFrame
Columns: [expr_0, expr_1, expr_2, expr_3, expr_4, expr_5, expr_6, expr_7, max]
Index: []


# Data cleaning + Label smoothing + Data cleaning

In [ ]:
new_final_train_dataset = pd.read_pickle("./new_final_train_dataset_l_0.7.pkl")  
valence_list = new_final_train_dataset["valence"]
arousal_list = new_final_train_dataset["arousal"]
file_loc = new_final_train_dataset["file_loc"]
del new_final_train_dataset["valence"]
del new_final_train_dataset["arousal"]
del new_final_train_dataset["file_loc"]
new_final_train_dataset

,expr_0,expr_1,expr_2,expr_3,expr_4,expr_5,expr_6,expr_7
0,3.155668e-03,2.247988e-03,2.417920e-03,7.765699e-03,1.816261e-01,8.624640e-03,7.548626e-01,3.929936e-02
1,1.733792e-03,1.366213e-03,6.419450e-04,9.874891e-01,5.195128e-03,8.789741e-04,1.498368e-03,1.196509e-03
2,3.112717e-64,2.256209e-64,1.947586e-64,2.658545e-63,1.000000e+00,4.184243e-64,1.442721e-63,9.283491e-64
3,1.204167e-13,6.284147e-14,1.028213e-13,1.408959e-13,4.419587e-13,5.396617e-12,2.978442e-01,7.021558e-01
4,9.998074e-01,2.560515e-05,3.522816e-05,2.647022e-05,2.519462e-05,2.548422e-05,2.855184e-05,2.607586e-05
...,...,...,...,...,...,...,...,...
180632,1.968489e-11,1.008304e-11,2.892551e-11,1.697976e-11,2.990605e-11,3.000000e-01,7.000000e-01,1.087163e-09
180633,1.000000e+00,1.656510e-15,1.638311e-15,1.672561e-15,1.495339e-15,1.360045e-15,1.589411e-15,1.435222e-15
180634,2.798399e-11,1.332234e-11,4.103933e-11,2.242628e-11,3.899706e-11,3.000000e-01,7.000000e-01,1.483784e-09
180635,5.636639e-03,1.355110e-02,2.430235e-03,9.569854e-01,1.027215e-02,3.026335e-03,4.326163e-03,3.772024e-03


In [ ]:
new_final_train_dataset['expr'] = new_final_train_dataset.idxmax(axis=1)
new_final_train_dataset['expr'] = new_final_train_dataset['expr'].map(lambda x: x.lstrip('expr_'))
del new_final_train_dataset['expr_0']
del new_final_train_dataset['expr_1']
del new_final_train_dataset['expr_2']
del new_final_train_dataset['expr_3']
del new_final_train_dataset['expr_4']
del new_final_train_dataset['expr_5']
del new_final_train_dataset['expr_6']
del new_final_train_dataset['expr_7']
expr_list = new_final_train_dataset['expr']
new_final_train_dataset

,expr
0,6
1,3
2,4
3,7
4,0
...,...
180632,6
180633,0
180634,6
180635,3


In [ ]:
new_final_train_dataset = pd.DataFrame(list(zip(file_loc, expr_list, valence_list, arousal_list)),
              columns=['file_loc','expr','valence','arousal'])
new_final_train_dataset

,file_loc,expr,valence,arousal
0,/content/train_set/images/100006.jpg,6,-0.341270,0.833333
1,/content/train_set/images/100008.jpg,3,0.150966,0.534852
2,/content/train_set/images/100009.jpg,4,-0.126422,0.758533
3,/content/train_set/images/100012.jpg,7,-0.493612,0.561363
4,/content/train_set/images/100013.jpg,0,-0.067751,-0.043554
...,...,...,...,...
180632,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
180633,/content/train_set/images/99998.jpg,0,0.017253,0.012940
180634,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
180635,/content/train_set/images/9999.jpg,3,0.406504,0.609756


In [ ]:
new_final_train_dataset

,file_loc,expr,valence,arousal
0,/content/train_set/images/100006.jpg,6,-0.341270,0.833333
1,/content/train_set/images/100008.jpg,3,0.150966,0.534852
2,/content/train_set/images/100009.jpg,4,-0.126422,0.758533
3,/content/train_set/images/100012.jpg,7,-0.493612,0.561363
4,/content/train_set/images/100013.jpg,0,-0.067751,-0.043554
...,...,...,...,...
180632,/content/train_set/images/99997.jpg,6,-0.753266,0.447745
180633,/content/train_set/images/99998.jpg,0,0.017253,0.012940
180634,/content/train_set/images/99999.jpg,6,-0.695322,0.400337
180635,/content/train_set/images/9999.jpg,3,0.406504,0.609756


Θέλουμε να εφαρμόσουμε μία μορφή data cleaning. Αρχικά παίρνουμε τα true labels και κρατάμε όσα είναι happy.

### happy

#### va > 0

In [ ]:
happy = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '1']
happy

,file_loc,expr,valence,arousal
5,/content/train_set/images/100015.jpg,1,0.777047,0.358637
8,/content/train_set/images/100024.jpg,1,0.510204,0.162982
14,/content/train_set/images/100033.jpg,1,0.507936,0.103175
15,/content/train_set/images/100034.jpg,1,0.759775,0.232288
16,/content/train_set/images/100038.jpg,1,0.500000,0.111111
...,...,...,...,...
180614,/content/train_set/images/99962.jpg,1,0.697171,0.193798
180616,/content/train_set/images/9996.jpg,1,0.492063,0.134921
180628,/content/train_set/images/99990.jpg,1,0.754936,0.116144
180631,/content/train_set/images/99995.jpg,1,0.431943,0.163295


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι και οι δύο θετικές. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
happy_wrong_1 = happy.loc[(happy['valence'] <= 0) | (happy['arousal'] <= 0)]
happy_wrong_1

,file_loc,expr,valence,arousal


#### va > 0.1

In [ ]:
happy = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '1']
happy

,file_loc,expr,valence,arousal
5,/content/train_set/images/100015.jpg,1,0.777047,0.358637
8,/content/train_set/images/100024.jpg,1,0.510204,0.162982
14,/content/train_set/images/100033.jpg,1,0.507936,0.103175
15,/content/train_set/images/100034.jpg,1,0.759775,0.232288
16,/content/train_set/images/100038.jpg,1,0.500000,0.111111
...,...,...,...,...
180614,/content/train_set/images/99962.jpg,1,0.697171,0.193798
180616,/content/train_set/images/9996.jpg,1,0.492063,0.134921
180628,/content/train_set/images/99990.jpg,1,0.754936,0.116144
180631,/content/train_set/images/99995.jpg,1,0.431943,0.163295


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι και οι δύο θετικές. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
happy_wrong_2 = happy.loc[(happy['valence'] <= 0.1) | (happy['arousal'] <= 0.1)]
happy_wrong_2

,file_loc,expr,valence,arousal


### neutral

In [ ]:
neutral = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '0']
neutral

,file_loc,expr,valence,arousal
4,/content/train_set/images/100013.jpg,0,-0.067751,-0.043554
9,/content/train_set/images/100025.jpg,0,-0.095238,-0.079365
12,/content/train_set/images/100031.jpg,0,-0.047619,-0.071429
24,/content/train_set/images/100048.jpg,0,0.023810,-0.015873
30,/content/train_set/images/10005.jpg,0,0.054792,-0.064754
...,...,...,...,...
180624,/content/train_set/images/99981.jpg,0,-0.004313,0.056073
180625,/content/train_set/images/99982.jpg,0,-0.058072,-0.091947
180626,/content/train_set/images/99984.jpg,0,-0.079365,-0.055556
180630,/content/train_set/images/99994.jpg,0,0.000000,0.000000


Θέλουμε να δούμε αν οι τιμές valence και arousal είναι ανάμεσα στο -0.1 και στο 0.1. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
neutral_wrong = neutral.loc[((neutral['valence'] > 0.1) | (neutral['valence'] < -0.1)) | ((neutral['arousal'] > 0.1) | (neutral['arousal'] < -0.1))]
neutral_wrong

,file_loc,expr,valence,arousal


### sad

In [ ]:
sad = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '2']
sad

,file_loc,expr,valence,arousal
7,/content/train_set/images/100023.jpg,2,-0.880952,-0.428571
17,/content/train_set/images/100039.jpg,2,-0.492063,-0.277778
22,/content/train_set/images/100045.jpg,2,-0.928571,-0.126984
26,/content/train_set/images/100052.jpg,2,-0.444444,-0.158730
29,/content/train_set/images/100058.jpg,2,-0.508130,-0.208091
...,...,...,...,...
180591,/content/train_set/images/99899.jpg,2,-0.759775,-0.348432
180619,/content/train_set/images/99973.jpg,2,-0.747508,-0.256091
180620,/content/train_set/images/99975.jpg,2,-0.849206,-0.357143
180621,/content/train_set/images/99977.jpg,2,-0.580720,-0.208091


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
sad_wrong = sad.loc[sad['valence'] >= 0]
sad_wrong

,file_loc,expr,valence,arousal


Άρα έχουμε 5 εικόνες που είναι annotated ως sad ενώ δεν θα έπρεπε.

### surprise -> not deleted

In [ ]:
surprise = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '3']
surprise

,file_loc,expr,valence,arousal
1,/content/train_set/images/100008.jpg,3,0.150966,0.534852
18,/content/train_set/images/100040.jpg,3,0.280681,0.692025
20,/content/train_set/images/100042.jpg,3,-0.048393,0.871080
38,/content/train_set/images/100077.jpg,3,0.211353,0.659938
53,/content/train_set/images/100113.jpg,3,0.329075,0.687185
...,...,...,...,...
180555,/content/train_set/images/99817.jpg,3,0.158562,0.649351
180562,/content/train_set/images/9982.jpg,3,0.145180,0.551684
180587,/content/train_set/images/9988.jpg,3,0.353656,0.647539
180603,/content/train_set/images/99925.jpg,3,-0.190476,0.920635


Θέλουμε να δούμε αν οι τιμές arousal είναι > 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
surprise_wrong = surprise.loc[surprise['arousal'] <= 0]
surprise_wrong

,file_loc,expr,valence,arousal
13551,/content/train_set/images/128202.jpg,3,-0.452381,-0.285714
20996,/content/train_set/images/143623.jpg,3,-0.444444,-0.246032
30730,/content/train_set/images/163672.jpg,3,-0.174603,-0.420635
38547,/content/train_set/images/179855.jpg,3,-0.759775,-0.033875
54065,/content/train_set/images/21205.jpg,3,-0.555556,-0.285714
60506,/content/train_set/images/225289.jpg,3,-0.857143,-0.309524
68231,/content/train_set/images/241081.jpg,3,-0.311462,-0.173034
99666,/content/train_set/images/306168.jpg,3,0.895277,-0.058072
103435,/content/train_set/images/314021.jpg,3,-0.493612,-0.014518
106589,/content/train_set/images/320443.jpg,3,-0.444444,-0.253968


Άρα έχουμε 20 εικόνες που είναι annotated ως surprise ενώ δεν θα έπρεπε.

### fear

In [ ]:
fear = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '4']
fear

,file_loc,expr,valence,arousal
2,/content/train_set/images/100009.jpg,4,-0.126422,0.758533
52,/content/train_set/images/100112.jpg,4,-0.116144,0.754936
103,/content/train_set/images/10022.jpg,4,-0.183895,0.740418
155,/content/train_set/images/100315.jpg,4,-0.095238,0.642857
209,/content/train_set/images/100424.jpg,4,-0.069138,0.717310
...,...,...,...,...
180539,/content/train_set/images/99783.jpg,4,-0.139470,0.876669
180570,/content/train_set/images/99852.jpg,4,-0.091947,0.958188
180588,/content/train_set/images/99891.jpg,4,-0.028088,0.661136
180617,/content/train_set/images/99970.jpg,4,-0.095238,0.777778


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
fear_wrong = fear.loc[fear['valence'] >= 0]
fear_wrong

,file_loc,expr,valence,arousal


### disgust

In [ ]:
disgust = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '5']
disgust

,file_loc,expr,valence,arousal
21,/content/train_set/images/100044.jpg,5,-0.725900,0.391986
57,/content/train_set/images/100119.jpg,5,-0.841270,0.468254
63,/content/train_set/images/100128.jpg,5,-0.571429,0.492063
84,/content/train_set/images/100186.jpg,5,-0.761905,0.460317
101,/content/train_set/images/100223.jpg,5,-0.743187,0.502340
...,...,...,...,...
180354,/content/train_set/images/99421.jpg,5,-0.740418,0.474255
180448,/content/train_set/images/99612.jpg,5,-0.849206,0.436508
180530,/content/train_set/images/99770.jpg,5,-0.769841,0.452381
180540,/content/train_set/images/99784.jpg,5,-0.706349,0.428571


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
disgust_wrong = disgust.loc[disgust['valence'] >= 0]
disgust_wrong

,file_loc,expr,valence,arousal


Άρα έχουμε 8 εικόνες που είναι annotated ως disgust ενώ δεν θα έπρεπε.

### anger

In [ ]:
anger = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '6']
anger

,file_loc,expr,valence,arousal
0,/content/train_set/images/100006.jpg,6,-0.341270,0.833333
6,/content/train_set/images/100019.jpg,6,-0.578013,0.807686
10,/content/train_set/images/100026.jpg,6,-0.333732,0.577804
11,/content/train_set/images/100028.jpg,6,-0.205255,0.589837
13,/content/train_set/images/100032.jpg,6,-0.168219,0.431332
...,...,...,...,...
180604,/content/train_set/images/99926.jpg,6,-0.460317,0.793651
180622,/content/train_set/images/99978.jpg,6,-0.750097,0.517809
180627,/content/train_set/images/99988.jpg,6,-0.742731,0.410872
180632,/content/train_set/images/99997.jpg,6,-0.753266,0.447745


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
anger_wrong = anger.loc[anger['valence'] >= 0]
anger_wrong

,file_loc,expr,valence,arousal


Άρα έχουμε 7 εικόνες που είναι annotated ως anger ενώ δεν θα έπρεπε.

### contempt

In [ ]:
contempt = new_final_train_dataset.loc[new_final_train_dataset['expr'] == '7']
contempt

,file_loc,expr,valence,arousal
3,/content/train_set/images/100012.jpg,7,-0.493612,0.561363
23,/content/train_set/images/100047.jpg,7,-0.590399,0.643631
73,/content/train_set/images/10015.jpg,7,-0.637379,0.674252
117,/content/train_set/images/100251.jpg,7,-0.421022,0.454897
181,/content/train_set/images/100368.jpg,7,-0.611111,0.634921
...,...,...,...,...
179911,/content/train_set/images/98490.jpg,7,-0.417799,0.495822
179940,/content/train_set/images/98540.jpg,7,-0.295199,0.609756
180128,/content/train_set/images/98971.jpg,7,-0.626984,0.650794
180215,/content/train_set/images/99129.jpg,7,-0.411343,0.488773


Θέλουμε να δούμε αν οι τιμές valence είναι < 0. Σε όσες δεν είναι κρατάμε τα image locations για να τις διαγράψουμε μετά.

In [ ]:
contempt_wrong = contempt.loc[contempt['valence'] >= 0]
contempt_wrong

,file_loc,expr,valence,arousal


Άρα έχουμε 109 εικόνες που είναι annotated ως contempt ενώ δεν θα έπρεπε.